# 🏢 Workday Health Reasoning Platform

## Overview

**Sedentary work is one of the most underestimated modern health risks.**  
The World Health Organization reports that **31% of adults worldwide (≈1.8 billion people)** are insufficiently physically active — a pattern strongly linked to chronic disease risk and premature mortality. WHO also warns that insufficient physical activity is associated with a **20–30% higher risk of death**, and recommends actively reducing sedentary time throughout the day.  

At the same time, long screen exposure has become the default. Digital Eye Strain (Computer Vision Syndrome) is widely reported among screen users, causing symptoms such as **dry eyes, blurred vision, headaches, and difficulty focusing**, especially when workdays are built around uninterrupted monitor time.

The result is a predictable cycle for desk workers:  
**back and neck pain → fatigue → low productivity → stress → worse sleep → repeat.**

The **Workday Health Reasoning Platform** is a **privacy-first, local-only, context-aware preventive health assistant** designed specifically for office and remote desk workers.  
It helps users reflect on their daily workplace behaviors (posture, hydration, stress, eye strain, productivity, musculoskeletal symptoms, sleep recovery, and longitudinal lab trends) and then produces **clear, non-diagnostic, evidence-aligned recommendations**.

This platform is not a symptom checker and does not attempt to replace clinicians.  
Instead, it acts as a **structured reasoning layer** between everyday work habits and actionable prevention — translating scattered inputs into practical steps, reminders, and habit nudges.

---

## 🎯 Problem & Target Users

### The problem
Desk work concentrates multiple risk factors into a single routine:
- prolonged sitting and low movement,
- poor posture and repetitive strain,
- screen-based eye fatigue and headaches,
- mental overload, stress, and burnout patterns,
- inconsistent hydration, recovery, and sleep.

Many existing tools either:
- collect health data without generating meaningful guidance, or
- rely on cloud-based processing that compromises privacy and user control.

### Target users
- Office and remote desk workers  
- Knowledge workers with long screen hours  
- Professionals with chronic fatigue, posture pain, or eye strain  
- Health-conscious users who value **privacy, autonomy, and clarity**

---

## ✨ Key Features

- **Modular Health Tabs**  
  Baseline, Workspace, MSK, Eye, Mental, Hydration, Productivity, Recovery/Sleep, Checklist, Reminders.

- **Local-Only Storage (Privacy by Design)**  
  All user data is stored locally as JSON — no automatic uploads, no hidden telemetry.

- **Context-Aware Reasoning**  
  The system builds a cross-domain context from real user inputs, allowing the assistant to detect patterns (e.g., hydration + headaches + screen time + poor sleep).

- **Safety Guardrails**  
  Explicit checks for high-risk signals (e.g., severe symptoms, mental distress keywords, extreme values) to trigger urgent warnings.

- **Optional AI Integration (Graceful Fallback)**  
  Uses a language model only when configured; otherwise runs locally with safe placeholder logic.

- **Actionable Output, Not Just Tracking**  
  Generates clear recommendations plus structured **tasks and reminders** that convert insight into daily behavior change.

- **Internationalization (i18n)**  
  Built-in multilingual support (English & Arabic included), designed for global deployment.

---

## 🧠 Architecture & Data

The platform is built around a simple but powerful principle:

**Collect structured user inputs → build contextual memory → apply reasoning → output practical prevention guidance.**

Each module stores user entries and AI outputs in domain-specific JSON files.  
A lightweight shared context layer allows cross-domain summaries without exposing raw histories, enabling explainable reasoning while maintaining performance and privacy.

This notebook is a **self-contained, executable skeleton** demonstrating:
- how user health inputs are collected,
- how context is constructed across domains,
- how safe reasoning is applied,
- and how the full system runs locally with optional AI support.

---

## 🌍 Why This Matters

Workplace health is not only a medical issue — it is a productivity and sustainability issue.  
Preventable desk-work habits contribute to pain, fatigue, reduced concentration, and long-term chronic risk.  
By turning routine behaviors into actionable guidance, this platform helps individuals regain control over their health **without giving up their data**.

The goal is simple:

**help desk workers build healthier workdays — one small decision at a time.**



**Key design choice:**  
The AI never sees raw historical logs unless explicitly composed — it only receives **curated, summarized context**.

---

## 🔐 Safety & Privacy Principles

- **Local-first by default**  
  No data leaves the device unless the user enables an AI model.

- **Non-diagnostic outputs**  
  All recommendations are preventive and informational.

- **Explicit safety flags**  
  When concerning signals appear (e.g., self-harm keywords), the system prioritizes clear warnings and encourages external help.

- **User control**  
  Context can be viewed, reset, or fully deleted at any time.

---

## ▶️ How to Run This Notebook

1. Ensure Python 3.10+ is available  
2. (Optional) Create a `.env` file for AI usage:
   ```text
   GEMINI_API_KEY=your_key_here


## Environment Setup & Dependencies

This cell installs the Python packages required to run the notebook.
It ensures a consistent execution environment across different machines,
including local development, review, and evaluation.

The dependencies include:
- The UI framework used to build the interactive application
- The generative AI client used for model calls
- Supporting libraries for configuration and HTTP requests

This installation step is included to make the notebook **self-contained**
and easy to run without prior environment preparation.

In a production setup, these dependencies would typically be managed using
a virtual environment and installed via a standard workflow (e.g. `pip install -r requirements.txt`
in an `app.py`-based project). For this notebook-based submission, installing
them inline prioritizes reproducibility and ease of evaluation.


In [1]:
### Setup

#Install dependencies before running the notebook:

%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Core Imports & Notebook Execution Model

This cell defines the core Python imports used throughout the notebook.
Because Jupyter notebooks share a single execution kernel, running this cell
once makes these imports available to all subsequent cells.

Some individual cells also repeat imports intentionally. This is done to ensure
that each section can be executed independently without relying on execution
order—a common requirement during review, debugging, or evaluation.

For this notebook-based submission, redundancy is preferred over hidden
dependencies to maximize robustness and reproducibility.

In a production or deployment setting, this structure can be easily refactored
into a standard `app.py` and supporting `.py` modules, where imports would be
centralized and managed conventionally. The current approach prioritizes
clarity and safe execution within a notebook environment.


In [2]:
# =========================================================
# Core Imports (run once at the top of the notebook)
# =========================================================

from __future__ import annotations

import json
import uuid
import time
import threading

from datetime import datetime, timedelta
from pathlib import Path
from typing import List, Dict, Any, Optional

import gradio as gr


In [3]:

import gradio as gr

gr.Markdown("""
# 🧠 Workday Health Reasoning Platform

⚠️ **Important Disclaimer**
This tool provides **general wellness and preventive guidance only**.  
It does **not** provide medical diagnosis or treatment advice.  
If you have severe symptoms or an emergency, seek professional medical help immediately.
""")

In [4]:
# ==========================================
# DEMO ENGINE (Gradual + Input Preview)
# ==========================================

import json
import time
from typing import Optional, List, Tuple
import pandas as pd
from dataclasses import dataclass
from typing import Callable, Dict

# Global hook registry
_DEMO_HOOKS: Dict[str, "DemoHook"] = {}

@dataclass
class DemoHook:
    domain: str
    label: str
    generate_fn: Callable


def register_demo_hook(domain: str, label: str, generate_fn: Callable):
    """Modules call this to register themselves for the demo button."""
    _DEMO_HOOKS[domain] = DemoHook(
        domain=domain,
        label=label,
        generate_fn=generate_fn
    )


def _load_latest_user_input(store, user_file: str):
    raw = store.load_json(user_file, [])
    if not isinstance(raw, list) or not raw:
        return None
    last = raw[-1]
    if not isinstance(last, dict):
        return None
    return last.get("user_input", None)


def _preview_dict(d: dict, max_items: int = 10) -> str:
    if not isinstance(d, dict) or not d:
        return "(No input data found)"

    items = list(d.items())[:max_items]
    lines = []
    for k, v in items:
        lines.append(f"- **{k}**: {v}")

    if len(d) > max_items:
        lines.append(f"- … _(and {len(d)-max_items} more fields)_")

    return "\n".join(lines)


def run_full_demo_gradual(
    domains_order: Optional[List[str]] = None,
    delay_seconds: float = 0.7
) -> Tuple[str, str, Optional[pd.DataFrame]]:
    """
    Same demo as before, but returns:
    - report_md (full report)
    - status_md
    - schedule_df
    """

    if "_DEMO_HOOKS" not in globals() or not _DEMO_HOOKS:
        return "", "❌ No demo hooks registered yet.", None

    store = globals().get("store")
    if store is None:
        return "", "❌ Store not found. Run setup cells first.", None

    if domains_order is None:
        domains_order = [
            "general_recommendations",
            "ask_ai",
            "baseline",
            "workspace",
            "longitudinal",
            "msk",
            "eye",
            "mental",
            "hydration",
            "productivity",
            "recovery_sleep",
        ]

    parts = []
    status_lines = []

    parts.append("# 🚀 Guided Demo Walkthrough")
    parts.append("This demo runs module-by-module and shows what the demo user entered before generating AI output.\n")

    for dom in domains_order:
        hook = _DEMO_HOOKS.get(dom)
        if not hook:
            status_lines.append(f"⚠️ {dom}: not registered")
            continue

        parts.append("\n---")
        parts.append(f"## {hook.label}")

        # Try to show saved input (if file exists)
        # We assume hook.domain matches file name pattern
        user_file_guess = f"{dom}_user_input.json"
        demo_input = None
        try:
            demo_input = _load_latest_user_input(store, user_file_guess)
        except Exception:
            demo_input = None

        if demo_input:
            parts.append("### 👤 Demo User Entered")
            parts.append(_preview_dict(demo_input, max_items=10))
        else:
            parts.append("### 👤 Demo User Entered")
            parts.append("_(No saved input found for this module — using general guidance if possible.)_")

        # Now generate AI output
        try:
            result = hook.generate_fn()

            if isinstance(result, tuple) and len(result) >= 1:
                output_text = result[0]
            else:
                output_text = result

            output_text = str(output_text or "").strip()
            if not output_text:
                output_text = "_(No output returned)_"

            parts.append("\n### 🤖 AI Output")
            parts.append(output_text)

            status_lines.append(f"✅ {hook.label}: generated")

        except Exception as e:
            parts.append(f"\n❌ Generate failed: `{e}`")
            status_lines.append(f"❌ {hook.label}: failed")

        # Delay between modules (feels gradual)
        time.sleep(delay_seconds)

    # Reminder schedule preview
    schedule_df = None
    try:
        sched = store.load_json("reminder_schedule.json", {})
        if isinstance(sched, dict) and isinstance(sched.get("schedule"), list):
            schedule_df = pd.DataFrame(sched["schedule"])
    except Exception:
        schedule_df = None

    report_md = "\n".join(parts)
    status_md = "## Demo Status\n" + "\n".join(f"- {s}" for s in status_lines)

    return report_md, status_md, schedule_df


In [5]:
# ==========================================
# 🚀 Notebook Demo Button (TRUE gradual output) — FIXED STORE RESOLUTION
# ==========================================

import time
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
import pandas as pd
from pathlib import Path

# Ensure hooks dict exists even if demo engine cell hasn't run yet
if "_DEMO_HOOKS" not in globals():
    _DEMO_HOOKS = {}

# --- UI Widgets ---
demo_btn = widgets.Button(
    description="🚀 Run Guided Demo",
    button_style="success"
)

refresh_btn = widgets.Button(
    description="🔄 Refresh Hooks",
    button_style="info"
)

clear_btn = widgets.Button(
    description="🧹 Clear Output",
    button_style=""
)

out = widgets.Output()


# =========================================================
# Store Resolver (FIX)
# =========================================================
def _resolve_store():
    """
    Attempts to find a working LocalStore instance.
    If missing, it creates one using DATA_DIR or ./data.
    Does NOT change any app logic.
    """

    # 1) Use existing global store if present
    store_obj = globals().get("store")
    if store_obj is not None:
        return store_obj

    # 2) Try to build it if LocalStore exists
    LocalStore_cls = globals().get("LocalStore")
    if LocalStore_cls is None:
        return None

    # 3) Determine data directory
    data_dir = globals().get("DATA_DIR")
    if data_dir is None:
        data_dir = Path("data")

    try:
        data_dir = Path(data_dir)
        data_dir.mkdir(parents=True, exist_ok=True)

        store_obj = LocalStore_cls(data_dir)

        # Save globally so all hooks see it
        globals()["store"] = store_obj
        return store_obj

    except Exception as e:
        print(f"[demo] ❌ Failed to create store: {e}")
        return None


def show_registered_hooks():
    if "_DEMO_HOOKS" not in globals():
        return "❌ `_DEMO_HOOKS` is not defined yet."

    if not _DEMO_HOOKS:
        return "⚠️ No hooks registered yet."

    lines = ["✅ Registered hooks:"]
    for k in sorted(_DEMO_HOOKS.keys()):
        lines.append(f"- **{k}**")
    return "\n".join(lines)


def _load_latest_user_input(store, user_file: str):
    raw = store.load_json(user_file, [])
    if not isinstance(raw, list) or not raw:
        return None
    last = raw[-1]
    if not isinstance(last, dict):
        return None
    return last.get("user_input", None)


def _preview_dict(d: dict, max_items: int = 10) -> str:
    if not isinstance(d, dict) or not d:
        return "(No input data found)"

    items = list(d.items())[:max_items]
    lines = []
    for k, v in items:
        lines.append(f"- **{k}**: {v}")

    if len(d) > max_items:
        lines.append(f"- … _(and {len(d)-max_items} more fields)_")

    return "\n".join(lines)


def run_demo_notebook_gradual():
    with out:
        clear_output(wait=True)

        # ✅ FIXED: auto resolve store
        store = _resolve_store()
        if store is None:
            display(Markdown(
                "❌ `store` not found and could not be created.\n\n"
                "Make sure the cell defining `LocalStore` and `DATA_DIR` has been executed."
            ))
            return

        if "_DEMO_HOOKS" not in globals() or not _DEMO_HOOKS:
            display(Markdown(
                "⚠️ Demo hooks are not registered yet.\n\n"
                "➡️ Run all module cells (Run All), then click again."
            ))
            display(Markdown(show_registered_hooks()))
            return

        domains_order = [
            "general_recommendations",
            "ask_ai",
            "baseline",
            "workspace",
            "longitudinal",
            "msk",
            "eye",
            "mental",
            "hydration",
            "productivity",
            "recovery_sleep",
        ]

        display(Markdown("# 🚀 Guided Demo Walkthrough"))
        display(Markdown("Generating module-by-module (output appears gradually)."))
        display(Markdown(show_registered_hooks()))
        display(Markdown("---"))

        status_lines = []

        for dom in domains_order:
            hook = _DEMO_HOOKS.get(dom)
            if not hook:
                status_lines.append(f"⚠️ {dom}: not registered")
                continue

            display(Markdown(f"## {hook.label}"))

            # Show saved input preview
            user_file_guess = f"{dom}_user_input.json"
            demo_input = None
            try:
                demo_input = _load_latest_user_input(store, user_file_guess)
            except Exception:
                demo_input = None

            display(Markdown("### 👤 Demo User Entered"))
            if demo_input:
                display(Markdown(_preview_dict(demo_input, max_items=10)))
            else:
                display(Markdown("_(No saved input found for this module.)_"))

            # Generate AI output
            display(Markdown("### 🤖 AI Output"))
            try:
                result = hook.generate_fn()

                if isinstance(result, tuple) and len(result) >= 1:
                    output_text = result[0]
                else:
                    output_text = result

                output_text = str(output_text or "").strip()
                if not output_text:
                    output_text = "_(No output returned)_"

                display(Markdown(output_text))
                status_lines.append(f"✅ {hook.label}: generated")

            except Exception as e:
                display(Markdown(f"❌ Generate failed: `{e}`"))
                status_lines.append(f"❌ {hook.label}: failed")

            display(Markdown("---"))
            time.sleep(0.4)

        # Show reminder schedule preview at end
        try:
            sched = store.load_json("reminder_schedule.json", {})
            if isinstance(sched, dict) and isinstance(sched.get("schedule"), list):
                df = pd.DataFrame(sched["schedule"])
                if not df.empty:
                    display(Markdown("## 📅 Reminder Schedule Preview"))
                    display(df)
        except Exception:
            pass

        # Final status summary
        display(Markdown("## ✅ Demo Status Summary"))
        display(Markdown("\n".join([f"- {s}" for s in status_lines])))


def refresh_hooks_display():
    with out:
        clear_output(wait=True)
        display(Markdown("## 🔄 Hook Status"))
        display(Markdown(show_registered_hooks()))


def clear_demo_output():
    with out:
        clear_output(wait=True)


# --- Wire Buttons ---
demo_btn.on_click(lambda _: run_demo_notebook_gradual())
refresh_btn.on_click(lambda _: refresh_hooks_display())
clear_btn.on_click(lambda _: clear_demo_output())


# --- Display Notebook Controls ---
display(widgets.HBox([demo_btn, refresh_btn, clear_btn]))
display(out)

# Show current hook status immediately
refresh_hooks_display()


Output()

In [6]:
import os
import sys
from pathlib import Path

# -------------------------------------------------
# 📂 Path Fixing Logic (Ensures visibility)
# -------------------------------------------------

# 1. Get the directory of the current notebook/script
try:
    # Works in standard Python scripts
    base_path = Path(__file__).resolve().parent
except NameError:
    # Works in Jupyter/VS Code Notebooks
    base_path = Path(os.getcwd()).resolve()

# 2. Force DATA_DIR to be a folder named 'data' in your project folder
DATA_DIR = base_path / "data"

# Create the directories
DATA_DIR.mkdir(parents=True, exist_ok=True)
LOCALES_DIR = base_path / "locales"
HELP_DIR = base_path / "help"

LOCALES_DIR.mkdir(exist_ok=True)
HELP_DIR.mkdir(exist_ok=True)

# -------------------------------------------------
# App configuration
# -------------------------------------------------
APP_NAME = "Workday Health Reasoning Platform"
DEFAULT_LANG = "en"
SUPPORTED_LANGS = {"en": "English", "ar": "العربية"}
MAX_HISTORY_PER_METRIC = 200 
MODEL_NAME = "gemini-1.5-flash" # Note: Ensure this matches your available model

# -------------------------------------------------
# ✅ SUCCESS CHECK
# -------------------------------------------------
print("-" * 50)
print(f"🚀 {APP_NAME} INITIALIZED")
print(f"📁 DATA FOLDER: {DATA_DIR.absolute()}")
print(f"🌍 LOCALES: {LOCALES_DIR.absolute()}")
print("-" * 50)
print("Copy the DATA FOLDER path above to your file explorer to see your JSON files.")

--------------------------------------------------
🚀 Workday Health Reasoning Platform INITIALIZED
📁 DATA FOLDER: F:\research projects\AI\HealthierAtDesk_SUBMIT\data
🌍 LOCALES: F:\research projects\AI\HealthierAtDesk_SUBMIT\locales
--------------------------------------------------
Copy the DATA FOLDER path above to your file explorer to see your JSON files.


Core Components
The project contains core modules that implement essential logic and services used by the UI. We will go through each of these core components:
Local Data Storage (LocalStore): Manages saving and loading JSON data files locally.
Internationalization (i18n): Handles loading language files and providing a translation function.
Safety Checks (safety.py): Provides functions to detect urgent health or mental health flags in user inputs.
Input Registry (input_registry.py): Defines which data files are considered "canonical" user inputs for global analysis.
Context Manager (context_manager.py): Manages summarized context memory for each domain (stores short summaries to be used across sessions or for global recommendations).
Gemini API Client (gemini_client.py): Placeholder client to interface with the Gemini LLM API (with a safe fallback if not configured).
OCR Pipeline (ocr_pipeline.py): Utility for optical character recognition on files (using Gemini's multimodal capabilities).
Below, each of these modules is presented with its code and a brief explanation.
Local Data Storage (LocalStore)
This module (storage.py) defines the LocalStore class, which provides methods for local persistence of data. It stores JSON data and text files under the configured data directory. Key features: - Initialization: Accepts a data directory path and ensures it exists. - Save/Load JSON: Methods save_json(name, obj) and load_json(name) to write and read JSON files (using json.dump/json.load). If a file doesn't exist, load_json returns a default value. - Save/Load Text: Similar save_text and load_text methods for plain text, used for storing things like last AI outputs. - Utility: A helper last_output_name(domain) to standardize filenames for the last output of a given domain (e.g., 'baseline_last.txt').
This class is used by the tabs to persist user inputs and AI outputs locally so that data is retained across sessions.

In [7]:
from __future__ import annotations

import json
from pathlib import Path
from typing import Any


class LocalStore:
    """
    Local-only storage for:
    - structured JSON (reasoning)
    - human-readable text (UI continuity)

    Improved:
    - flexible JSON saving (handles non-JSON types safely)
    - prints detailed errors instead of failing silently
    """

    def __init__(self, data_dir: Path):
        self.data_dir = Path(data_dir)
        self.data_dir.mkdir(parents=True, exist_ok=True)

    def _path(self, name: str) -> Path:
        """Helper to get full path for a file name in the data directory."""
        return self.data_dir / name

    def save_json(self, name: str, obj: Any) -> None:
        """
        Save JSON safely.
        - Uses default=str to avoid crashing on non-serializable objects
          (numpy types, tuples, Path, datetime, etc.)
        - Prints errors if something goes wrong
        """
        path = self._path(name)
        path.parent.mkdir(parents=True, exist_ok=True)

        try:
            with open(path, "w", encoding="utf-8") as f:
                json.dump(obj, f, ensure_ascii=False, indent=2, default=str)

            print(f"[LocalStore] ✅ JSON saved: {path.resolve()}")

        except Exception as e:
            print(f"[LocalStore] ❌ ERROR saving JSON: {path.resolve()}")
            print(f"[LocalStore] Exception: {e}")
            raise

    def load_json(self, name: str, default: Any = None) -> Any:
        """
        Load JSON safely.
        - If file doesn't exist → returns default
        - If file is corrupted or unreadable → prints error and returns default
        """
        path = self._path(name)

        if not path.exists():
            print(f"[LocalStore] ⚠️ JSON file not found: {path.resolve()}")
            return default

        try:
            with open(path, "r", encoding="utf-8") as f:
                data = json.load(f)

            print(f"[LocalStore] ✅ JSON loaded: {path.resolve()}")
            return data

        except Exception as e:
            print(f"[LocalStore] ❌ ERROR loading JSON: {path.resolve()}")
            print(f"[LocalStore] Exception: {e}")
            return default

    def save_text(self, name: str, text: str) -> None:
        """
        Save plain text safely with error logging.
        """
        path = self._path(name)
        path.parent.mkdir(parents=True, exist_ok=True)

        try:
            with open(path, "w", encoding="utf-8") as f:
                f.write(text)

            print(f"[LocalStore] ✅ Text saved: {path.resolve()}")

        except Exception as e:
            print(f"[LocalStore] ❌ ERROR saving text: {path.resolve()}")
            print(f"[LocalStore] Exception: {e}")
            raise

    def load_text(self, name: str, default: str = "") -> str:
        """
        Load plain text safely.
        """
        path = self._path(name)

        if not path.exists():
            print(f"[LocalStore] ⚠️ Text file not found: {path.resolve()}")
            return default

        try:
            with open(path, "r", encoding="utf-8") as f:
                text = f.read()

            print(f"[LocalStore] ✅ Text loaded: {path.resolve()}")
            return text

        except Exception as e:
            print(f"[LocalStore] ❌ ERROR loading text: {path.resolve()}")
            print(f"[LocalStore] Exception: {e}")
            return default

    def last_output_name(self, domain: str) -> str:
        return f"{domain}_last.txt"

    def load_last_output(self, domain: str) -> str:
        return self.load_text(self.last_output_name(domain))


Internationalization (i18n)
The i18n.py module handles loading of locale files and translating text keys. It loads language files (JSON) from the locales directory and caches them. Key elements: - load_locale(lang, locales_dir): Reads the JSON file for the given language (lang.json) and stores it in a cache. This allows quick reuse without re-reading the file repeatedly. - t(lang, key, locales_dir, default=None): Retrieves the translated string for the given key in the specified language. It falls back to English if the key is missing in the target language, and if not found there either, it returns a provided default or the key itself. This function also supports formatting with **kwargs if placeholders are present in the string.
This allows the UI to display text (like titles, labels) in the user's language (English or Arabic by default, extendable to others by adding locale files).

In [8]:
from __future__ import annotations
import json
from pathlib import Path
from typing import Any, Dict

_CACHE: Dict[str, Dict[str, str]] = {}

def load_locale(lang: str, locales_dir: Path) -> Dict[str, str]:
    if lang in _CACHE:
        return _CACHE[lang]
    path = locales_dir / f"{lang}.json"
    try:
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
    except FileNotFoundError:
        data = {}
    _CACHE[lang] = data
    return data

def t(lang: str, key: str, locales_dir: Path, default: str | None = None, **kwargs: Any) -> str:
    """Translate a key using JSON locales; fallback to English; then to default; then to key."""
    locale = load_locale(lang, locales_dir)
    if key in locale:
        text = locale[key]
    else:
        # Fallback to English
        locale_en = load_locale("en", locales_dir)
        if key in locale_en:
            text = locale_en[key]
        elif default is not None:
            text = default
        else:
            text = key
    # If there are placeholders for formatting, fill them in
    try:
        return text.format(**kwargs)
    except Exception:
        return text

Safety Checks
The safety.py module provides simple functions to detect urgent health concerns in the user's input: - bp_red_flag(systolic, diastolic): Checks blood pressure readings. If either systolic ≥ 180 or diastolic ≥ 120, it returns a flag (True) with a warning message indicating a hypertensive crisis (advising prompt medical attention). Otherwise it returns False with no message. - mental_red_flag(text): Checks a piece of text for certain keywords that suggest severe distress or suicidal ideation (like "suicide", "kill myself", "can't go on", etc.). If any such phrase is found, it flags True with a message urging the user to seek immediate help. - apply_safety_checks(payload): Given a dictionary of user input fields (e.g., might contain 'bp' for blood pressure and 'free_text' for user-entered text), this function runs all relevant checks. It aggregates any flags into a list payload['safety_flags'] and returns the payload. Tabs like the baseline or mental health tab can use this to check if the user's data contains any urgent issues and ensure the UI can alert the user appropriately.
These safety checks add a layer of precaution, ensuring the app can warn the user if something critical is detected in their inputs.

# 🛡️ Central Safety Engine (One-Stop Safety Layer)

This cell defines the **global safety framework** for the entire platform.

Instead of adding red-flag logic inside every tab (Baseline, Mental, MSK, Eye, Hydration, etc.),  
we use **one centralized safety engine** that can be integrated directly into the **GeminiClient**.

That means:

✅ Every AI response passes through one safety checkpoint  
✅ Red-flag warnings are detected consistently across all modules  
✅ No need to repeat emergency logic in each tab prompt  
✅ Safety rules can be updated once, and the whole system benefits immediately  

---

## ✅ What this Safety Engine Does

The safety system works in **three layers**:

### 1) Rule-Based Risk Detection (Fast + Reliable)
The engine checks structured user data such as:

- **Blood pressure values**
- **mental health crisis keywords**
- **emergency symptom keywords in free text**

It returns a list of **safety flags**, each containing:

- `type` (e.g., bp_risk, mental_urgent)
- `severity` (info / warning / urgent)
- `message` (human-readable warning)

---

### 2) Standardized Safety Flag Output
All detected risks are stored as:

```python
payload["safety_flags"] = [...]


In [9]:
from __future__ import annotations
from typing import Dict, Any, Tuple, List, Optional


# ==========================================================
# SAFETY FLAGS HELPERS
# ==========================================================

def _flag(flag_type: str, message: str, severity: str = "warning") -> Dict[str, str]:
    """
    Standard safety flag structure.
    severity: "info" | "warning" | "urgent"
    """
    return {
        "type": flag_type,
        "severity": severity,
        "message": message.strip(),
    }


# ==========================================================
# RULE-BASED SAFETY CHECKS
# ==========================================================

def bp_red_flag(systolic: Optional[float], diastolic: Optional[float]) -> Tuple[bool, str]:
    if systolic is None or diastolic is None:
        return False, ""

    # Conservative urgent thresholds (non-diagnostic).
    if systolic >= 180 or diastolic >= 120:
        return True, (
            "Blood pressure is very high (urgent range). "
            "If you have chest pain, severe headache, shortness of breath, weakness, or confusion, "
            "seek urgent medical care immediately."
        )

    # Moderate warning threshold (optional)
    if systolic >= 160 or diastolic >= 100:
        return True, (
            "Blood pressure is high. Consider re-checking after resting 5–10 minutes "
            "and consult a healthcare professional soon if it stays elevated."
        )

    return False, ""


def mental_red_flag(text: str) -> Tuple[bool, str]:
    if not text:
        return False, ""

    low = text.lower()
    keywords = [
        "suicide", "kill myself", "self harm", "self-harm",
        "end my life", "end it", "can't go on", "cant go on",
        "hopeless", "i want to die"
    ]

    if any(k in low for k in keywords):
        return True, (
            "If you're in immediate danger or feel you might harm yourself, "
            "please seek urgent help right now (local emergency services, a trusted person, "
            "or a nearby medical facility). You do not have to handle this alone."
        )

    return False, ""


def emergency_symptom_red_flag(text: str) -> Tuple[bool, str]:
    """
    Optional generic emergency detection based on free text symptoms.
    Not diagnostic, only triggers caution warnings.
    """
    if not text:
        return False, ""

    low = text.lower()

    emergency_keywords = [
        "chest pain",
        "pressure in chest",
        "can't breathe",
        "cannot breathe",
        "shortness of breath",
        "severe headache",
        "fainting",
        "passed out",
        "stroke",
        "slurred speech",
        "one side weakness",
        "seizure",
        "vomiting blood",
        "blood in stool",
    ]

    if any(k in low for k in emergency_keywords):
        return True, (
            "Some symptoms described may require urgent medical evaluation. "
            "If symptoms are severe, sudden, or worsening, seek emergency care immediately."
        )

    return False, ""


# ==========================================================
# MAIN SAFETY ENGINE
# ==========================================================

def apply_safety_checks(payload: Dict[str, Any]) -> Dict[str, Any]:
    """
    Runs safety checks and attaches safety_flags to payload.
    Expected payload keys (optional):
      - payload["bp"] = {"sys": ..., "dia": ...}
      - payload["free_text"] = "... notes ..."
    """

    flags: List[Dict[str, str]] = []

    # --- Blood Pressure ---
    bp = payload.get("bp") or {}
    sys_val = bp.get("sys")
    dia_val = bp.get("dia")

    try:
        sys_val = float(sys_val) if sys_val is not None else None
    except Exception:
        sys_val = None

    try:
        dia_val = float(dia_val) if dia_val is not None else None
    except Exception:
        dia_val = None

    rf, msg = bp_red_flag(sys_val, dia_val)
    if rf and msg:
        flags.append(_flag("bp_risk", msg, severity="urgent"))

    # --- Mental Health / Self-harm ---
    free_text = (payload.get("free_text") or "").strip()
    rf, msg = mental_red_flag(free_text)
    if rf and msg:
        flags.append(_flag("mental_urgent", msg, severity="urgent"))

    # --- Emergency Symptoms in Free Text ---
    rf, msg = emergency_symptom_red_flag(free_text)
    if rf and msg:
        flags.append(_flag("symptom_urgent", msg, severity="urgent"))

    payload["safety_flags"] = flags
    return payload


# ==========================================================
# DISPLAY HELPERS (OPTION A: PREPEND WARNING TO AI OUTPUT)
# ==========================================================

def format_safety_warnings(flags: List[Dict[str, str]], lang: str = "en") -> str:
    """
    Convert safety flags into a warning block that can be prepended to AI output.
    """

    if not flags:
        return ""

    # sort urgent first
    def _rank(f):
        sev = f.get("severity", "warning")
        if sev == "urgent":
            return 0
        if sev == "warning":
            return 1
        return 2

    flags_sorted = sorted(flags, key=_rank)

    # Basic language support (simple)
    if lang.startswith("ar"):
        header = "⚠️ تنبيه مهم"
    else:
        header = "⚠️ Important Safety Notice"

    lines = [header]

    for f in flags_sorted:
        msg = (f.get("message") or "").strip()
        if msg:
            lines.append(f"- {msg}")

    return "\n".join(lines).strip()


def inject_safety_warnings(ai_text: str, flags: List[Dict[str, str]], lang: str = "en") -> str:
    """
    Option A behavior:
    - If flags exist, prepend them to AI response.
    """
    ai_text = (ai_text or "").strip()

    warning_block = format_safety_warnings(flags, lang=lang)

    if not warning_block:
        return ai_text

    if ai_text:
        return f"{warning_block}\n\n{ai_text}"

    return warning_block


Input Registry
The input_registry.py module defines which user input files are considered "canonical" for inclusion in global reasoning (recommendations). It avoids automatically using any arbitrary file by explicitly listing allowed files. It contains: - CANONICAL_INPUT_FILES: A set of file names (strings) that represent user inputs (such as 'baseline_data.json', 'mental.json', etc.). Only data from these files should be aggregated for global analysis. - is_canonical_input(filename): Checks if a given file name is in the above set (returns True/False). - list_canonical_inputs(): Returns a sorted list of the canonical input file names.
This ensures the global recommendations or reports only consider intended data sources, following the principle of being explicit about what data to aggregate.
"""
Input Registry

This module defines which files represent canonical user-provided inputs.
Only files listed here are allowed to participate in global reasoning
(e.g. Global Recommendations).

Design principles:
- Explicit > implicit
- No auto-discovery
- No model involvement
- Easy to expand safely
"""

from typing import List, Set

# -------------------------------------------------------------------
# Canonical input files (single source of truth)
# -------------------------------------------------------------------

CANONICAL_INPUT_FILES: Set[str] = {
    "workspace.json",
    "msk.json",
    "mental.json",
    "hydration.json",
    "eye.json",
    "baseline_data.json",
    "baseline_surveys.json",
    "longitudinal_logs.json",
}

# -------------------------------------------------------------------
# Public helpers
# -------------------------------------------------------------------

def is_canonical_input(filename: str) -> bool:
    """
    Returns True if the given filename is a declared user input file.
    """
    return filename in CANONICAL_INPUT_FILES

def list_canonical_inputs() -> List[str]:
    """
    Returns a sorted list of canonical input filenames.
    Useful for audits, debugging, or UI explanations.
    """
    return sorted(CANONICAL_INPUT_FILES)
Context Manager
The context_manager.py module manages the storage of summarized context for each domain. As the user interacts with different tabs, each tab can produce a short summary of the user's state in that area (to avoid having to process full history each time). The module includes: - ContextBundle dataclass: Defines a structured object with fields for each domain (baseline, longitudinal, msk, mental, etc.), which could be used to hold context strings. (This is not heavily used in the skeleton but outlines the context structure.) - CONTEXT_FILE: Name of the JSON file (e.g., 'context_summaries.json') where these summaries are saved. - load_context(store): Loads the entire context summaries dictionary from storage (or returns an empty dict if none saved). - save_context(store, ctx): Saves the context summaries dictionary to file. - update_domain_summary(store, domain, summary): Convenience function to update one domain's summary in the context file (loads current context, updates/adds the given domain's entry, and saves it back). Returns the updated context dict. - delete_domain(store, domain): Removes a domain from the context summaries (for example, if one wants to reset context for that domain) and saves changes. - compose_relevant_context(store, domains): Given a list of domain names, it fetches their summaries and composes a single string with each domain's summary on a new line (prefixed by the domain name). This is useful to build a prompt context string for the AI that includes only relevant domains.
By maintaining a short summary of each domain's information, the app can provide the AI model with a concise context (rather than long histories) when generating recommendations or reports.

In [10]:
from __future__ import annotations
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, Optional, List



CONTEXT_FILE = "context_summaries.json"

@dataclass
class ContextBundle:
    baseline: str = ""
    longitudinal: str = ""
    msk: str = ""
    mental: str = ""
    eye: str = ""
    hydration: str = ""
    productivity: str = ""

    # You can extend this dataclass with more fields if needed (e.g., recovery_sleep, etc.)

    def to_dict(self) -> Dict[str, str]:
        return {
            "baseline": self.baseline,
            "longitudinal": self.longitudinal,
            "msk": self.msk,
            "mental": self.mental,
            "eye": self.eye,
            "hydration": self.hydration,
            "productivity": self.productivity,
        }

def load_context(store: LocalStore) -> Dict[str, str]:
    data = store.load_json(CONTEXT_FILE, default={})
    return data if isinstance(data, dict) else {}

def save_context(store: LocalStore, ctx: Dict[str, str]) -> None:
    store.save_json(CONTEXT_FILE, ctx)

def update_domain_summary(store: LocalStore, domain: str, summary: str) -> Dict[str, str]:
    ctx = load_context(store)
    ctx[domain] = summary.strip()
    save_context(store, ctx)
    return ctx

def delete_domain(store: LocalStore, domain: str) -> Dict[str, str]:
    ctx = load_context(store)
    if domain in ctx:
        del ctx[domain]
    save_context(store, ctx)
    return ctx

def compose_relevant_context(store: LocalStore, domains: List[str]) -> str:
    """Return a compact, readable context block using only requested domains."""
    ctx = load_context(store)
    lines = []
    for d in domains:
        val = (ctx.get(d) or "").strip()
        if val:
            lines.append(f"- {d}: {val}")
    return "\n".join(lines).strip()
def load_latest_session(domain: str, store) -> Optional[dict]:
    """
    Loads the latest saved session record for a given domain.
    Expected file format: LIST of records [{timestamp, user_input, ai_output}, ...]
    """
    user_file = f"{domain}_user_input.json"
    ai_file = f"{domain}_ai_output.json"

    # Try AI file first (usually contains ai_output)
    raw_ai = store.load_json(ai_file, [])
    if isinstance(raw_ai, list) and raw_ai:
        return raw_ai[-1]

    # Fallback: try user file
    raw_user = store.load_json(user_file, [])
    if isinstance(raw_user, list) and raw_user:
        return raw_user[-1]

    return None
from pathlib import Path
from typing import Optional

def _safe_t(lang: str, key: str, locales_dir: Optional[Path], default: str) -> str:
    """Safe translation wrapper (won't crash if t() is missing)."""
    try:
        if "t" in globals() and callable(globals()["t"]):
            return globals()["t"](lang, key, locales_dir, default=default)
    except Exception as e:
        print(f"[i18n] ⚠️ t() failed for key={key}: {e}")
    return default


In [11]:
from __future__ import annotations

import json
from pathlib import Path
from datetime import datetime, timezone
from typing import Any, Optional, Sequence

import gradio as gr


def _now() -> str:
    return datetime.now(timezone.utc).isoformat().replace("+00:00", "")


def _safe_t(lang: str, key: str, locales_dir: Optional[Path], default: str) -> str:
    """Safe translation wrapper: uses global t() if available, otherwise returns default."""
    try:
        if "t" in globals() and callable(globals()["t"]):
            return globals()["t"](lang, key, locales_dir, default=default)
    except Exception as e:
        print(f"[i18n] ⚠️ t() failed for key={key}: {e}")
    return default


def _to_int(x: Any, default: int = 0) -> int:
    try:
        if x is None or x == "":
            return default
        return int(float(x))
    except Exception:
        return default


def _to_float(x: Any, default: float = 0.0) -> float:
    try:
        if x is None or x == "":
            return default
        return float(x)
    except Exception:
        return default


def _to_bool(x: Any) -> bool:
    try:
        return bool(x)
    except Exception:
        return False


def _to_str(x: Any, default: str = "") -> str:
    try:
        if x is None:
            return default
        s = str(x).strip()
        return s if s else default
    except Exception:
        return default


def _to_list(x: Any) -> list:
    if x is None:
        return []
    if isinstance(x, list):
        return x
    if isinstance(x, tuple):
        return list(x)
    if isinstance(x, set):
        return list(x)
    if isinstance(x, str):
        return [x]
    if isinstance(x, Sequence):
        return list(x)
    return [x]


def _debug_store_path(store: Any, filename: str) -> str:
    try:
        if hasattr(store, "_path") and callable(store._path):
            return str(store._path(filename).resolve())
    except Exception:
        pass
    try:
        if hasattr(store, "data_dir"):
            return str((Path(store.data_dir) / filename).resolve())
    except Exception:
        pass
    return filename


def _save_json_best_effort(store: Any, name: str, obj: Any) -> None:
    """Prefer store.save_json; fallback to writing to store.data_dir (or ./data)."""
    try:
        store.save_json(name, obj)
        print(f"[store] ✅ store.save_json OK -> {_debug_store_path(store, name)}")
        return
    except Exception as e:
        print(f"[store] ❌ store.save_json failed for {name}: {e}")

    data_dir = Path(getattr(store, "data_dir", Path("data")))
    data_dir.mkdir(parents=True, exist_ok=True)
    path = (data_dir / name).resolve()

    with open(path, "w", encoding="utf-8") as f:
        json.dump(obj, f, ensure_ascii=False, indent=2, default=str)

    print(f"[store] ✅ fallback write OK -> {path}")


def save_session(domain: str, user_input: Any, ai_output: str, store: Any) -> None:
    """Append snapshot to <domain>_user_input.json and <domain>_ai_output.json."""
    ts = _now()
    user_file = f"{domain}_user_input.json"
    ai_file = f"{domain}_ai_output.json"

    if user_input is not None:
        existing = store.load_json(user_file, [])
        if not isinstance(existing, list):
            existing = []
        existing.append({"timestamp": ts, "user_input": user_input})
        _save_json_best_effort(store, user_file, existing)

    if ai_output is not None:
        existing = store.load_json(ai_file, [])
        if not isinstance(existing, list):
            existing = []
        existing.append({"timestamp": ts, "ai_output": str(ai_output)})
        _save_json_best_effort(store, ai_file, existing)




def build_context_from_data_folder(store) -> str:
    """
    Builds unified context ONLY from user input files in data folder.
    Reads: *_user_input.json
    Uses the latest record from each file (if list).
    Excludes AI outputs and summaries.
    """

    data_dir = Path(getattr(store, "data_dir", Path("data")))
    if not data_dir.exists():
        return ""

    domains_context = {}

    for p in sorted(data_dir.glob("*_user_input.json")):
        try:
            raw = json.loads(p.read_text(encoding="utf-8"))

            # file stem: baseline_user_input -> baseline
            domain = p.stem.replace("_user_input", "").strip()

            # If stored as list (append history)
            if isinstance(raw, list) and raw:
                last = raw[-1]
                if isinstance(last, dict) and "user_input" in last:
                    domains_context[domain] = last["user_input"]
                else:
                    domains_context[domain] = last

            # If stored as dict
            elif isinstance(raw, dict):
                if "user_input" in raw:
                    domains_context[domain] = raw["user_input"]
                else:
                    domains_context[domain] = raw

        except Exception as e:
            print(f"[context] ⚠️ Failed reading {p.name}: {e}")
            continue

    if not domains_context:
        return ""

    # Pretty text output for prompt usage
    blocks = []
    for dom, data in domains_context.items():
        blocks.append(f"## {dom.upper()}\n{json.dumps(data, indent=2, ensure_ascii=False)}")

    return "\n\n".join(blocks)




def preflight_check() -> None:
    required = [
        "_safe_t","_now","_save_json_best_effort",
        "_to_int","_to_float","_to_bool","_to_str","_to_list",
        "save_session","build_context_from_data_folder",
    ]
    missing = [n for n in required if n not in globals()]
    if missing:
        raise NameError(f"Missing globals: {missing}")
    print("✅ Preflight OK")
from pathlib import Path
from typing import Any, Dict, List, Optional, Tuple

def domain_files(domain: str) -> Tuple[str, str]:
    return f"{domain}_user_input.json", f"{domain}_ai_output.json"

def _as_history_list(raw: Any) -> List[Dict[str, Any]]:
    # supports old formats too
    if isinstance(raw, list):
        return raw
    if isinstance(raw, dict) and raw:
        return [raw]
    return []

def _first_existing(store: Any, candidates: List[str]) -> str:
    # Try to detect which filename actually exists on disk
    for name in candidates:
        try:
            if hasattr(store, "_path") and callable(store._path):
                p = Path(store._path(name))
            else:
                p = Path(getattr(store, "data_dir", "data")) / name
            if p.exists():
                return name
        except Exception:
            pass
    return candidates[0]  # fallback

def load_domain_history(
    store: Any,
    domain: str,
    legacy_user_files: Optional[List[str]] = None,
    legacy_ai_files: Optional[List[str]] = None,
) -> Tuple[Dict[str, Any], str]:
    user_file, ai_file = domain_files(domain)

    user_candidates = [user_file] + (legacy_user_files or [])
    ai_candidates   = [ai_file]   + (legacy_ai_files or [])

    user_file = _first_existing(store, user_candidates)
    ai_file   = _first_existing(store, ai_candidates)

    raw_user = store.load_json(user_file, [])
    user_hist = _as_history_list(raw_user)
    latest_user_record = user_hist[-1] if user_hist else {}
    saved_user_input = latest_user_record.get("user_input", {})
    if not isinstance(saved_user_input, dict):
        saved_user_input = {}

    raw_ai = store.load_json(ai_file, [])
    ai_hist = _as_history_list(raw_ai)
    latest_ai_record = ai_hist[-1] if ai_hist else {}
    saved_ai_text = latest_ai_record.get("ai_output", "")
    if not isinstance(saved_ai_text, str):
        saved_ai_text = str(saved_ai_text)

    # Debug (super useful during restart issues)
    try:
        print(f"[{domain}] load user={user_file} ai={ai_file}")
    except Exception:
        pass

    return saved_user_input, saved_ai_text


Helper for the Add info button

In [12]:
import gradio as gr
from info_variables import (
    HYDRATION_INFO_MD,
    MENTAL_WELLBEING_INFO_MD,
    WORKSPACE_INFO_MD,
    MSK_INFO_MD,
    EYE_HEALTH_INFO_MD,
    PRODUCTIVITY_INFO_MD,
    RECOVERY_SLEEP_INFO_MD,ABOUT_APP_MD
)



def add_info_panel(
    title: str,
    content_md: str,
    button_label: str = "ℹ️ About this tab",
    open_label: str = "❌ Hide info",
):
    """
    Creates a toggle button + hidden markdown panel.
    Returns: (button, panel_component)
    """

    info_state = gr.State(False)  # hidden by default

    btn = gr.Button(button_label, variant="secondary")

    with gr.Accordion(title, open=False, visible=False) as panel:
        md = gr.Markdown(content_md)

    def _toggle(current):
        new_state = not current
        return (
            new_state,
            gr.update(visible=new_state),
            open_label if new_state else button_label
        )

    btn.click(
        fn=_toggle,
        inputs=[info_state],
        outputs=[info_state, panel, btn],
    )

    return btn, panel


Gemini API Client
The gemini_client.py module provides a wrapper for interacting with the Google Gemini API (a large language model). In this skeleton, it functions as a placeholder with a fallback if no API key is configured: - GeminiResponse (dataclass): A simple container for responses, holding the generated text, the model name, and a used_fallback flag indicating if the result is a placeholder. - GeminiClient class: - Initialization (__init__): Attempts to retrieve an API key from the environment (GEMINI_API_KEY). If found, it sets up a genai.Client (from Google's genai library) for the specified model. If no key is present, it marks the client as disabled (self.enabled=False), meaning actual API calls won't be made. - generate(prompt, response_language): If the client is enabled, it sends the prompt to the Gemini model (with an instruction to respond in the given language) and returns the model's response as a GeminiResponse. If not enabled (no API key), it returns a GeminiResponse with a placeholder text (indicating Gemini is not configured). - generate_with_file(prompt, file_path, response_language): Similar to generate, but allows including a file (image or PDF) in the prompt. It reads the file bytes and sends them along with the prompt to the model. If not enabled, returns a placeholder response. There's also an internal helper _detect_mime to determine the file's MIME type (only certain types are supported).
All tabs use gemini.generate() (and some use gemini.generate_with_file()) to get AI-generated insights. In the absence of a real API key, the calls will just produce placeholder messages, which is suitable for testing the UI flow.

In [ ]:
import os
import google.generativeai as genai
from typing import Optional, Dict, Any
from dotenv import load_dotenv

# Load .env to ensure keys are available
load_dotenv(override=True)

# =================================================
# 1. Find Available Gemini Model (Auto)
# =================================================
def pick_best_gemini_model(prefer_flash=True) -> str:
    """
    Picks the best available Gemini model.
    """
    try:
        models = list(genai.list_models())
    except Exception as e:
        print(f"⚠️ Could not list Gemini models: {e}")
        return "gemini-1.5-flash"

    valid = [m.name.replace("models/", "") for m in models 
             if "generateContent" in m.supported_generation_methods]

    if not valid:
        return "gemini-1.5-flash"

    # Priority 1: Flash models (fast/cheap)
    # Priority 2: Pro models
    flash_models = [x for x in valid if "flash" in x]
    pro_models = [x for x in valid if "pro" in x]

    if prefer_flash and flash_models: 
        return flash_models[0]
    return pro_models[0] if pro_models else valid[0]


# =================================================
# Response Wrapper
# =================================================
class AIResponse:
    def __init__(self, text, model, safety_flags=None):
        self.text = text
        self.model = model
        self.safety_flags = safety_flags or []


# =================================================
# 2. Gemini-Only Client
# =================================================
class GeminiClient:
    def __init__(self, gemini_api_key: Optional[str] = None):
        # API Key lookup
        self.api_key = gemini_api_key or os.getenv("GEMINI_API_KEY")
        
        if not self.api_key:
            print("❌ Error: No GEMINI_API_KEY found in .env or passed as argument.")
            self.enabled = False
            self.model_name = "none"
        else:
            genai.configure(api_key=self.api_key)
            self.model_name = pick_best_gemini_model()
            self.enabled = True
            print(f"✅ Gemini Client Active: {self.model_name}")

    # --- Safety & Utilities ---

    def _run_safety_checks(self, safety_payload: Optional[Dict[str, Any]]) -> list:
        """Looks for an external safety check function in globals."""
        if not safety_payload: 
            return []
        try:
            if "apply_safety_checks" in globals():
                checked = globals()["apply_safety_checks"](safety_payload)
                return checked.get("safety_flags", []) if isinstance(checked, dict) else []
        except: 
            pass
        return []

    def _format_safety_banner(self, flags) -> str:
        """Formats the safety warning banner for the UI."""
        if not flags: 
            return ""
        lines = ["## ⚠️ Safety Notice"]
        for f in flags:
            msg = f.get("message") if isinstance(f, dict) else str(f)
            if msg: lines.append(f"- {msg}")
        return "\n".join(lines) + "\n\n"

    # =================================================
    # Main Generate Function
    # =================================================
    def generate(self, prompt, response_language="en", safety_payload: Optional[Dict[str, Any]] = None):
        # 1. Safety Checks
        safety_flags = self._run_safety_checks(safety_payload)
        safety_banner = self._format_safety_banner(safety_flags)

        if not self.enabled:
            return AIResponse(
                text=safety_banner + "Gemini is not configured. Please check your API key.",
                model="none",
                safety_flags=safety_flags
            )

        # 2. Prepare Prompt
        system_rule = (
            "STYLE: Friendly, supportive, encouraging. Simple language.\n"
            "DATA RULE: Ignore numeric 0 unless clearly valid.\n\n"
        )
        full_prompt = f"Respond in {response_language}.\n\n{system_rule}{prompt}"

        # 3. Call Gemini
        try:
            model = genai.GenerativeModel(self.model_name)
            response = model.generate_content(full_prompt)
            
            # Extract text safely
            generated_text = response.text if response.text else "The AI returned an empty response."
            
            return AIResponse(
                text=safety_banner + generated_text.strip(),
                model=self.model_name,
                safety_flags=safety_flags
            )

        except Exception as e:
            print(f"❌ Gemini Execution Error: {e}")
            return AIResponse(
                text=safety_banner + f"Sorry, I encountered an error: {str(e)}",
                model=self.model_name,
                safety_flags=safety_flags
            )

# --- Initialize global client ---
# Every other cell in your notebook should use 'gemini.generate(...)'
gemini = GeminiClient()

C:\Users\Owner\AppData\Local\Temp\ipykernel_30548\1049567649.py:5: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  import google.generativeai as genai


✅ DeepSeek enabled (Primary): deepseek-chat
✅ Gemini enabled (Secondary): gemini-2.5-flash
✅ Groq enabled (Fallback): llama-3.3-70b-versatile


OCR Pipeline
The ocr_pipeline.py module provides a function for handling OCR (Optical Character Recognition) using the AI (Gemini) for file inputs. It contains: - extract_text_from_file(gemini, file_path, lang='en'): This function is meant to take a file path (to an image or PDF) and use the GeminiClient to extract text from it. It likely constructs a prompt instructing the model to perform OCR on the file. If gemini is not enabled (no API), this would return a placeholder or raise an error.
This utility can be used in conjunction with the Longitudinal tab or other features where a user might upload a document or image to extract health-related text (for example, scanning a medical report for analysis). It's not extensively used in the default tabs but is provided for future extension.

In [14]:
from pathlib import Path

def extract_text_from_file(gemini, file_path: str | Path, lang: str = "en") -> str:
    """
    OCR / transcription pipeline.
    Uses Gemini multimodal capability to extract raw text from
    PDF or image files.

    Returns:
        Plain extracted text (string)

    Raises:
        Exception: If Gemini is not configured or an error occurs.
    """
    # Define an OCR-specific prompt (this can be adjusted as needed)
    ocr_prompt = "Extract all textual content from this document/image."
    # Use the Gemini client to perform the OCR
    resp = gemini.generate_with_file(
        prompt=ocr_prompt,
        file_path=file_path,
        response_language=lang,
    )
    if resp.used_fallback:
        # If Gemini isn't configured, raise an exception (or handle accordingly)
        raise Exception("Gemini is not configured for OCR.")
    return resp.text or ""

User Interface (Gradio) Setup
The main user interface is defined in the app.py module. This script ties together the core components and the tabs within a Gradio app. Key points about app.py: - It uses Gradio Blocks to build the interface and define the layout (with a title, some description, and tabs). - It loads environment variables from a .env file (if present) via load_dotenv() to configure things like API keys. - It initializes the core services: a LocalStore for data storage and a GeminiClient for AI (which will be disabled if no API key is set). - It sets up a state variable lang_state to keep track of the current language (with default from config.DEFAULT_LANG). This state can be updated by the UI (e.g., if a language switcher were implemented) and is passed to tabs so they know what language to generate responses in. - Header: The app displays a title and subtitle using Markdown, pulling the text from the locale files via the translation function t(...). It also shows a notice about data being local-only (also from locales, likely msg.local_only). - Tabs: The interface is organized into tabs using gr.TabItem. The first tab is a general "🏢 General Recommendations" tab (the Action Center) which is built by build_global_recommendations_tab(...). Then, the app dynamically creates tabs for each domain by iterating over the TAB_BUILDERS list (imported from the tabs package). For each entry (which has a translation key for the title and the corresponding build function), it opens a new tab and calls the builder function with the necessary parameters (store, gemini, etc.). This way, all tabs are added to the UI. - Finally, the build_app() function returns the constructed Gradio Blocks app (assigned to demo in the code). If the script is run as __main__, it calls build_app() and then launches the app with app.launch(...) on the specified host and port.
Below is the content of app.py illustrating the above functionality:

In [15]:
from __future__ import annotations

import os
import requests
from dataclasses import dataclass, field
from typing import Optional, List, Any

from dotenv import load_dotenv
import google.generativeai as genai


# -------------------------------------------------
# Load environment variables from .env
# -------------------------------------------------
load_dotenv()


@dataclass
class GeminiResponse:
    text: str
    model: str
    used_fallback: bool = False
    safety_flags: List[Any] = field(default_factory=list)


class GeminiClient:
    """
    Hybrid Gemini client with:
    - Text generation
    - File (PDF / image) generation for OCR
    - Local LM Studio fallback for text-only calls

    Priority:
    1. Gemini API
    2. Local LM Studio (text only)
    3. Placeholder
    """

    def __init__(
        self,
        api_key: Optional[str] = None,
        model_name: str = "gemini-2.5-flash-lite",
        use_local_fallback: bool = True,
        lmstudio_url: Optional[str] = None,
        lmstudio_model: str = "ewaast-medgemma-1.5-4b",
    ):
        # -------------------------
        # Gemini setup
        # -------------------------
        self.api_key = api_key or os.getenv("GEMINI_API_KEY")
        self.model_name = model_name
        self.gemini_enabled = bool(self.api_key)

        if self.gemini_enabled:
            genai.configure(api_key=self.api_key)

        # -------------------------
        # Local LM Studio setup (text-only)
        # -------------------------
        self.use_local_fallback = use_local_fallback
        self.lmstudio_url = (
            lmstudio_url
            or os.getenv("LMSTUDIO_URL")
            or "http://192.168.1.34:1234/v1/chat/completions"
        )
        self.lmstudio_model = lmstudio_model

    # =================================================
    # TEXT GENERATION
    # =================================================
    def generate(
        self,
        prompt: str,
        response_language: str = "en",
    ) -> GeminiResponse:

        # 1️⃣ Gemini text generation
        if self.gemini_enabled:
            try:
                model = genai.GenerativeModel(self.model_name)
                response = model.generate_content(
                    f"Respond in {response_language}.\n\n{prompt}"
                )

                return GeminiResponse(
                    text=response.text or "",
                    model=self.model_name,
                    used_fallback=False,
                )

            except Exception as e:
                gemini_error = str(e)
        else:
            gemini_error = "Gemini API key not configured."

        # 2️⃣ Local LM Studio fallback (text only)
        if self.use_local_fallback:
            try:
                text = self._generate_local(
                    prompt=prompt,
                    response_language=response_language,
                )

                return GeminiResponse(
                    text=text,
                    model=self.lmstudio_model,
                    used_fallback=True,
                )

            except Exception as e:
                local_error = str(e)
        else:
            local_error = "Local fallback disabled."

        # 3️⃣ Placeholder
        return GeminiResponse(
            text=(
                "AI is temporarily unavailable.\n\n"
                "General guidance:\n"
                "- Take regular breaks\n"
                "- Prioritize sleep and hydration\n"
                "- Use simple stress-reduction techniques\n"
                "- Seek professional support if concerns persist"
            ),
            model="placeholder",
            used_fallback=True,
        )

    # =================================================
    # FILE / OCR GENERATION (Gemini only)
    # =================================================
    def generate_with_file(
        self,
        file_path: str,
        prompt: str,
        response_language: str = "en",
    ) -> GeminiResponse:
        """
        Generate content using a file (PDF / image) + text prompt.
        Used for OCR and document understanding.
        """

        if not self.gemini_enabled:
            return GeminiResponse(
                text="File processing unavailable (Gemini API not configured).",
                model="fallback",
                used_fallback=True,
            )

        try:
            uploaded_file = genai.upload_file(file_path)

            model = genai.GenerativeModel(self.model_name)
            response = model.generate_content(
                [
                    uploaded_file,
                    f"Respond in {response_language}.\n\n{prompt}",
                ]
            )

            return GeminiResponse(
                text=response.text or "",
                model=self.model_name,
                used_fallback=False,
            )

        except Exception as e:
            return GeminiResponse(
                text=f"Gemini file processing error: {e}",
                model=self.model_name,
                used_fallback=True,
            )

    # =================================================
    # LOCAL LM STUDIO (TEXT ONLY)
    # =================================================
    def _generate_local(self, prompt: str, response_language: str) -> str:
        headers = {"Content-Type": "application/json"}

        payload = {
            "model": self.lmstudio_model,
            "messages": [
                {
                    "role": "system",
                    "content": (
                        f"Respond in {response_language}. "
                        "Provide safe, supportive, non-diagnostic health guidance."
                    ),
                },
                {
                    "role": "user",
                    "content": prompt,
                },
            ],
            "temperature": 0.5,
            "max_tokens": 200,
            "stream": False,
        }

        response = requests.post(
            self.lmstudio_url,
            headers=headers,
            json=payload,
            timeout=120,
        )

        response.raise_for_status()
        data = response.json()

        return data["choices"][0]["message"]["content"]


Tabs Implementation
The application includes numerous tabs, each corresponding to a specific aspect of health or app functionality. Each tab is built by a function (e.g., build_baseline_tab) that creates Gradio interface elements and defines how user inputs are handled. All tab builder functions accept the same parameters: the store (for data persistence), gemini (for AI calls), lang_state (to track the current language), and locales_dir (for translations). They typically load any previously saved data, set up input fields (numbers, text boxes, sliders, etc.), and define output displays or actions (like generating a summary or saving data).
The tabs can be grouped into categories:
General Assessment: Baseline, Workspace, Longitudinal – initial assessments, workspace setup info, and long-term tracking of health metrics.
Health Domains: Musculoskeletal (MSK), Eye, Mental, Hydration – domain-specific health tracking (e.g., exercise/posture, eye strain, mental well-being, water intake), each providing recommendations or summaries related to that domain.
Work & Rest: Productivity, Recovery/Sleep – tracking work productivity and rest/sleep quality, offering suggestions to balance work and recovery.
Action Plans: Checklist, Reminders – execution-oriented tabs; a checklist for healthy tasks and a reminders schedule for routine health activities.
Meta/Utility: Context, Reports, Settings, Help – supporting features, including viewing aggregated context summaries, generating overall reports, adjusting settings (like clearing data), and viewing help information.
We will go through each tab module below, showing its code and describing its functionality.
General Recommendations Tab
This tab (the "Action Center") aggregates inputs from all domains and provides overall health recommendations. It loads recent outputs or context from each domain and uses the AI to generate general advice. It may highlight urgent issues (safety flags) and give the user a summary of what to focus on.

Baseline Tab
The Baseline tab collects the user's baseline health information (e.g., initial metrics like height and weight, and possibly other survey data). On submission, it saves the data to a JSON file and uses the AI to produce initial health recommendations. It also runs safety checks on the provided data (for example, checking for dangerously high metrics) and includes any urgent warnings in the output. The resulting summary is saved to the context memory for future reference.

# 🩺 Baseline Module (Baseline Health Check-In)

This module collects baseline biometric and lifestyle information such as height, weight, blood pressure, resting heart rate,
and physical activity level.

It is designed to generate **preventive, non-diagnostic** recommendations, focusing on lifestyle improvements and workplace
health risks.

Baseline data is used as the foundation for other modules and future personalized guidance.


In [16]:
# ==========================================
# Baseline Module — Config + History Load (SAFE)
# ==========================================

def load_baseline_history(store):
    """
    Loads the most recent saved user input + AI output for the Baseline module.
    Uses the universal loader: load_domain_history()
    """
    local_domain = "baseline"
    return load_domain_history(store, local_domain)


## Step 1 — User Inputs (Baseline Biometrics)

This section builds the Baseline tab input UI.

The baseline data includes:
- Height and weight
- Optional blood pressure and resting heart rate
- Optional body fat and waist circumference
- Activity level
- Additional notes (free text)

These inputs allow the system to provide general preventive recommendations and detect workplace health risk patterns.


In [17]:
# ==========================================
# Baseline Module — Inputs UI Builder
# ==========================================

import gradio as gr


def build_baseline_inputs(saved_user_input: dict, lang: str, locales_dir: Optional[Path]):

    with gr.Row():
        height_input = gr.Number(
            value=saved_user_input.get("height"),
            label=_safe_t(lang, "baseline.height", locales_dir, default="Height (cm)")
        )
        weight_input = gr.Number(
            value=saved_user_input.get("weight"),
            label=_safe_t(lang, "baseline.weight", locales_dir, default="Weight (kg)")
        )

    gr.Markdown("### Vitals & Biometrics (Optional)")
    with gr.Row():
        systolic = gr.Number(value=saved_user_input.get("bp_systolic"), label="Systolic BP (mmHg)")
        diastolic = gr.Number(value=saved_user_input.get("bp_diastolic"), label="Diastolic BP (mmHg)")
        rhr = gr.Number(value=saved_user_input.get("rhr"), label="Resting Heart Rate (BPM)")

    with gr.Row():
        body_fat = gr.Number(value=saved_user_input.get("body_fat"), label="Body Fat %")
        waist = gr.Number(value=saved_user_input.get("waist_cm"), label="Waist Circumference (cm)")

    activity_level = gr.Dropdown(
        choices=["Sedentary", "Moderately active", "Very active"],
        value=saved_user_input.get("activity_level", "Sedentary"),
        label=_safe_t(lang, "baseline.activity", locales_dir, default="Activity Level"),
    )

    notes_input = gr.Textbox(
        value=saved_user_input.get("notes", ""),
        label=_safe_t(lang, "baseline.notes", locales_dir, default="Additional Notes"),
        lines=3,
    )

    return {
        "height_input": height_input,
        "weight_input": weight_input,
        "systolic": systolic,
        "diastolic": diastolic,
        "rhr": rhr,
        "body_fat": body_fat,
        "waist": waist,
        "activity_level": activity_level,
        "notes_input": notes_input,
    }


## Step 2 — Output Components

This section defines:

- A status message area for saving / generation feedback
- A read-only output textbox that displays Gemini-generated recommendations

The latest saved AI output is loaded automatically so the user can resume from their previous baseline report.


In [18]:
# ==========================================
# Baseline Module — Output UI Builder
# ==========================================

def build_baseline_outputs(saved_ai_text: str, lang: str, locales_dir: Optional[Path]):

    status = gr.Markdown("")

    output = gr.Textbox(
        label=_safe_t(lang, "baseline.output", locales_dir, default="Baseline Recommendations"),
        interactive=False,
        value=saved_ai_text,
        lines=12
    )

    return output, status


## Step 3 — AI Recommendations + Safety Checks + Saving

This section implements the Baseline logic:

- Collect baseline user inputs into a structured JSON format
- Run optional safety checks on free text / blood pressure values
- Generate preventive recommendations using Gemini
- Save user inputs and AI outputs as timestamped history records

This ensures baseline data can be reused later in longitudinal tracking and other health modules.


In [19]:
# ==========================================
# Baseline Module — AI + Saving + Wiring
# ==========================================

def connect_baseline_logic(
    store,
    gemini,
    lang_state: gr.State,
    locales_dir: Optional[Path],
    ui_inputs: dict,
    output: gr.Textbox,
    status: gr.Markdown,
):

    # IMPORTANT: local names (prevents cross-tab overwrite bugs)
    local_domain = "baseline"
    local_user_file = f"{local_domain}_user_input.json"
    local_ai_file = f"{local_domain}_ai_output.json"

    # -------------------------
    # Collect user input helper
    # -------------------------
    def collect_user_input(h, w, sys, dia, heart, fat, waist_val, activity, notes):
        return {
            "height": _to_float(h, 0.0),
            "weight": _to_float(w, 0.0),
            "bp_systolic": _to_int(sys, 0),
            "bp_diastolic": _to_int(dia, 0),
            "rhr": _to_int(heart, 0),
            "body_fat": _to_float(fat, 0.0),
            "waist_cm": _to_float(waist_val, 0.0),
            "activity_level": str(activity or ""),
            "notes": (notes or "").strip(),
        }

    # -------------------------
    # Save user input (APPEND)
    # -------------------------
    def save_user_input(h, w, sys, dia, heart, fat, waist_val, activity, notes):
        print("[baseline] 🔘 Save User Input clicked")

        try:
            user_input = collect_user_input(h, w, sys, dia, heart, fat, waist_val, activity, notes)
            record = {"timestamp": _now(), "user_input": user_input}

            existing = store.load_json(local_user_file, [])
            if not isinstance(existing, list):
                existing = []

            existing.append(record)

            # ✅ direct save
            store.save_json(local_user_file, existing)

            return f"✅ Baseline user input saved -> `{_debug_store_path(store, local_user_file)}`"

        except Exception as e:
            print(f"[baseline] ❌ Save failed: {e}")
            return f"❌ Save failed: {e}"

    # -------------------------
    # Generate AI output
    # -------------------------
    def generate_ai(h, w, sys, dia, heart, fat, waist_val, activity, notes):
        print("[baseline] 🔘 Generate AI clicked")

        user_input = collect_user_input(h, w, sys, dia, heart, fat, waist_val, activity, notes)

        payload = {
            "bp": {
                "systolic": user_input.get("bp_systolic"),
                "diastolic": user_input.get("bp_diastolic"),
            },
            "free_text": user_input.get("notes", "")
        }

        flags = []
        try:
            safety_fn = globals().get("apply_safety_checks")
            if safety_fn:
                payload_checked = safety_fn(payload)
                flags = payload_checked.get("safety_flags", []) if isinstance(payload_checked, dict) else []
        except Exception as e:
            print(f"[baseline] ⚠️ apply_safety_checks failed: {e}")

        rec_prompt = (
            "You are a preventive health assistant.\n"
            "Use the baseline data below to provide non-diagnostic health recommendations.\n"
            "Focus on lifestyle, prevention, desk-work risk factors, and practical next steps.\n\n"
            f"BASELINE DATA:\n{json.dumps(user_input, indent=2, ensure_ascii=False)}"
        )

        current_lang = lang_state.value if hasattr(lang_state, "value") else lang_state

        try:
            rec = gemini.generate(prompt=rec_prompt, response_language=current_lang)
            rec_text = rec.text.strip() if hasattr(rec, "text") and rec.text else str(rec).strip()
        except Exception as e:
            return "", f"⚠️ AI generation failed: {e}"

        if not rec_text:
            return "", "⚠️ AI returned empty output."

        if flags:
            warnings = " ".join(f.get("message", "") for f in flags if isinstance(f, dict))
            warnings = warnings.strip()
            if warnings:
                rec_text = f"⚠️ {warnings}\n\n{rec_text}"

        return rec_text, "✅ AI output generated (not saved yet)."

    # -------------------------
    # Save AI output (APPEND)
    # -------------------------
    def save_ai_output(text: str):
        print("[baseline] 🔘 Save AI Output clicked")

        if not (text or "").strip():
            return "⚠️ Nothing to save."

        try:
            record = {"timestamp": _now(), "ai_output": text.strip()}

            existing = store.load_json(local_ai_file, [])
            if not isinstance(existing, list):
                existing = []

            existing.append(record)

            # ✅ direct save
            store.save_json(local_ai_file, existing)

            # Update domain summary
            try:
                updater = globals().get("update_domain_summary")
                if updater:
                    updater(store, local_domain, text[:200])
            except Exception as e:
                print(f"[baseline] ⚠️ update_domain_summary failed: {e}")

            return f"💾 Baseline AI output saved -> `{_debug_store_path(store, local_ai_file)}`"

        except Exception as e:
            print(f"[baseline] ❌ Save failed: {e}")
            return f"❌ Save failed: {e}"

    # -------------------------
    # Buttons UI
    # -------------------------
    with gr.Row():
        save_input_btn = gr.Button("💾 Save User Input", variant="secondary")
        gen_btn = gr.Button("🤖 Generate AI Recommendations", variant="primary")
        save_ai_btn = gr.Button("💾 Save AI Output", variant="secondary")

    inputs_list = [
        ui_inputs["height_input"],
        ui_inputs["weight_input"],
        ui_inputs["systolic"],
        ui_inputs["diastolic"],
        ui_inputs["rhr"],
        ui_inputs["body_fat"],
        ui_inputs["waist"],
        ui_inputs["activity_level"],
        ui_inputs["notes_input"],
    ]

    save_input_btn.click(fn=save_user_input, inputs=inputs_list, outputs=[status])
    gen_btn.click(fn=generate_ai, inputs=inputs_list, outputs=[output, status])
    save_ai_btn.click(fn=save_ai_output, inputs=[output], outputs=[status])

    # -------------------------
    # Demo Hook Registration (Baseline) — SAFE
    # -------------------------
    try:
        def _baseline_demo_generate():
            print("[baseline-demo] 🚀 Demo generate using saved baseline data")

            raw = store.load_json(local_user_file, [])
            if not isinstance(raw, list) or not raw:
                return "⚠️ No saved baseline demo input found.", "No data"

            last = raw[-1].get("user_input", {})
            if not isinstance(last, dict) or not last:
                return "⚠️ Saved baseline record is empty.", "No data"

            prompt = (
                "You are a preventive workplace health assistant.\n"
                "Analyze the following baseline health data and provide actionable, non-diagnostic recommendations.\n"
                "Focus on lifestyle prevention, general health risk awareness, and workplace habits.\n"
                "Mention missing values gently.\n"
                "Include red-flag warning signs if needed.\n\n"
                f"BASELINE DATA:\n{last}"
            )

            current_lang = lang_state.value if hasattr(lang_state, "value") else lang_state

            resp = gemini.generate(prompt, response_language=current_lang)
            text = resp.text.strip() if hasattr(resp, "text") else str(resp).strip()

            return text, "✅ Baseline demo output generated."

        register_demo_hook(
            domain=local_domain,
            label="Baseline Module",
            generate_fn=_baseline_demo_generate
        )

    except Exception as e:
        print(f"[baseline] ⚠️ Demo hook registration failed: {e}")


## Final Assembly — Baseline Tab Builder

This function is the entry point used by the main Gradio app.

It:
- Loads the latest Baseline history
- Builds the input UI components
- Builds the output UI components
- Connects button callbacks to AI generation and saving logic

The Baseline module is usually the first module users complete, and it provides foundational data for other modules.


In [20]:
# ==========================================
# Baseline Module — Final Tab Builder (FIXED)
# ==========================================

def build_baseline_tab(
    store,
    gemini,
    lang_state: gr.State,
    locales_dir: Optional[Path] = None
):

    # -------------------------
    # Local safe constants
    # -------------------------
    local_domain = "baseline"
    local_user_file = f"{local_domain}_user_input.json"
    local_ai_file = f"{local_domain}_ai_output.json"

    print(f"[baseline] ✅ build_baseline_tab executed. user_file={local_user_file}, ai_file={local_ai_file}")

    lang = lang_state.value if hasattr(lang_state, "value") else lang_state

    # -------------------------
    # Load latest saved history
    # -------------------------
    saved_user_input, saved_ai_text = load_domain_history(store, local_domain)

    # -------------------------
    # UI
    # -------------------------
    gr.Markdown(
        _safe_t(
            lang,
            "baseline.title",
            locales_dir,
            default="Baseline Health Check-In"
        )
    )

    ui_inputs = build_baseline_inputs(saved_user_input, lang, locales_dir)

    output, status = build_baseline_outputs(saved_ai_text, lang, locales_dir)

    # -------------------------
    # Logic wiring
    # -------------------------
    connect_baseline_logic(
        store=store,
        gemini=gemini,
        lang_state=lang_state,
        locales_dir=locales_dir,
        ui_inputs=ui_inputs,
        output=output,
        status=status,
    )


Workspace Tab
The Workspace tab gathers information about the user's work environment or habits (like ergonomics, posture, or work setup). Based on these inputs, it generates recommendations to improve the workspace for better health (e.g., posture tips, ergonomic adjustments). The tab updates the context summary for the workspace domain, which can be used for global recommendations.

In [21]:
from IPython.display import Markdown, display

display(Markdown(WORKSPACE_INFO_MD))



# 🪑 Workspace & Ergonomics Module  
*Physical Environment & Posture Awareness*

The physical workspace plays a central role in musculoskeletal health, comfort,
and sustained productivity. Desk-based work often involves prolonged sitting,
repetitive movements, and fixed postures that can gradually place strain on the
neck, shoulders, back, wrists, and eyes—often without immediate pain.

This module is designed to support **workspace and posture awareness**, not
diagnosis. It helps identify everyday environmental and ergonomic factors that may
contribute to discomfort, fatigue, or long-term strain during desk work.

### What this module captures
- Type of workspace (office, home, shared)
- Chair and desk setup
- Screen height and viewing distance
- Keyboard and mouse positioning
- Sitting duration and movement breaks
- Posture-related discomfort or pain signals
- Use of ergonomic supports (chair, stand, external devices)

### Why this matters
Poor ergonomics rarely cause sudden injury. Instead, strain typically accumulates
through small mismatches between the body and the work environment—such as low
screens, unsupported seating, or long periods without movement.

Over time, these factors can contribute to musculoskeletal discomfort, reduced
concentration, and decreased work endurance. Early awareness allows for simple,
low-cost adjustments that may significantly reduce long-term risk.

### How AI is used in this section
The AI provides **non-diagnostic, preventive guidance** focused on:
- Practical ergonomic adjustments for desk-based work
- Posture and movement reminders that fit real workdays
- Reducing strain through small environmental changes
- Simple habits to support long-term musculoskeletal health

The goal is to promote comfort, sustainability, and prevention—not to replace
professional ergonomic or medical assessment.


In [22]:
# ==========================================
# Workspace Module — Config + History Load (FIXED)
# ==========================================

def load_workspace_history(store):
    """
    Loads the most recent saved user input + AI output for the Workspace module.
    Uses the unified load_domain_history() helper.
    """
    local_domain = "workspace"
    return load_domain_history(store, local_domain)


## Step 1 — User Inputs (Workspace Setup + Daily Habits)

This section builds the workspace input UI.

The user records:
- Whether posture was good today
- Break frequency and eating at desk
- Workstation equipment and physical setup
- Environmental factors such as noise and temperature
- Additional notes

These signals are later converted into a structured JSON payload and passed to Gemini.


In [23]:
# ==========================================
# Workspace Module — Inputs UI Builder
# ==========================================

import gradio as gr


def build_workspace_inputs(saved_user_input: dict, lang: str, locales_dir: Optional[Path]):

    gr.Markdown("### Daily Habits")

    with gr.Row():
        posture = gr.Checkbox(
            value=bool(saved_user_input.get("good_posture", False)),
            label=_safe_t(lang, "workspace.posture", locales_dir, default="Maintained good posture today")
        )

        breaks = gr.Radio(
            choices=["Hourly breaks", "Few breaks", "No breaks"],
            value=saved_user_input.get("breaks", "Few breaks"),
            label=_safe_t(lang, "workspace.breaks", locales_dir, default="Break Frequency")
        )

        eat_at_desk = gr.Checkbox(
            value=bool(saved_user_input.get("eat_at_desk", False)),
            label="Ate lunch at desk"
        )

    with gr.Accordion("Workstation Equipment & Geometry (Click to Expand)", open=False):
        gr.Markdown("Update this section only when your physical setup changes.")

        with gr.Row():
            input_device = gr.Dropdown(
                choices=["Standard Mouse", "Vertical/Ergo Mouse", "Trackpad", "Trackball"],
                value=saved_user_input.get("input_device", "Standard Mouse"),
                label="Primary Input Device"
            )

            keyboard_type = gr.Dropdown(
                choices=["Standard", "Mechanical", "Split/Ergonomic", "Laptop Keyboard"],
                value=saved_user_input.get("keyboard_type", "Standard"),
                label="Keyboard Type"
            )

        with gr.Row():
            wrist_support = gr.Radio(
                choices=["Yes", "No"],
                value=saved_user_input.get("wrist_support", "No"),
                label="Uses Wrist Support?"
            )

            armrests = gr.Dropdown(
                choices=["Level with desk", "Too High", "Too Low", "None"],
                value=saved_user_input.get("armrests", "Level with desk"),
                label="Armrest Position"
            )

            lumbar = gr.Checkbox(
                value=bool(saved_user_input.get("lumbar_support", True)),
                label="Chair has Lumbar Support"
            )

        with gr.Row():
            monitor_height = gr.Dropdown(
                choices=["Eye Level", "Below Eye Level (Looking Down)", "Above Eye Level"],
                value=saved_user_input.get("monitor_height", "Below Eye Level (Looking Down)"),
                label="Monitor Height"
            )

            feet_position = gr.Dropdown(
                choices=["Flat on floor", "On footrest", "Dangling/Crossed"],
                value=saved_user_input.get("feet_position", "Flat on floor"),
                label="Feet Position"
            )

    with gr.Accordion("Environment & Atmosphere", open=False):
        with gr.Row():
            noise = gr.Dropdown(
                choices=["Silent", "Hum/White Noise", "Distracting/Loud"],
                value=saved_user_input.get("noise_level", "Hum/White Noise"),
                label="Noise Level"
            )

            temp = gr.Dropdown(
                choices=["Cold", "Comfortable", "Warm"],
                value=saved_user_input.get("temperature", "Comfortable"),
                label="Room Temperature"
            )

            clutter = gr.Dropdown(
                choices=["Minimal", "Average", "Cluttered"],
                value=saved_user_input.get("clutter", "Average"),
                label="Desk Clutter"
            )

    notes_input = gr.Textbox(
        value=saved_user_input.get("notes", ""),
        label=_safe_t(lang, "workspace.notes", locales_dir, default="Additional Notes"),
        lines=2
    )

    return {
        "posture": posture,
        "breaks": breaks,
        "eat_at_desk": eat_at_desk,
        "input_device": input_device,
        "keyboard_type": keyboard_type,
        "wrist_support": wrist_support,
        "armrests": armrests,
        "lumbar": lumbar,
        "monitor_height": monitor_height,
        "feet_position": feet_position,
        "noise": noise,
        "temp": temp,
        "clutter": clutter,
        "notes_input": notes_input,
    }


## Step 2 — Output Components

This section defines:

- A status message area (save / error / generation feedback)
- A read-only AI output textbox that displays ergonomic recommendations

The latest saved output is loaded automatically to preserve continuity.


In [24]:
# ==========================================
# Workspace Module — Output UI Builder
# ==========================================

def build_workspace_outputs(saved_ai_text: str, lang: str, locales_dir: Optional[Path]):

    status = gr.Markdown("")

    output = gr.Textbox(
        label=_safe_t(lang, "workspace.output", locales_dir, default="Workspace Recommendations"),
        interactive=False,
        value=saved_ai_text,
        lines=10
    )

    return output, status


## Step 3 — AI Recommendations + Saving

This section handles the core logic:

- Collecting workspace inputs into a structured JSON object
- Saving user input history to `workspace_user_input.json`
- Sending the workspace data to Gemini
- Saving AI output history to `workspace_ai_output.json`

The generated recommendations focus on ergonomic improvements, micro-break planning,
and prevention of pain and fatigue.


In [25]:
# ==========================================
# Workspace Module — AI + Saving + Wiring (FIXED)
# ==========================================

def connect_workspace_logic(
    store,
    gemini,
    lang_state: gr.State,
    locales_dir: Optional[Path],
    ui_inputs: dict,
    output: gr.Textbox,
    status: gr.Markdown,
):

    # -----------------------------------
    # Local domain config (FIXED)
    # -----------------------------------
    local_domain = "workspace"
    local_user_file = f"{local_domain}_user_input.json"
    local_ai_file = f"{local_domain}_ai_output.json"

    # -------------------------
    # Collect input helper
    # -------------------------
    def collect_user_input(
        posture_val, breaks_val, eat_val,
        input_dev, key_type, wrist, arms, lumb, mon_h, feet,
        noise_val, temp_val, clutter_val,
        notes_val
    ):
        return {
            "good_posture": _to_bool(posture_val),
            "breaks": _to_str(breaks_val, ""),
            "eat_at_desk": _to_bool(eat_val),
            "input_device": _to_str(input_dev, ""),
            "keyboard_type": _to_str(key_type, ""),
            "wrist_support": _to_str(wrist, ""),
            "armrests": _to_str(arms, ""),
            "lumbar_support": _to_bool(lumb),
            "monitor_height": _to_str(mon_h, ""),
            "feet_position": _to_str(feet, ""),
            "noise_level": _to_str(noise_val, ""),
            "temperature": _to_str(temp_val, ""),
            "clutter": _to_str(clutter_val, ""),
            "notes": _to_str(notes_val, "").strip(),
        }

    # -------------------------
    # Save user input (APPEND)
    # -------------------------
    def save_user_input(
        posture_val, breaks_val, eat_val,
        input_dev, key_type, wrist, arms, lumb, mon_h, feet,
        noise_val, temp_val, clutter_val,
        notes_val
    ):
        print("[workspace] 🔘 Save User Input clicked")

        try:
            user_input = collect_user_input(
                posture_val, breaks_val, eat_val,
                input_dev, key_type, wrist, arms, lumb, mon_h, feet,
                noise_val, temp_val, clutter_val,
                notes_val
            )

            record = {"timestamp": _now(), "user_input": user_input}

            existing = store.load_json(local_user_file, [])
            if not isinstance(existing, list):
                existing = []

            existing.append(record)
            _save_json_best_effort(store, local_user_file, existing)

            return f"✅ Workspace user input saved -> `{_debug_store_path(store, local_user_file)}`"
        except Exception as e:
            print(f"[workspace] ❌ Save failed: {e}")
            return f"❌ Save failed: {e}"

    # -------------------------
    # Generate AI output
    # -------------------------
    def generate_ai(
        posture_val, breaks_val, eat_val,
        input_dev, key_type, wrist, arms, lumb, mon_h, feet,
        noise_val, temp_val, clutter_val,
        notes_val
    ):
        print("[workspace] 🔘 Generate AI clicked")

        user_input = collect_user_input(
            posture_val, breaks_val, eat_val,
            input_dev, key_type, wrist, arms, lumb, mon_h, feet,
            noise_val, temp_val, clutter_val,
            notes_val
        )

        prompt = (
            "You are a preventive workplace ergonomics assistant.\n"
            "Analyze the following workspace data and provide actionable, non-diagnostic recommendations.\n"
            "Identify ergonomic risks and suggest specific physical adjustments.\n"
            "If breaks are 'No breaks', recommend a realistic micro-break plan.\n\n"
            f"WORKSPACE DATA:\n{json.dumps(user_input, indent=2, ensure_ascii=False)}"
        )

        current_lang = lang_state.value if hasattr(lang_state, "value") else lang_state

        try:
            resp = gemini.generate(prompt=prompt, response_language=current_lang)
            text = resp.text.strip() if hasattr(resp, "text") and resp.text else str(resp).strip()
        except Exception as e:
            return "", f"⚠️ AI generation failed: {e}"

        if not text:
            return "", "⚠️ AI returned empty output."

        return text, "✅ AI output generated (not saved yet)."

    # -------------------------
    # Save AI output (APPEND)
    # -------------------------
    def save_ai_output(text: str):
        print("[workspace] 🔘 Save AI Output clicked")

        if not (text or "").strip():
            return "⚠️ Nothing to save."

        try:
            record = {"timestamp": _now(), "ai_output": text.strip()}

            existing = store.load_json(local_ai_file, [])
            if not isinstance(existing, list):
                existing = []

            existing.append(record)
            _save_json_best_effort(store, local_ai_file, existing)

            # Update domain summary
            try:
                updater = globals().get("update_domain_summary")
                if updater:
                    updater(store, local_domain, text[:200])
            except Exception as e:
                print(f"[workspace] ⚠️ update_domain_summary failed: {e}")

            return f"💾 Workspace AI output saved -> `{_debug_store_path(store, local_ai_file)}`"
        except Exception as e:
            print(f"[workspace] ❌ Save failed: {e}")
            return f"❌ Save failed: {e}"

    # -------------------------
    # Buttons UI
    # -------------------------
    with gr.Row():
        save_input_btn = gr.Button("💾 Save User Input", variant="secondary")
        gen_btn = gr.Button("🤖 Generate AI Recommendations", variant="primary")
        save_ai_btn = gr.Button("💾 Save AI Output", variant="secondary")

    inputs_list = [
        ui_inputs["posture"],
        ui_inputs["breaks"],
        ui_inputs["eat_at_desk"],
        ui_inputs["input_device"],
        ui_inputs["keyboard_type"],
        ui_inputs["wrist_support"],
        ui_inputs["armrests"],
        ui_inputs["lumbar"],
        ui_inputs["monitor_height"],
        ui_inputs["feet_position"],
        ui_inputs["noise"],
        ui_inputs["temp"],
        ui_inputs["clutter"],
        ui_inputs["notes_input"],
    ]

    save_input_btn.click(fn=save_user_input, inputs=inputs_list, outputs=[status])
    gen_btn.click(fn=generate_ai, inputs=inputs_list, outputs=[output, status])
    save_ai_btn.click(fn=save_ai_output, inputs=[output], outputs=[status])

    # =====================================================
    # Demo Hook Registration (Workspace)  ✅ FIXED
    # =====================================================
    try:
        def _workspace_demo_generate():
            print("[workspace-demo] 🚀 Demo generate using saved workspace data")

            raw = store.load_json(local_user_file, [])
            if not isinstance(raw, list) or not raw:
                return "⚠️ No saved workspace demo input found.", "No data"

            last = raw[-1].get("user_input", {})
            if not isinstance(last, dict) or not last:
                return "⚠️ Saved workspace record is empty.", "No data"

            prompt = (
                "You are a preventive workplace ergonomics assistant.\n"
                "Analyze the following workspace setup data and provide actionable, non-diagnostic ergonomic recommendations.\n"
                "Focus on posture, chair/desk setup, screen height, breaks, and practical adjustments.\n"
                "Mention missing values gently.\n\n"
                f"WORKSPACE DATA:\n{json.dumps(last, indent=2, ensure_ascii=False)}"
            )

            current_lang = lang_state.value if hasattr(lang_state, "value") else lang_state

            resp = gemini.generate(prompt=prompt, response_language=current_lang)
            text = resp.text.strip() if hasattr(resp, "text") and resp.text else str(resp).strip()

            return text, "✅ Workspace demo output generated."

        register_demo_hook(
            domain=local_domain,
            label="Workspace Module",
            generate_fn=_workspace_demo_generate
        )

    except Exception as e:
        print(f"[workspace] ⚠️ Demo hook registration failed: {e}")


## Final Assembly — Workspace Tab Builder

This function is the main entry point for the Workspace tab.

It:
- Loads the latest saved history
- Builds the UI inputs and outputs
- Connects the UI buttons to saving + Gemini generation logic

The Workspace module supports ergonomic improvements and injury prevention for desk workers.


In [26]:
# ==========================================
# Workspace Module — Final Tab Builder (FIXED)
# ==========================================

def build_workspace_tab(
    store,
    gemini,
    lang_state: gr.State,
    locales_dir: Optional[Path] = None,
):

    # -------------------------
    # Local safe constants
    # -------------------------
    local_domain = "workspace"
    local_user_file = f"{local_domain}_user_input.json"
    local_ai_file = f"{local_domain}_ai_output.json"

    print(f"[workspace] ✅ build_workspace_tab executed. user_file={local_user_file}, ai_file={local_ai_file}")

    lang = lang_state.value if hasattr(lang_state, "value") else lang_state

    # -------------------------
    # Load latest saved history
    # -------------------------
    saved_user_input, saved_ai_text = load_workspace_history(store)

    # --- Title ---
    gr.Markdown(
        _safe_t(
            lang,
            "workspace.title",
            locales_dir,
            default="🪑 Workspace Ergonomics and Habits",
        )
    )

    # --- Info Panel Button (Workspace Tab) ---
    add_info_panel(
        title="📘 Workspace Tab Guide",
        content_md=WORKSPACE_INFO_MD,
        button_label="ℹ️ Workspace Info",
        open_label="❌ Hide Workspace Info",
    )

    # --- Inputs ---
    ui_inputs = build_workspace_inputs(
        saved_user_input,
        lang,
        locales_dir,
    )

    # --- Outputs ---
    output, status = build_workspace_outputs(
        saved_ai_text,
        lang,
        locales_dir,
    )

    # --- Logic wiring ---
    connect_workspace_logic(
        store=store,
        gemini=gemini,
        lang_state=lang_state,
        locales_dir=locales_dir,
        ui_inputs=ui_inputs,
        output=output,
        status=status,
    )


Longitudinal Tab
The Longitudinal tab handles long-term tracking of the user's health metrics or progress. It might log periodic inputs (like weekly or monthly check-ins) and uses the AI to identify trends or changes over time. This helps in understanding improvements or deteriorations in the user's health across all tracked metrics. The tab likely composes summaries of these trends and updates the context for the longitudinal domain.

# 📈 Longitudinal Module (Progress Tracker & Health Wallet)

This module acts as a personal "Health Wallet" where users can store repeated lab test results and health notes over time.

Key features:
- Manual entry of common lab values (CBC, metabolic panel, vitamins, hormones)
- Free-text journaling of symptoms, fatigue, energy, and progress
- Upload of PDF or image lab reports
- OCR extraction (via Gemini or OCR pipeline) to auto-fill lab fields
- AI interpretation of each longitudinal entry

All saved entries are stored as timestamped records to support trend tracking over time.


In [27]:
# ==========================================
# Longitudinal Module — Config + History Load (FIXED)
# ==========================================

def _safe_float(x):
    try:
        if x is None or x == "":
            return None
        return float(x)
    except Exception:
        return None


def load_longitudinal_history(store):
    """
    Loads the most recent saved user input + AI output for the Longitudinal module.
    Uses the unified load_domain_history() helper.
    """
    local_domain = "longitudinal"
    return load_domain_history(store, local_domain)


## Step 1 — User Inputs (Manual Entry + Upload + Journal)

This section builds the Longitudinal input UI.

Users can enter:
- Notes or journal text (symptoms, fatigue, energy, progress)
- Upload a lab report (PDF/image)
- Manual structured lab values (CBC, metabolic, vitamins, hormones)
- Custom marker (any test the user wants)

The uploaded file can be processed via OCR and used to auto-fill structured lab fields.


In [28]:
# ==========================================
# Longitudinal Module — Inputs UI Builder
# ==========================================

import gradio as gr


def build_longitudinal_inputs(saved_user_input: dict, lang: str, locales_dir: Optional[Path]):

    with gr.Row():
        log_input = gr.Textbox(
            label=_safe_t(lang, "longitudinal.log", locales_dir, default="Journal/Notes"),
            placeholder="How are you feeling? Or paste text from a report...",
            lines=3,
            value=saved_user_input.get("notes", "")
        )

        file_input = gr.File(
            label=_safe_t(lang, "longitudinal.upload", locales_dir, default="Upload Lab Report (PDF/Image)")
        )

    gr.Markdown("### Manual Lab Entry (Optional)")

    with gr.Accordion("🩸 CBC & Immunity (Energy Levels)", open=False):
        with gr.Row():
            hb = gr.Number(label="Hemoglobin (g/dL)", value=saved_user_input.get("hb"))
            wbc = gr.Number(label="WBC Count (10^9/L)", value=saved_user_input.get("wbc"))
            platelets = gr.Number(label="Platelets (10^9/L)", value=saved_user_input.get("platelets"))

    with gr.Accordion("🍬 Metabolic & Heart Health", open=False):
        with gr.Row():
            glucose = gr.Number(label="Fasting Glucose (mg/dL)", value=saved_user_input.get("glucose"))
            hba1c = gr.Number(label="HbA1c (%)", value=saved_user_input.get("hba1c"))
        with gr.Row():
            cholesterol = gr.Number(label="Total Cholesterol (mg/dL)", value=saved_user_input.get("cholesterol"))
            triglycerides = gr.Number(label="Triglycerides (mg/dL)", value=saved_user_input.get("triglycerides"))

    with gr.Accordion("⚡ Focus Drivers (Vitamins & Hormones)", open=False):
        with gr.Row():
            vit_d = gr.Number(label="Vitamin D (ng/mL)", value=saved_user_input.get("vit_d"))
            vit_b12 = gr.Number(label="Vitamin B12 (pg/mL)", value=saved_user_input.get("vit_b12"))
            tsh = gr.Number(label="TSH (Thyroid) (uIU/mL)", value=saved_user_input.get("tsh"))

    with gr.Accordion("🔧 Custom / Other Marker", open=False):
        with gr.Row():
            custom_name = gr.Textbox(label="Marker Name (e.g. Magnesium)", value=saved_user_input.get("custom_name", ""))
            custom_val = gr.Number(label="Value", value=saved_user_input.get("custom_value"))
            custom_unit = gr.Textbox(label="Unit", value=saved_user_input.get("custom_unit", ""))

    return {
        "log_input": log_input,
        "file_input": file_input,

        "hb": hb,
        "wbc": wbc,
        "platelets": platelets,

        "glucose": glucose,
        "hba1c": hba1c,
        "cholesterol": cholesterol,
        "triglycerides": triglycerides,

        "vit_d": vit_d,
        "vit_b12": vit_b12,
        "tsh": tsh,

        "custom_name": custom_name,
        "custom_val": custom_val,
        "custom_unit": custom_unit,
    }


## Step 2 — Output Components

This section defines the main output UI:

- A status box for extraction/saving feedback
- A read-only AI output textbox showing the interpretation of the current entry

The latest saved interpretation is automatically loaded when the tab is opened.


In [29]:
# ==========================================
# Longitudinal Module — Output UI Builder
# ==========================================

def build_longitudinal_outputs(saved_ai_text: str, lang: str, locales_dir: Optional[Path]):

    status_box = gr.Markdown("")

    output = gr.Textbox(
        label=_safe_t(lang, "longitudinal.output", locales_dir, default="Longitudinal Analysis"),
        interactive=False,
        value=saved_ai_text,
        lines=12
    )

    return output, status_box


## Step 3A — OCR + Auto-Fill Extraction

This part implements automatic extraction of lab values.

Workflow:
1. User writes notes OR uploads a lab report (PDF/image)
2. OCR extracts raw text from the file (if provided)
3. Gemini receives the text and returns STRICT JSON
4. Extracted values are parsed and inserted into the structured lab fields

This allows the app to support paper-based lab reports and scanned PDFs.


In [30]:
# ==========================================
# Longitudinal Module — OCR + Auto-Fill Extractor
# ==========================================

def build_longitudinal_extractor(store, gemini, lang_state, ui_inputs, status_box):
    """
    Creates the "Extract Lab Values" button and its callback.
    Returns the extract button so it can be used in the main tab.
    """

    def extract_structured_from_text_and_file(log_text, file):
        print("[longitudinal] 🔘 Extract Lab Values clicked")

        text_content = (log_text or "").strip()

        # OCR if file provided
        if file is not None:
            try:
                extractor = globals().get("extract_text_from_file")
                if extractor:
                    extracted = extractor(
                        gemini,
                        file.name,
                        lang=lang_state.value if hasattr(lang_state, "value") else lang_state,
                    )
                    extracted = extracted.strip() if extracted else ""
                    if extracted:
                        text_content += f"\n\n[FILE CONTENT START]\n{extracted}\n[FILE CONTENT END]"
                else:
                    return (
                        None, None, None,
                        None, None, None, None,
                        None, None, None,
                        "", None, "",
                        "⚠️ OCR function not found."
                    )
            except Exception as e:
                return (
                    None, None, None,
                    None, None, None, None,
                    None, None, None,
                    "", None, "",
                    f"⚠️ OCR failed: {e}"
                )

        if not text_content.strip():
            return (
                None, None, None,
                None, None, None, None,
                None, None, None,
                "", None, "",
                "⚠️ No text or file content provided."
            )

        prompt = (
            "You are a medical data extraction assistant.\n"
            "Extract any lab values from the text below.\n\n"
            "Return STRICT JSON only (no explanation, no markdown).\n"
            "JSON schema:\n"
            "{\n"
            '  "cbc": {"hb": number|null, "wbc": number|null, "platelets": number|null},\n'
            '  "metabolic": {"glucose": number|null, "hba1c": number|null, "cholesterol": number|null, "triglycerides": number|null},\n'
            '  "vitamins": {"vit_d": number|null, "vit_b12": number|null, "tsh": number|null},\n'
            '  "custom": {"name": string|null, "value": number|null, "unit": string|null}\n'
            "}\n\n"
            f"TEXT:\n{text_content}\n"
        )

        current_lang = lang_state.value if hasattr(lang_state, "value") else lang_state

        try:
            resp = gemini.generate(prompt=prompt, response_language=current_lang)
            raw = resp.text.strip() if hasattr(resp, "text") and resp.text else ""
        except Exception as e:
            return (
                None, None, None,
                None, None, None, None,
                None, None, None,
                "", None, "",
                f"⚠️ Extraction AI failed: {e}"
            )

        if not raw:
            return (
                None, None, None,
                None, None, None, None,
                None, None, None,
                "", None, "",
                "⚠️ Model returned empty extraction output."
            )

        try:
            extracted_json = json.loads(raw)
        except Exception:
            return (
                None, None, None,
                None, None, None, None,
                None, None, None,
                "", None, "",
                "⚠️ Extraction failed: model did not return valid JSON."
            )

        cbc = extracted_json.get("cbc", {}) if isinstance(extracted_json, dict) else {}
        metabolic = extracted_json.get("metabolic", {}) if isinstance(extracted_json, dict) else {}
        vitamins = extracted_json.get("vitamins", {}) if isinstance(extracted_json, dict) else {}
        custom = extracted_json.get("custom", {}) if isinstance(extracted_json, dict) else {}

        hb_val = _safe_float(cbc.get("hb"))
        wbc_val = _safe_float(cbc.get("wbc"))
        plt_val = _safe_float(cbc.get("platelets"))

        gluc_val = _safe_float(metabolic.get("glucose"))
        a1c_val = _safe_float(metabolic.get("hba1c"))
        chol_val = _safe_float(metabolic.get("cholesterol"))
        tri_val = _safe_float(metabolic.get("triglycerides"))

        d_val = _safe_float(vitamins.get("vit_d"))
        b12_val = _safe_float(vitamins.get("vit_b12"))
        tsh_val = _safe_float(vitamins.get("tsh"))

        cust_name = custom.get("name") or ""
        cust_value = _safe_float(custom.get("value"))
        cust_unit = custom.get("unit") or ""

        return (
            hb_val, wbc_val, plt_val,
            gluc_val, a1c_val, chol_val, tri_val,
            d_val, b12_val, tsh_val,
            cust_name, cust_value, cust_unit,
            "✅ Extracted values filled into the fields."
        )

    extract_btn = gr.Button("Extract Lab Values (Auto-Fill)", variant="secondary")

    extract_btn.click(
        fn=extract_structured_from_text_and_file,
        inputs=[ui_inputs["log_input"], ui_inputs["file_input"]],
        outputs=[
            ui_inputs["hb"], ui_inputs["wbc"], ui_inputs["platelets"],
            ui_inputs["glucose"], ui_inputs["hba1c"], ui_inputs["cholesterol"], ui_inputs["triglycerides"],
            ui_inputs["vit_d"], ui_inputs["vit_b12"], ui_inputs["tsh"],
            ui_inputs["custom_name"], ui_inputs["custom_val"], ui_inputs["custom_unit"],
            status_box
        ]
    )

    return extract_btn


## Step 3B — Longitudinal Reasoning + Saving History

This section performs the main reasoning step:

- Combines structured lab fields + notes + OCR text (if provided)
- Sends the combined entry to Gemini for interpretation
- Highlights possible abnormal markers and their relevance to energy, focus, fatigue, and desk-work performance
- Saves user input history and AI output history as timestamped records

This enables the platform to function as a long-term personal health progress tracker.


In [31]:
# ==========================================
# Longitudinal Module — AI + Saving + Wiring (FIXED)
# ==========================================

def connect_longitudinal_logic(
    store,
    gemini,
    lang_state: gr.State,
    locales_dir: Optional[Path],
    ui_inputs: dict,
    output: gr.Textbox,
    status_box: gr.Markdown,
):

    # -------------------------
    # LOCAL DOMAIN CONSTANTS
    # -------------------------
    local_domain = "longitudinal"
    local_user_file = f"{local_domain}_user_input.json"
    local_ai_file = f"{local_domain}_ai_output.json"

    # -------------------------
    # Collect user input helper
    # -------------------------
    def collect_user_input(
        log_text,
        hb_v, wbc_v, plt_v,
        gluc_v, a1c_v, chol_v, tri_v,
        d_v, b12_v, tsh_v,
        cust_n, cust_v, cust_u
    ):
        return {
            "notes": (log_text or "").strip(),
            "hb": hb_v,
            "wbc": wbc_v,
            "platelets": plt_v,
            "glucose": gluc_v,
            "hba1c": a1c_v,
            "cholesterol": chol_v,
            "triglycerides": tri_v,
            "vit_d": d_v,
            "vit_b12": b12_v,
            "tsh": tsh_v,
            "custom_name": cust_n,
            "custom_value": cust_v,
            "custom_unit": cust_u,
        }

    # -------------------------
    # Save user input (APPEND)
    # -------------------------
    def save_user_input(
        log_text,
        hb_v, wbc_v, plt_v,
        gluc_v, a1c_v, chol_v, tri_v,
        d_v, b12_v, tsh_v,
        cust_n, cust_v, cust_u
    ):
        print("[longitudinal] 🔘 Save User Input clicked")

        try:
            user_input = collect_user_input(
                log_text,
                hb_v, wbc_v, plt_v,
                gluc_v, a1c_v, chol_v, tri_v,
                d_v, b12_v, tsh_v,
                cust_n, cust_v, cust_u
            )

            record = {"timestamp": _now(), "user_input": user_input}

            existing = store.load_json(local_user_file, [])
            if not isinstance(existing, list):
                existing = []

            existing.append(record)
            _save_json_best_effort(store, local_user_file, existing)

            return f"✅ Longitudinal user input saved -> `{_debug_store_path(store, local_user_file)}`"

        except Exception as e:
            print(f"[longitudinal] ❌ Save failed: {e}")
            return f"❌ Save failed: {e}"

    # -------------------------
    # Generate AI interpretation
    # -------------------------
    def generate_ai(
        log_text, file,
        hb_v, wbc_v, plt_v,
        gluc_v, a1c_v, chol_v, tri_v,
        d_v, b12_v, tsh_v,
        cust_n, cust_v, cust_u
    ):
        print("[longitudinal] 🔘 Generate AI clicked")

        text_content = (log_text or "").strip()

        # OCR if file provided
        if file is not None:
            try:
                extractor = globals().get("extract_text_from_file")
                if extractor:
                    extracted = extractor(
                        gemini,
                        file.name,
                        lang=lang_state.value if hasattr(lang_state, "value") else lang_state,
                    )
                    extracted = extracted.strip() if extracted else ""
                    if extracted:
                        text_content += f"\n\n[FILE CONTENT START]\n{extracted}\n[FILE CONTENT END]"
                else:
                    text_content += "\n[OCR Function Not Found]"
            except Exception as e:
                text_content += f"\n[OCR failed: {e}]"

        user_input = collect_user_input(
            text_content,
            hb_v, wbc_v, plt_v,
            gluc_v, a1c_v, chol_v, tri_v,
            d_v, b12_v, tsh_v,
            cust_n, cust_v, cust_u
        )

        if not text_content.strip() and all(v is None or v == "" for v in [
            hb_v, wbc_v, plt_v, gluc_v, a1c_v, chol_v, tri_v, d_v, b12_v, tsh_v, cust_v
        ]):
            return "", "⚠️ No input provided."

        prompt = (
            "You are a preventive health reasoning assistant.\n"
            "Interpret the following longitudinal entry.\n"
            "Highlight abnormal values and explain potential impact on desk-work energy, focus, fatigue, sleep.\n"
            "Provide non-diagnostic actionable guidance and when to seek medical review.\n\n"
            f"CURRENT ENTRY:\n{json.dumps(user_input, indent=2, ensure_ascii=False)}\n"
        )

        current_lang = lang_state.value if hasattr(lang_state, "value") else lang_state

        try:
            analysis_resp = gemini.generate(prompt=prompt, response_language=current_lang)
            analysis_text = analysis_resp.text.strip() if hasattr(analysis_resp, "text") and analysis_resp.text else ""
        except Exception as e:
            return "", f"⚠️ AI generation failed: {e}"

        if not analysis_text:
            return "", "⚠️ AI returned empty output."

        return analysis_text, "✅ AI output generated (not saved yet)."

    # -------------------------
    # Save AI output (APPEND)
    # -------------------------
    def save_ai_output(text: str):
        print("[longitudinal] 🔘 Save AI Output clicked")

        if not (text or "").strip():
            return "⚠️ Nothing to save."

        try:
            record = {"timestamp": _now(), "ai_output": text.strip()}

            existing = store.load_json(local_ai_file, [])
            if not isinstance(existing, list):
                existing = []

            existing.append(record)
            _save_json_best_effort(store, local_ai_file, existing)

            # optional summary update
            try:
                updater = globals().get("update_domain_summary")
                if updater:
                    updater(store, local_domain, text[:200])
            except Exception as e:
                print(f"[longitudinal] ⚠️ update_domain_summary failed: {e}")

            return f"💾 Longitudinal AI output saved -> `{_debug_store_path(store, local_ai_file)}`"

        except Exception as e:
            print(f"[longitudinal] ❌ Save failed: {e}")
            return f"❌ Save failed: {e}"

    # -------------------------
    # Buttons UI
    # -------------------------
    with gr.Row():
        save_input_btn = gr.Button("💾 Save User Input", variant="secondary")
        gen_btn = gr.Button("🤖 Generate AI Interpretation", variant="primary")
        save_ai_btn = gr.Button("💾 Save AI Output", variant="secondary")

    save_inputs_list = [
        ui_inputs["log_input"],
        ui_inputs["hb"], ui_inputs["wbc"], ui_inputs["platelets"],
        ui_inputs["glucose"], ui_inputs["hba1c"], ui_inputs["cholesterol"], ui_inputs["triglycerides"],
        ui_inputs["vit_d"], ui_inputs["vit_b12"], ui_inputs["tsh"],
        ui_inputs["custom_name"], ui_inputs["custom_val"], ui_inputs["custom_unit"],
    ]

    gen_inputs_list = [
        ui_inputs["log_input"], ui_inputs["file_input"],
        ui_inputs["hb"], ui_inputs["wbc"], ui_inputs["platelets"],
        ui_inputs["glucose"], ui_inputs["hba1c"], ui_inputs["cholesterol"], ui_inputs["triglycerides"],
        ui_inputs["vit_d"], ui_inputs["vit_b12"], ui_inputs["tsh"],
        ui_inputs["custom_name"], ui_inputs["custom_val"], ui_inputs["custom_unit"],
    ]

    save_input_btn.click(fn=save_user_input, inputs=save_inputs_list, outputs=[status_box])
    gen_btn.click(fn=generate_ai, inputs=gen_inputs_list, outputs=[output, status_box])
    save_ai_btn.click(fn=save_ai_output, inputs=[output], outputs=[status_box])

    # -------------------------
    # Demo Hook Registration (Longitudinal) — FIXED
    # -------------------------
    try:
        def _longitudinal_demo_generate():
            print("[longitudinal-demo] 🚀 Demo generate using saved longitudinal data")

            raw = store.load_json(local_user_file, [])
            if not isinstance(raw, list) or not raw:
                return "⚠️ No saved longitudinal demo input found.", "No data"

            last = raw[-1].get("user_input", {})
            if not isinstance(last, dict) or not last:
                return "⚠️ Saved longitudinal record is empty.", "No data"

            prompt = (
                "You are a preventive workplace health trend assistant.\n"
                "Analyze the following longitudinal lab/health tracking data.\n"
                "Provide trend-based, non-diagnostic insights.\n"
                "Highlight possible improvement areas and when medical review may be needed.\n"
                "Mention missing values gently.\n\n"
                f"LONGITUDINAL DATA:\n{last}"
            )

            current_lang = lang_state.value if hasattr(lang_state, "value") else lang_state

            resp = gemini.generate(prompt, response_language=current_lang)
            text = resp.text.strip() if hasattr(resp, "text") else str(resp).strip()

            return text, "✅ Longitudinal demo output generated."

        register_demo_hook(
            domain=local_domain,
            label="Longitudinal Module",
            generate_fn=_longitudinal_demo_generate
        )

    except Exception as e:
        print(f"[longitudinal] ⚠️ Demo hook registration failed: {e}")


## Final Assembly — Longitudinal Tab Builder

This is the final wrapper function used by the main Gradio app.

It:
- Loads the latest saved longitudinal history
- Builds the input UI (notes, upload, structured lab fields)
- Builds the output UI (analysis + status)
- Adds OCR extraction button (auto-fill)
- Connects saving and Gemini reasoning logic

This module is the core "Health Wallet" component of the platform.


In [32]:
# ==========================================
# Longitudinal Module — Final Tab Builder
# ==========================================

def build_longitudinal_tab(store, gemini, lang_state: gr.State, locales_dir: Optional[Path] = None):
    print("[longitudinal] ✅ build_longitudinal_tab executed")

    lang = lang_state.value if hasattr(lang_state, "value") else lang_state

    saved_user_input, saved_ai_text = load_longitudinal_history(store)


    gr.Markdown(_safe_t(lang, "longitudinal.title", locales_dir, default="Longitudinal Progress Tracker & Health Wallet"))

    ui_inputs = build_longitudinal_inputs(saved_user_input, lang, locales_dir)

    output, status_box = build_longitudinal_outputs(saved_ai_text, lang, locales_dir)

    # OCR + Extraction Button
    build_longitudinal_extractor(store, gemini, lang_state, ui_inputs, status_box)

    # Saving + AI reasoning logic
    connect_longitudinal_logic(
        store=store,
        gemini=gemini,
        lang_state=lang_state,
        locales_dir=locales_dir,
        ui_inputs=ui_inputs,
        output=output,
        status_box=status_box,
    )


MSK Tab
The Musculoskeletal (MSK) tab focuses on physical health related to muscles and posture. The user can input information about physical activity, aches/pains, or posture habits. The AI then provides suggestions such as exercises, stretches, or ergonomic advice to improve musculoskeletal health. A summary of the user's MSK status or advice is stored for context.

In [33]:
from IPython.display import Markdown, display

display(Markdown(MSK_INFO_MD))



# 🦴 Musculoskeletal (MSK) Health Module  
*Movement, Load & Pain Awareness*

Musculoskeletal discomfort is one of the most common consequences of prolonged
desk-based work. Limited movement, static postures, repetitive tasks, and poor
load distribution can gradually affect the neck, shoulders, back, hips, and
upper limbs—often starting as mild discomfort before progressing further.

This module is designed to support **musculoskeletal health awareness**, not
diagnosis. It focuses on identifying movement patterns, pain signals, and daily
habits that may influence physical comfort and long-term joint and muscle health.

### What this module captures
- Presence and location of musculoskeletal pain
- Pain intensity and frequency
- Stiffness or reduced mobility
- Duration of sitting and physical inactivity
- Break frequency and movement habits
- Physical strain related to work tasks
- Use of stretching or physical activity for relief

### Why this matters
Musculoskeletal strain typically develops gradually rather than through sudden
injury. Ignoring early warning signs such as stiffness or recurring pain can allow
problems to accumulate over time.

By increasing awareness of pain patterns and movement behavior, this module helps
highlight opportunities for simple preventive actions that may reduce discomfort
and support long-term physical function.

### How AI is used in this section
The AI provides **non-diagnostic, preventive guidance** focused on:
- Gentle movement and stretching suggestions
- Reducing prolonged static load during work
- Encouraging regular movement breaks
- Supporting sustainable physical habits at work

The aim is prevention and early awareness—not medical assessment or treatment.


In [34]:
# ==========================================
# MSK Module — Config + History Load (FIXED)
# ==========================================

def load_msk_history(store):
    """
    Loads the most recent saved user input + AI output for the MSK module.
    Uses the unified load_domain_history() helper.
    """
    local_domain = "msk"
    return load_domain_history(store, local_domain)


## Step 1 — User Inputs (Symptoms + Triggers)

This section builds the MSK symptom input UI.

It captures:
- Pain severity (0–10)
- Timing of pain onset
- Pain location(s) and type of sensation
- Mobility limitations (neck ROM, sitting duration)
- Triggers and relief strategies
- Whether symptoms affect work and sleep

These inputs are converted into a structured JSON payload used by the AI reasoning engine.


In [35]:
# ==========================================
# MSK Module — Inputs UI Builder
# ==========================================

import gradio as gr


def build_msk_inputs(saved_user_input: dict, lang: str, locales_dir: Optional[Path]):

    gr.Markdown("### Current Symptoms")

    with gr.Row():
        pain_level = gr.Slider(
            minimum=0, maximum=10, step=1,
            value=_to_int(saved_user_input.get("pain_level", 0), 0),
            label=_safe_t(lang, "msk.pain", locales_dir, default="Pain Intensity (0-10)")
        )

        onset_choices = ["Upon Waking", "Mid-Day", "End of Workday", "After Specific Tasks", "Constant"]
        saved_onset = _to_str(saved_user_input.get("onset_timing", "End of Workday"), "End of Workday")
        if saved_onset not in onset_choices:
            saved_onset = "End of Workday"

        onset_timing = gr.Dropdown(
            choices=onset_choices,
            value=saved_onset,
            label="When does it start?"
        )

    focus_choices = ["Neck", "Shoulders/Upper Back", "Lower Back", "Wrists/Hands", "Hips/Glutes", "Knees", "None"]
    saved_focus = _to_list(saved_user_input.get("focus_area", []))
    saved_focus = [x for x in saved_focus if x in focus_choices]

    if not saved_focus:
        saved_focus = ["None"]

    nature_choices = ["Stiffness/Tightness", "Dull Ache", "Sharp/Stabbing", "Burning", "Numbness/Tingling"]
    saved_nature = _to_str(saved_user_input.get("pain_nature", "Stiffness/Tightness"), "Stiffness/Tightness")
    if saved_nature not in nature_choices:
        saved_nature = "Stiffness/Tightness"

    with gr.Row():
        focus_area = gr.Dropdown(
            choices=focus_choices,
            value=saved_focus,
            multiselect=True,
            label=_safe_t(lang, "msk.area", locales_dir, default="Location(s) of Discomfort")
        )

        pain_nature = gr.Dropdown(
            choices=nature_choices,
            value=saved_nature,
            label="Nature of Sensation"
        )

    with gr.Accordion("Mobility & Stiffness Checks", open=False):
        neck_choices = ["Full & Painless", "Limited (Stiff)", "Painful to Move"]
        saved_neck = _to_str(saved_user_input.get("neck_rom", "Full & Painless"), "Full & Painless")
        if saved_neck not in neck_choices:
            saved_neck = "Full & Painless"

        seat_choices = ["< 30 mins", "1 hour", "2+ hours"]
        saved_seat = _to_str(saved_user_input.get("seated_duration", "1 hour"), "1 hour")
        if saved_seat not in seat_choices:
            saved_seat = "1 hour"

        with gr.Row():
            neck_rom = gr.Radio(
                choices=neck_choices,
                value=saved_neck,
                label="Neck Range of Motion"
            )

            seated_duration = gr.Radio(
                choices=seat_choices,
                value=saved_seat,
                label="Typ. Time Seated w/o Move"
            )

        with gr.Row():
            morning_stiffness = gr.Checkbox(
                value=_to_bool(saved_user_input.get("morning_stiffness", False)),
                label="Wake up feeling stiff?"
            )

            posture = gr.Checkbox(
                value=_to_bool(saved_user_input.get("good_posture", False)),
                label=_safe_t(lang, "msk.posture", locales_dir, default="Maintains good posture")
            )

    with gr.Accordion("Impact & Triggers", open=False):
        triggers = gr.CheckboxGroup(
            choices=["Stress/Tension", "Heavy Lifting", "Long Typing Sessions", "Cold Temperature", "Poor Sleep"],
            value=_to_list(saved_user_input.get("triggers", [])),
            label="Suspected Triggers"
        )

        relief_methods = gr.CheckboxGroup(
            choices=["Stretching", "Walking", "Heat/Cold Pack", "Painkillers", "Rest"],
            value=_to_list(saved_user_input.get("relief_methods", [])),
            label="What helps?"
        )

        with gr.Row():
            impact_work = gr.Checkbox(
                value=_to_bool(saved_user_input.get("impact_work", False)),
                label="Interferes with work?"
            )

            impact_sleep = gr.Checkbox(
                value=_to_bool(saved_user_input.get("impact_sleep", False)),
                label="Interferes with sleep?"
            )

    notes_input = gr.Textbox(
        value=_to_str(saved_user_input.get("notes", ""), ""),
        label=_safe_t(lang, "msk.notes", locales_dir, default="Additional Notes"),
        lines=2
    )

    return {
        "pain_level": pain_level,
        "onset_timing": onset_timing,
        "focus_area": focus_area,
        "pain_nature": pain_nature,

        "neck_rom": neck_rom,
        "seated_duration": seated_duration,
        "morning_stiffness": morning_stiffness,
        "posture": posture,

        "triggers": triggers,
        "relief_methods": relief_methods,
        "impact_work": impact_work,
        "impact_sleep": impact_sleep,

        "notes_input": notes_input,
    }


## Step 2 — Output Components

This section defines:
- A status message box for feedback (save success/errors)
- A read-only AI output field containing MSK analysis and ergonomic recommendations

The latest saved AI output is automatically loaded to preserve continuity.


In [36]:
# ==========================================
# MSK Module — Output UI Builder
# ==========================================

def build_msk_outputs(saved_ai_text: str, lang: str, locales_dir: Optional[Path]):

    status = gr.Markdown("")

    output = gr.Textbox(
        label=_safe_t(lang, "msk.output", locales_dir, default="MSK Analysis & Recommendations"),
        interactive=False,
        value=saved_ai_text,
        lines=10
    )

    return output, status


## Step 3 — AI Reasoning + Saving

This section connects the MSK module to the reasoning engine.

Workflow:
1. Convert the UI inputs into a structured JSON object
2. Save the user input entry as a timestamped record
3. Generate AI recommendations using Gemini
4. Save the AI interpretation as a timestamped record

The model focuses on prevention, ergonomics, mobility, and red-flag symptom awareness.


In [37]:
# ==========================================
# MSK Module — AI + Saving + Wiring (FIXED)
# ==========================================

def connect_msk_logic(
    store,
    gemini,
    lang_state: gr.State,
    locales_dir: Optional[Path],
    ui_inputs: dict,
    output: gr.Textbox,
    status: gr.Markdown,
):

    # -------------------------
    # LOCAL DOMAIN CONSTANTS
    # -------------------------
    local_domain = "msk"
    local_user_file = f"{local_domain}_user_input.json"
    local_ai_file = f"{local_domain}_ai_output.json"

    # -------------------------
    # Collect user input helper
    # -------------------------
    def collect_user_input(
        pain_val, onset_val, focus_val, nature_val,
        neck_val, seat_val, morn_val, post_val,
        trig_val, relief_val, work_val, sleep_val,
        notes_val
    ):
        return {
            "pain_level": _to_int(pain_val, 0),
            "onset_timing": _to_str(onset_val, "End of Workday"),
            "focus_area": _to_list(focus_val),
            "pain_nature": _to_str(nature_val, "Stiffness/Tightness"),
            "neck_rom": _to_str(neck_val, "Full & Painless"),
            "seated_duration": _to_str(seat_val, "1 hour"),
            "morning_stiffness": _to_bool(morn_val),
            "good_posture": _to_bool(post_val),
            "triggers": _to_list(trig_val),
            "relief_methods": _to_list(relief_val),
            "impact_work": _to_bool(work_val),
            "impact_sleep": _to_bool(sleep_val),
            "notes": _to_str(notes_val, ""),
        }

    # -------------------------
    # Save user input (APPEND)
    # -------------------------
    def save_user_input(
        pain_val, onset_val, focus_val, nature_val,
        neck_val, seat_val, morn_val, post_val,
        trig_val, relief_val, work_val, sleep_val,
        notes_val
    ):
        print("[msk] 🔘 Save User Input clicked")

        try:
            user_input = collect_user_input(
                pain_val, onset_val, focus_val, nature_val,
                neck_val, seat_val, morn_val, post_val,
                trig_val, relief_val, work_val, sleep_val,
                notes_val
            )

            record = {"timestamp": _now(), "user_input": user_input}

            existing = store.load_json(local_user_file, [])
            if not isinstance(existing, list):
                existing = []

            existing.append(record)
            _save_json_best_effort(store, local_user_file, existing)

            return f"✅ MSK input saved. File: `{_debug_store_path(store, local_user_file)}`"

        except Exception as e:
            print(f"[msk] ❌ Save failed: {e}")
            return f"❌ Save failed: {e}"

    # -------------------------
    # Generate AI output
    # -------------------------
    def generate_ai(
        pain_val, onset_val, focus_val, nature_val,
        neck_val, seat_val, morn_val, post_val,
        trig_val, relief_val, work_val, sleep_val,
        notes_val
    ):
        print("[msk] 🔘 Generate AI clicked")

        user_input = collect_user_input(
            pain_val, onset_val, focus_val, nature_val,
            neck_val, seat_val, morn_val, post_val,
            trig_val, relief_val, work_val, sleep_val,
            notes_val
        )

        prompt = (
            "You are a preventive workplace musculoskeletal assistant.\n"
            "Analyze the following MSK data and provide actionable, non-diagnostic recommendations.\n"
            "1. Analyze relationship between pain nature, location, and triggers.\n"
            "2. If numbness/tingling exists, assess nerve compression risks.\n"
            "3. Suggest specific stretches and workstation adjustments.\n"
            "4. Provide red-flag warning signs requiring medical review.\n\n"
            f"MSK DATA:\n{json.dumps(user_input, indent=2, ensure_ascii=False)}"
        )

        current_lang = lang_state.value if hasattr(lang_state, "value") else lang_state

        try:
            resp = gemini.generate(prompt=prompt, response_language=current_lang)
            text = resp.text.strip() if hasattr(resp, "text") and resp.text else str(resp).strip()
        except Exception as e:
            print(f"[msk] ❌ AI generation failed: {e}")
            return "", f"⚠️ AI generation failed: {e}"

        if not text:
            return "", "⚠️ AI returned empty output."

        return text, "✅ AI output generated (not saved yet)."

    # -------------------------
    # Save AI output (APPEND)
    # -------------------------
    def save_ai_output(text: str):
        print("[msk] 🔘 Save AI Output clicked")

        if not (text or "").strip():
            return "⚠️ Nothing to save."

        try:
            record = {"timestamp": _now(), "ai_output": text.strip()}

            existing = store.load_json(local_ai_file, [])
            if not isinstance(existing, list):
                existing = []

            existing.append(record)
            _save_json_best_effort(store, local_ai_file, existing)

            try:
                updater = globals().get("update_domain_summary")
                if updater:
                    updater(store, local_domain, text[:200])
            except Exception as e:
                print(f"[msk] ⚠️ update_domain_summary failed: {e}")

            return f"💾 MSK AI output saved. File: `{_debug_store_path(store, local_ai_file)}`"

        except Exception as e:
            print(f"[msk] ❌ Save failed: {e}")
            return f"❌ Save failed: {e}"

    # -------------------------
    # Buttons UI
    # -------------------------
    with gr.Row():
        save_input_btn = gr.Button("💾 Save User Input", variant="secondary")
        gen_btn = gr.Button("🤖 Generate AI MSK Analysis", variant="primary")
        save_ai_btn = gr.Button("💾 Save AI Output", variant="secondary")

    inputs_list = [
        ui_inputs["pain_level"], ui_inputs["onset_timing"], ui_inputs["focus_area"], ui_inputs["pain_nature"],
        ui_inputs["neck_rom"], ui_inputs["seated_duration"], ui_inputs["morning_stiffness"], ui_inputs["posture"],
        ui_inputs["triggers"], ui_inputs["relief_methods"], ui_inputs["impact_work"], ui_inputs["impact_sleep"],
        ui_inputs["notes_input"]
    ]

    save_input_btn.click(fn=save_user_input, inputs=inputs_list, outputs=[status])
    gen_btn.click(fn=generate_ai, inputs=inputs_list, outputs=[output, status])
    save_ai_btn.click(fn=save_ai_output, inputs=[output], outputs=[status])

    # -------------------------
    # Demo Hook Registration (MSK) — FIXED
    # -------------------------
    try:
        def _msk_demo_generate():
            print("[msk-demo] 🚀 Demo generate using saved MSK data")

            raw = store.load_json(local_user_file, [])
            if not isinstance(raw, list) or not raw:
                return "⚠️ No saved MSK demo input found.", "No data"

            last = raw[-1].get("user_input", {})
            if not isinstance(last, dict) or not last:
                return "⚠️ Saved MSK record is empty.", "No data"

            prompt = (
                "You are a preventive workplace musculoskeletal assistant.\n"
                "Analyze the following MSK data and provide actionable, non-diagnostic recommendations.\n"
                "1. Analyze relationship between pain nature, location, and triggers.\n"
                "2. Suggest specific stretches and workstation adjustments.\n"
                "3. Provide red-flag warning signs requiring medical review.\n\n"
                f"MSK DATA:\n{last}"
            )

            current_lang = lang_state.value if hasattr(lang_state, "value") else lang_state

            resp = gemini.generate(prompt, response_language=current_lang)
            text = resp.text.strip() if hasattr(resp, "text") else str(resp).strip()

            return text, "✅ MSK demo output generated."

        register_demo_hook(
            domain=local_domain,
            label="MSK Module",
            generate_fn=_msk_demo_generate
        )

    except Exception as e:
        print(f"[msk] ⚠️ Demo hook registration failed: {e}")


## Final Assembly — MSK Tab Builder

This is the wrapper function used by the main Gradio app.

It:
- Loads the latest MSK history
- Builds the symptom input UI
- Builds the output components
- Connects AI generation + saving logic

The MSK module is designed for prevention of desk-related pain and early detection of warning patterns.


In [38]:
# ==========================================
# MSK Module — Final Tab Builder (FIXED)
# ==========================================

def build_msk_tab(
    store,
    gemini,
    lang_state: gr.State,
    locales_dir: Optional[Path] = None,
):

    local_domain = "msk"
    local_user_file = f"{local_domain}_user_input.json"
    local_ai_file = f"{local_domain}_ai_output.json"

    print(f"[msk] ✅ build_msk_tab executed. user_file={local_user_file}, ai_file={local_ai_file}")

    lang = lang_state.value if hasattr(lang_state, "value") else lang_state

    saved_user_input, saved_ai_text = load_msk_history(store)

    # --- Title ---
    gr.Markdown(
        _safe_t(
            lang,
            "msk.title",
            locales_dir,
            default="🦴 Musculoskeletal Health & Pain Tracker",
        )
    )

    # --- Info Panel Button (MSK Tab) ---
    add_info_panel(
        title="📘 MSK Tab Guide",
        content_md=MSK_INFO_MD,
        button_label="ℹ️ MSK Info",
        open_label="❌ Hide MSK Info",
    )

    # --- Inputs ---
    ui_inputs = build_msk_inputs(
        saved_user_input,
        lang,
        locales_dir,
    )

    # --- Outputs ---
    output, status = build_msk_outputs(
        saved_ai_text,
        lang,
        locales_dir,
    )

    # --- Logic wiring ---
    connect_msk_logic(
        store=store,
        gemini=gemini,
        lang_state=lang_state,
        locales_dir=locales_dir,
        ui_inputs=ui_inputs,
        output=output,
        status=status,
    )


Eye Tab
The Eye health tab addresses eye strain and vision care. It may take inputs like screen time, break frequency, or eye comfort level. Using those inputs, it outputs recommendations (for example, following the 20-20-20 rule, doing eye exercises, etc.) to reduce eye strain. The summary or key points are saved to context for later use.

In [39]:
from IPython.display import Markdown, display

display(Markdown(EYE_HEALTH_INFO_MD))



# 👁️ Eye Health Module  
*Visual Load & Screen Strain Awareness*

Extended screen exposure is a defining feature of modern desk work. Prolonged
visual focus, reduced blinking, glare, and improper screen positioning can place
significant strain on the eyes—often leading to symptoms such as dryness, blurred
vision, headaches, or eye fatigue.

This module is designed to support **eye health awareness**, not diagnosis. It
helps identify daily screen-related behaviors and visual symptoms that may affect
comfort, focus, and visual endurance during work.

### What this module captures
- Daily screen time duration
- Frequency of visual breaks
- Eye strain or discomfort symptoms
- Headaches related to screen use
- Screen distance and height
- Lighting conditions and glare
- Use of corrective lenses or filters

### Why this matters
Eye strain often develops silently and may be mistaken for general fatigue or
headache. Without regular visual breaks and proper screen setup, symptoms can
accumulate and affect both comfort and work performance.

Early awareness allows for small, practical adjustments that can significantly
reduce visual load and support long-term eye comfort.

### How AI is used in this section
The AI provides **non-diagnostic, preventive guidance** focused on:
- Visual break strategies suitable for workdays
- Screen positioning and lighting adjustments
- Habits that support eye comfort during screen use
- Reducing cumulative visual strain over time

The goal is to promote visual comfort and sustainability—not to replace eye care
assessment.


In [40]:
# ==========================================
# Eye Module — Config + History Load
# ==========================================




def load_eye_history(store):
    """
    Loads the most recent saved user input + AI output for the Eye module.
    Uses the unified load_domain_history() helper.
    """
    local_domain = "eye"
    return load_domain_history(store, local_domain)


## Step 1 — User Inputs (Symptoms + Environment + Habits)

This section builds the Eye Health input UI.

The user records:
- Strain severity and uninterrupted screen time
- Common Computer Vision Syndrome symptoms
- Lighting and glare environment
- Vision correction tools used (glasses/contacts)
- Eye care habits (drops, 20-20-20 rule)

These values are later structured into JSON and sent to Gemini for analysis.


In [41]:
# ==========================================
# Eye Module — Inputs UI Builder
# ==========================================

import gradio as gr


def build_eye_inputs(saved_user_input: dict, lang: str, locales_dir: Optional[Path]):

    gr.Markdown("### Current Eye Status")

    with gr.Row():
        strain_level = gr.Slider(
            minimum=0, maximum=10, step=1,
            value=_to_int(saved_user_input.get("strain_level", 0), 0),
            label=_safe_t(lang, "eye.strain", locales_dir, default="Eye Strain Level (0-10)")
        )

        session_choices = ["< 1 hour", "1-2 hours", "2-4 hours", "4+ hours"]
        saved_session = _to_str(saved_user_input.get("session_length", "1-2 hours"), "1-2 hours")
        if saved_session not in session_choices:
            saved_session = "1-2 hours"

        session_length = gr.Dropdown(
            choices=session_choices,
            value=saved_session,
            label="Longest Uninterrupted Screen Time"
        )

    symptoms = gr.CheckboxGroup(
        choices=[
            "Dryness / Gritty feeling",
            "Blurred Vision (end of day)",
            "Light Sensitivity",
            "Eye Twitching",
            "Headache (behind eyes)",
            "Watery Eyes"
        ],
        value=_to_list(saved_user_input.get("symptoms", [])),
        label="Specific Symptoms Today"
    )

    with gr.Accordion("Visual Environment & Lighting", open=False):

        lighting_choices = ["Natural Light", "Fluorescent (Office)", "Dim/Dark Room", "Mixed"]
        saved_lighting = _to_str(saved_user_input.get("lighting", "Natural Light"), "Natural Light")
        if saved_lighting not in lighting_choices:
            saved_lighting = "Natural Light"

        brightness_choices = ["Brighter than room", "Balanced", "Dimmer than room"]
        saved_brightness = _to_str(saved_user_input.get("screen_brightness", "Balanced"), "Balanced")
        if saved_brightness not in brightness_choices:
            saved_brightness = "Balanced"

        with gr.Row():
            lighting = gr.Dropdown(
                choices=lighting_choices,
                value=saved_lighting,
                label="Ambient Lighting"
            )

            screen_brightness = gr.Radio(
                choices=brightness_choices,
                value=saved_brightness,
                label="Screen Brightness vs. Room"
            )

        with gr.Row():
            glare = gr.Checkbox(
                value=_to_bool(saved_user_input.get("glare", False)),
                label="Glare/Reflection on Screen?"
            )

            distance_check = gr.Checkbox(
                value=_to_bool(saved_user_input.get("distance_check", True)),
                label="Screen is arm's length away?"
            )

    with gr.Accordion("Correction & Habits", open=False):

        correction_choices = ["None (Naked Eye)", "Glasses", "Contact Lenses", "Blue Light Blockers"]
        saved_corr = _to_str(saved_user_input.get("correction", "None (Naked Eye)"), "None (Naked Eye)")
        if saved_corr not in correction_choices:
            saved_corr = "None (Naked Eye)"

        rule_choices = ["Strictly followed", "Occasionally", "Forgot completely"]
        saved_rule = _to_str(saved_user_input.get("rule_20_20_20", "Occasionally"), "Occasionally")
        if saved_rule not in rule_choices:
            saved_rule = "Occasionally"

        with gr.Row():
            correction = gr.Dropdown(
                choices=correction_choices,
                value=saved_corr,
                label="Vision Correction Worn"
            )

            rule_20_20_20 = gr.Radio(
                choices=rule_choices,
                value=saved_rule,
                label="Adherence to 20-20-20 Rule"
            )

        with gr.Row():
            drops = gr.Checkbox(
                value=_to_bool(saved_user_input.get("used_drops", False)),
                label="Used Artificial Tears?"
            )

    notes_input = gr.Textbox(
        value=_to_str(saved_user_input.get("notes", ""), ""),
        label=_safe_t(lang, "eye.notes", locales_dir, default="Additional Observations"),
        lines=2
    )

    return {
        "strain_level": strain_level,
        "session_length": session_length,
        "symptoms": symptoms,

        "lighting": lighting,
        "screen_brightness": screen_brightness,
        "glare": glare,
        "distance_check": distance_check,

        "correction": correction,
        "rule_20_20_20": rule_20_20_20,
        "drops": drops,

        "notes_input": notes_input,
    }


## Step 2 — Output Components

This section defines:
- A status box to display save/generation messages
- A read-only AI output field for eye-care recommendations

The latest saved AI response is loaded automatically for continuity.


In [42]:
# ==========================================
# Eye Module — Output UI Builder
# ==========================================

def build_eye_outputs(saved_ai_text: str, lang: str, locales_dir: Optional[Path]):

    status = gr.Markdown("")

    output = gr.Textbox(
        label=_safe_t(lang, "eye.output", locales_dir, default="Eye Care Recommendations"),
        interactive=False,
        value=saved_ai_text,
        lines=10
    )

    return output, status


## Step 3 — AI Reasoning + Saving

This section connects the Eye module to Gemini.

Workflow:
1. Collect user inputs into structured JSON
2. Save user input history (timestamped records)
3. Generate non-diagnostic eye-care recommendations
4. Save AI output history

The AI focuses on Computer Vision Syndrome prevention, dryness relief, and healthy screen habits.


In [43]:
# ==========================================
# Eye Module — AI + Saving + Wiring (FIXED)
# ==========================================

def connect_eye_logic(
    store,
    gemini,
    lang_state: gr.State,
    locales_dir: Optional[Path],
    ui_inputs: dict,
    output: gr.Textbox,
    status: gr.Markdown,
):

    # IMPORTANT: local names (prevents cross-tab overwrite bugs)
    domain = "eye"
    user_file = f"{domain}_user_input.json"
    ai_file = f"{domain}_ai_output.json"

    # -------------------------
    # Collect input helper
    # -------------------------
    def collect_user_input(
        strain_val, session_val, symp_val,
        light_val, bright_val, glare_val, dist_val,
        corr_val, rule_val, drops_val,
        notes_val
    ):
        return {
            "strain_level": _to_int(strain_val, 0),
            "session_length": _to_str(session_val, "1-2 hours"),
            "symptoms": _to_list(symp_val),
            "lighting": _to_str(light_val, "Natural Light"),
            "screen_brightness": _to_str(bright_val, "Balanced"),
            "glare": _to_bool(glare_val),
            "distance_check": _to_bool(dist_val),
            "correction": _to_str(corr_val, "None (Naked Eye)"),
            "rule_20_20_20": _to_str(rule_val, "Occasionally"),
            "used_drops": _to_bool(drops_val),
            "notes": _to_str(notes_val, ""),
        }

    # -------------------------
    # Save user input (APPEND)
    # -------------------------
    def save_user_input(
        strain_val, session_val, symp_val,
        light_val, bright_val, glare_val, dist_val,
        corr_val, rule_val, drops_val,
        notes_val
    ):
        print("[eye] 🔘 Save User Input clicked")

        try:
            user_input = collect_user_input(
                strain_val, session_val, symp_val,
                light_val, bright_val, glare_val, dist_val,
                corr_val, rule_val, drops_val,
                notes_val
            )

            record = {"timestamp": _now(), "user_input": user_input}

            existing = store.load_json(user_file, [])
            if not isinstance(existing, list):
                existing = []

            existing.append(record)
            _save_json_best_effort(store, user_file, existing)

            return f"✅ Eye input saved. File: `{_debug_store_path(store, user_file)}`"

        except Exception as e:
            print(f"[eye] ❌ Save failed: {e}")
            return f"❌ Save failed: {e}"

    # -------------------------
    # Generate AI output
    # -------------------------
    def generate_ai(
        strain_val, session_val, symp_val,
        light_val, bright_val, glare_val, dist_val,
        corr_val, rule_val, drops_val,
        notes_val
    ):
        print("[eye] 🔘 Generate AI clicked")

        data = collect_user_input(
            strain_val, session_val, symp_val,
            light_val, bright_val, glare_val, dist_val,
            corr_val, rule_val, drops_val,
            notes_val
        )

        prompt = (
            "You are a preventive eye health assistant for desk workers.\n"
            "Provide non-diagnostic, practical guidance.\n"
            "1. Identify risk factors for Computer Vision Syndrome.\n"
            "2. If strain_level > 5, suggest immediate relief techniques.\n"
            "3. Give advice about dryness (especially with contact lenses).\n"
            "4. Provide a short prevention plan for tomorrow.\n\n"
            f"EYE DATA:\n{json.dumps(data, indent=2, ensure_ascii=False)}"
        )

        current_lang = lang_state.value if hasattr(lang_state, "value") else lang_state

        try:
            resp = gemini.generate(prompt=prompt, response_language=current_lang)
            text = resp.text.strip() if hasattr(resp, "text") and resp.text else str(resp).strip()
        except Exception as e:
            print(f"[eye] ❌ AI generation failed: {e}")
            return "", f"⚠️ AI generation failed: {e}"

        if not text:
            return "", "⚠️ AI returned empty output."

        return text, "✅ AI output generated (not saved yet)."

    # -------------------------
    # Save AI output (APPEND)
    # -------------------------
    def save_ai_output(text: str):
        print("[eye] 🔘 Save AI Output clicked")

        if not (text or "").strip():
            return "⚠️ Nothing to save."

        try:
            record = {"timestamp": _now(), "ai_output": text.strip()}

            existing = store.load_json(ai_file, [])
            if not isinstance(existing, list):
                existing = []

            existing.append(record)
            _save_json_best_effort(store, ai_file, existing)

            # Update domain summary
            try:
                updater = globals().get("update_domain_summary")
                if updater:
                    updater(store, domain, text[:200])
            except Exception as e:
                print(f"[eye] ⚠️ update_domain_summary failed: {e}")

            return f"💾 Eye AI output saved. File: `{_debug_store_path(store, ai_file)}`"

        except Exception as e:
            print(f"[eye] ❌ Save failed: {e}")
            return f"❌ Save failed: {e}"

    # -------------------------
    # Buttons UI
    # -------------------------
    with gr.Row():
        save_input_btn = gr.Button("💾 Save User Input", variant="secondary")
        gen_btn = gr.Button("🤖 Generate AI Eye Analysis", variant="primary")
        save_ai_btn = gr.Button("💾 Save AI Output", variant="secondary")

    inputs_list = [
        ui_inputs["strain_level"], ui_inputs["session_length"], ui_inputs["symptoms"],
        ui_inputs["lighting"], ui_inputs["screen_brightness"], ui_inputs["glare"], ui_inputs["distance_check"],
        ui_inputs["correction"], ui_inputs["rule_20_20_20"], ui_inputs["drops"],
        ui_inputs["notes_input"]
    ]

    save_input_btn.click(fn=save_user_input, inputs=inputs_list, outputs=[status])
    gen_btn.click(fn=generate_ai, inputs=inputs_list, outputs=[output, status])
    save_ai_btn.click(fn=save_ai_output, inputs=[output], outputs=[status])

    # =========================================================
    # Demo Hook Registration (Eye) — MUST BE INSIDE FUNCTION
    # =========================================================
    try:
        def _eye_demo_generate():
            print("[eye-demo] 🚀 Demo generate using saved eye data")

            raw = store.load_json(user_file, [])
            if not isinstance(raw, list) or not raw:
                return "⚠️ No saved eye demo input found.", "No data"

            last = raw[-1].get("user_input", {})
            if not isinstance(last, dict) or not last:
                return "⚠️ Saved eye record is empty.", "No data"

            prompt = (
                "You are a preventive workplace eye strain assistant.\n"
                "Analyze the following eye health and screen exposure data.\n"
                "Provide actionable, non-diagnostic advice to reduce eye strain.\n"
                "Include lighting, screen distance, breaks (20-20-20 rule), and hydration links.\n"
                "Mention red-flag symptoms requiring eye doctor review.\n\n"
                f"EYE DATA:\n{json.dumps(last, indent=2, ensure_ascii=False)}"
            )

            current_lang = lang_state.value if hasattr(lang_state, "value") else lang_state

            resp = gemini.generate(prompt, response_language=current_lang)
            text = resp.text.strip() if hasattr(resp, "text") else str(resp).strip()

            return text, "✅ Eye demo output generated."

        register_demo_hook(
            domain=domain,
            label="Eye Module",
            generate_fn=_eye_demo_generate
        )

    except Exception as e:
        print(f"[eye] ⚠️ Demo hook registration failed: {e}")


## Final Assembly — Eye Tab Builder

This wrapper function is the entry point for the Eye Health module.

It:
- Loads the latest stored history
- Builds the Eye input UI (symptoms + habits + environment)
- Builds the output UI (recommendations + status)
- Connects Gemini reasoning and saving logic

This makes the Eye module fully modular and easy to maintain.


In [44]:
# ==========================================
# Eye Module — Final Tab Builder (FIXED)
# ==========================================

def build_eye_tab(
    store,
    gemini,
    lang_state: gr.State,
    locales_dir: Optional[Path] = None,
):

    local_domain = "eye"
    local_user_file = f"{local_domain}_user_input.json"
    local_ai_file = f"{local_domain}_ai_output.json"

    print(f"[eye] ✅ build_eye_tab executed. user_file={local_user_file}, ai_file={local_ai_file}")

    lang = lang_state.value if hasattr(lang_state, "value") else lang_state

    saved_user_input, saved_ai_text = load_domain_history(store, local_domain)

    # --- Title ---
    gr.Markdown(
        _safe_t(
            lang,
            "eye.title",
            locales_dir,
            default="👁️ Eye Health & Computer Vision Tracker",
        )
    )

    # --- Info Panel Button (Eye Tab) ---
    add_info_panel(
        title="📘 Eye Health Tab Guide",
        content_md=EYE_HEALTH_INFO_MD,
        button_label="ℹ️ Eye Health Info",
        open_label="❌ Hide Eye Health Info",
    )

    # --- Inputs ---
    ui_inputs = build_eye_inputs(
        saved_user_input,
        lang,
        locales_dir,
    )

    # --- Outputs ---
    output, status = build_eye_outputs(
        saved_ai_text,
        lang,
        locales_dir,
    )

    # --- Logic wiring ---
    connect_eye_logic(
        store=store,
        gemini=gemini,
        lang_state=lang_state,
        locales_dir=locales_dir,
        ui_inputs=ui_inputs,
        output=output,
        status=status,
    )


Mental Tab
The Mental health tab allows the user to record their mood or feelings (possibly via a slider or text input for journaling). The AI can analyze the input to provide supportive suggestions or coping strategies. If any concerning keywords are detected (e.g., indications of self-harm), the safety check will flag an urgent message to encourage seeking help. The mental health summary is updated in the context memory.

# 🧠 Mental Wellbeing Module  
*Cognitive Load & Burnout Awareness*

Desk-based work places sustained demands on attention, emotional regulation, and
mental energy. Over time, high cognitive load, constant interruptions, and limited
recovery can contribute to stress accumulation, reduced focus, and early burnout
signals—often before individuals consciously recognize them.

This module is designed to support **mental wellbeing awareness**, not diagnosis.
It helps identify daily patterns related to stress, cognitive fatigue, and emotional
load in the context of work routines.

### What this module captures
- Perceived stress level
- Mood state
- Mental energy and brain fog
- Focus quality and workload perception
- Distractions (internal vs external)
- Burnout warning signs (detachment, overwhelm)
- Sleep quality and coping mechanisms

### Why this matters
Mental fatigue and burnout rarely appear suddenly. They typically develop through
small, repeated stressors combined with insufficient recovery. Desk work can
intensify this process due to prolonged screen exposure, cognitive overload, and
blurred boundaries between work and rest.

By reflecting on daily mental signals, this module helps highlight trends that may
affect productivity, wellbeing, and long-term resilience—before more severe
consequences occur.

### How AI is used in this section
The AI provides **supportive, non-diagnostic guidance** focused on:
- Simple grounding and reset techniques
- Burnout prevention through small, realistic actions
- Improving coping strategies during demanding workdays
- Practical sleep hygiene recommendations

The aim is to promote awareness, balance, and early prevention—not to replace
professional care.


In [45]:
# ==========================================
# Mental Module — Config + History Loader
# ==========================================




def load_mental_history(store):
    """
    Loads the most recent saved user input + AI output for the Mental module.
    Uses the unified load_domain_history() helper.
    """
    local_domain = "mental"
    return load_domain_history(store, local_domain)


## Step 1 — User Inputs (Mental Vitals + Burnout Risk)

This section builds the Mental Wellbeing UI.

The input fields are structured into 3 major blocks:
1. Emotional Vitals (stress, mood, energy)
2. Cognitive Load (focus quality, workload, distractions)
3. Burnout & Recovery (detachment, overwhelm, sleep quality, coping methods)

A free-text journal is included for personal reflections.
This free-text is also checked by safety filters when available.


In [46]:
# ==========================================
# Mental Module — Inputs UI Builder
# ==========================================

import gradio as gr


def build_mental_inputs(saved_user_input: dict, lang: str, locales_dir: Optional[Path]):

    gr.Markdown("### Emotional Vitals")

    with gr.Row():
        stress = gr.Slider(
            minimum=1, maximum=10, step=1,
            value=_to_int(saved_user_input.get("stress", 5), 5),
            label=_safe_t(lang, "mental.stress", locales_dir, default="Stress Level (1=Zen, 10=Panic)")
        )

        mood_choices = ["Calm", "Anxious", "Irritable", "Motivated", "Numb/Detached", "Sad"]
        saved_mood = _to_str(saved_user_input.get("mood", "Calm"), "Calm")
        if saved_mood not in mood_choices:
            saved_mood = "Calm"

        mood = gr.Dropdown(
            choices=mood_choices,
            value=saved_mood,
            label="Current Mood"
        )

        energy_choices = ["Sharp", "Average", "Brain Fog/Sluggish"]
        saved_energy = _to_str(saved_user_input.get("energy", "Average"), "Average")
        if saved_energy not in energy_choices:
            saved_energy = "Average"

        energy = gr.Radio(
            choices=energy_choices,
            value=saved_energy,
            label="Mental Energy"
        )

    with gr.Accordion("Cognitive Function & Workload", open=False):

        focus_choices = ["Deep Work / Flow", "Fragmented / Multitasking", "Unable to Concentrate"]
        saved_focus = _to_str(
            saved_user_input.get("focus_quality", "Fragmented / Multitasking"),
            "Fragmented / Multitasking"
        )
        if saved_focus not in focus_choices:
            saved_focus = "Fragmented / Multitasking"

        workload_choices = ["Light", "Manageable", "Heavy", "Impossible"]
        saved_workload = _to_str(saved_user_input.get("workload", "Manageable"), "Manageable")
        if saved_workload not in workload_choices:
            saved_workload = "Manageable"

        with gr.Row():
            focus_quality = gr.Dropdown(
                choices=focus_choices,
                value=saved_focus,
                label="Quality of Focus"
            )
            workload = gr.Radio(
                choices=workload_choices,
                value=saved_workload,
                label="Perceived Workload"
            )

        distractions = gr.CheckboxGroup(
            choices=["External (Noise/People)", "Digital (Notifications)", "Internal (Worry/Thoughts)"],
            value=_to_list(saved_user_input.get("distractions", [])),
            label="Main Distractions"
        )

    with gr.Accordion("Burnout Risk & Recovery", open=False):

        social_choices = ["Isolated", "Text/Chat only", "Meaningful Interaction"]
        saved_social = _to_str(saved_user_input.get("social", "Text/Chat only"), "Text/Chat only")
        if saved_social not in social_choices:
            saved_social = "Text/Chat only"

        sleepq_choices = ["Restorative", "Trouble Falling Asleep", "Woke up Tired", "Insomnia"]
        saved_sleepq = _to_str(saved_user_input.get("sleep_quality", "Restorative"), "Restorative")
        if saved_sleepq not in sleepq_choices:
            saved_sleepq = "Restorative"

        with gr.Row():
            detachment = gr.Checkbox(
                value=_to_bool(saved_user_input.get("detachment", False)),
                label="Feeling cynical/detached?"
            )
            overwhelm = gr.Checkbox(
                value=_to_bool(saved_user_input.get("overwhelm", False)),
                label="Feeling unable to keep up?"
            )
            social = gr.Dropdown(
                choices=social_choices,
                value=saved_social,
                label="Social Connection"
            )

        with gr.Row():
            sleep_quality = gr.Dropdown(
                choices=sleepq_choices,
                value=saved_sleepq,
                label="Sleep Quality (Last Night)"
            )

            coping = gr.CheckboxGroup(
                choices=["Breathing/Meditation", "Venting to Friend", "Caffeine/Sugar", "Procrastination", "Exercise"],
                value=_to_list(saved_user_input.get("coping", [])),
                label="Coping Mechanisms Used"
            )

    notes = gr.Textbox(
        value=_to_str(saved_user_input.get("notes", ""), ""),
        label=_safe_t(lang, "mental.notes", locales_dir, default="Journal / Thoughts"),
        lines=3,
        placeholder="What's on your mind?"
    )

    return {
        "stress": stress,
        "mood": mood,
        "energy": energy,
        "focus_quality": focus_quality,
        "workload": workload,
        "distractions": distractions,
        "detachment": detachment,
        "overwhelm": overwhelm,
        "social": social,
        "sleep_quality": sleep_quality,
        "coping": coping,
        "notes": notes,
    }


## Step 2 — Output Components

This section defines:
- Status display messages (save success, errors, warnings)
- A read-only AI output field that shows the generated guidance

The most recent saved AI output is loaded automatically.


In [47]:
# ==========================================
# Mental Module — Output UI Builder
# ==========================================

def build_mental_outputs(saved_ai_text: str, lang: str, locales_dir: Optional[Path]):

    status = gr.Markdown("")

    output = gr.Textbox(
        label=_safe_t(lang, "mental.output", locales_dir, default="Mental Health Guidance"),
        interactive=False,
        lines=8,
        value=saved_ai_text
    )

    return output, status


## Step 3 — AI Guidance + Safety Checks + Saving

This module is slightly more advanced than other tabs because it supports safety filtering.

Workflow:
1. Collect structured input into JSON
2. Save user input history (timestamped)
3. Run safety checks on the journal text (if available)
4. Generate supportive non-diagnostic guidance using Gemini
5. Save AI output history

The AI is instructed to focus on:
- burnout prevention
- coping mechanism improvement
- grounding techniques
- sleep hygiene advice


In [48]:
# ==========================================
# Mental Module — AI + Saving + Wiring
# ==========================================

def connect_mental_logic(
    store,
    gemini,
    lang_state: gr.State,
    locales_dir: Optional[Path],
    ui_inputs: dict,
    output: gr.Textbox,
    status: gr.Markdown,
):

    # IMPORTANT: local names (prevents cross-tab overwrite bugs)
    local_domain = "mental"
    local_user_file = f"{local_domain}_user_input.json"
    local_ai_file = f"{local_domain}_ai_output.json"

    # -------------------------
    # Collect user input helper
    # -------------------------
    def collect_user_input(
        stress_val, mood_val, energy_val,
        focus_val, work_val, dist_val,
        detach_val, over_val, social_val,
        sleep_val, coping_val, notes_val
    ):
        return {
            "stress": _to_int(stress_val, 5),
            "mood": _to_str(mood_val, "Calm"),
            "energy": _to_str(energy_val, "Average"),
            "focus_quality": _to_str(focus_val, "Fragmented / Multitasking"),
            "workload": _to_str(work_val, "Manageable"),
            "distractions": _to_list(dist_val),
            "detachment": _to_bool(detach_val),
            "overwhelm": _to_bool(over_val),
            "social": _to_str(social_val, "Text/Chat only"),
            "sleep_quality": _to_str(sleep_val, "Restorative"),
            "coping": _to_list(coping_val),
            "notes": _to_str(notes_val, ""),
        }

    # -------------------------
    # Save user input (APPEND)
    # -------------------------
    def save_user_input(
        stress_val, mood_val, energy_val,
        focus_val, work_val, dist_val,
        detach_val, over_val, social_val,
        sleep_val, coping_val, notes_val
    ):
        print("[mental] 🔘 Save User Input clicked")

        try:
            user_input = collect_user_input(
                stress_val, mood_val, energy_val,
                focus_val, work_val, dist_val,
                detach_val, over_val, social_val,
                sleep_val, coping_val, notes_val
            )

            record = {"timestamp": _now(), "user_input": user_input}

            existing = store.load_json(local_user_file, [])
            if not isinstance(existing, list):
                existing = []

            existing.append(record)
            _save_json_best_effort(store, local_user_file, existing)

            return f"✅ Mental input saved. File: `{_debug_store_path(store, local_user_file)}`"

        except Exception as e:
            print(f"[mental] ❌ Save failed: {e}")
            return f"❌ Save failed: {e}"

    # -------------------------
    # Generate AI output
    # -------------------------
    def generate_ai(
        stress_val, mood_val, energy_val,
        focus_val, work_val, dist_val,
        detach_val, over_val, social_val,
        sleep_val, coping_val, notes_val
    ):
        print("[mental] 🔘 Generate AI clicked")

        entry = collect_user_input(
            stress_val, mood_val, energy_val,
            focus_val, work_val, dist_val,
            detach_val, over_val, social_val,
            sleep_val, coping_val, notes_val
        )

        # safety flags (optional)
        flags = []
        if "apply_safety_checks" in globals() and callable(globals().get("apply_safety_checks")):
            payload = {"free_text": str(notes_val or "")}
            try:
                result = globals()["apply_safety_checks"](payload)
                flags = result.get("safety_flags", []) if isinstance(result, dict) else []
            except Exception as e:
                print(f"[mental] ⚠️ Safety check error: {e}")

        prompt = (
            "You are a supportive preventive mental wellbeing assistant.\n"
            "Provide non-diagnostic guidance.\n"
            "1. Analyze coping mechanisms (healthy vs avoidance).\n"
            "2. If detachment/overwhelm is present, provide grounding techniques.\n"
            "3. Link sleep quality to energy and give one specific sleep hygiene tip.\n"
            "4. Provide burnout prevention micro-actions for desk workers.\n\n"
            f"MENTAL LOG:\n{json.dumps(entry, indent=2, ensure_ascii=False)}"
        )

        current_lang = lang_state.value if hasattr(lang_state, "value") else lang_state

        try:
            resp = gemini.generate(prompt=prompt, response_language=current_lang)
            text = resp.text.strip() if hasattr(resp, "text") and resp.text else str(resp).strip()
        except Exception as e:
            print(f"[mental] ❌ AI generation failed: {e}")
            return "", f"⚠️ AI generation failed: {e}"

        if not text:
            return "", "⚠️ AI returned empty output."

        # prepend safety warnings if needed
        if flags:
            warnings = "\n".join(
                f"⚠️ {flag.get('message','')}"
                for flag in flags
                if isinstance(flag, dict)
            ).strip()

            if warnings:
                text = f"{warnings}\n\n{text}"

        return text, "✅ AI output generated (not saved yet)."

    # -------------------------
    # Save AI output (APPEND)
    # -------------------------
    def save_ai_output(text: str):
        print("[mental] 🔘 Save AI Output clicked")

        if not (text or "").strip():
            return "⚠️ Nothing to save."

        try:
            record = {"timestamp": _now(), "ai_output": text.strip()}

            existing = store.load_json(local_ai_file, [])
            if not isinstance(existing, list):
                existing = []

            existing.append(record)
            _save_json_best_effort(store, local_ai_file, existing)

            # update domain summary
            try:
                updater = globals().get("update_domain_summary")
                if updater:
                    updater(store, local_domain, text[:200])
            except Exception as e:
                print(f"[mental] ⚠️ update_domain_summary failed: {e}")

            return f"💾 Mental AI output saved. File: `{_debug_store_path(store, local_ai_file)}`"

        except Exception as e:
            print(f"[mental] ❌ Save failed: {e}")
            return f"❌ Save failed: {e}"

    # -------------------------
    # Buttons UI
    # -------------------------
    with gr.Row():
        save_input_btn = gr.Button("💾 Save User Input", variant="secondary")
        gen_btn = gr.Button("🤖 Generate AI Guidance", variant="primary")
        save_ai_btn = gr.Button("💾 Save AI Output", variant="secondary")

    inputs_list = [
        ui_inputs["stress"], ui_inputs["mood"], ui_inputs["energy"],
        ui_inputs["focus_quality"], ui_inputs["workload"], ui_inputs["distractions"],
        ui_inputs["detachment"], ui_inputs["overwhelm"], ui_inputs["social"],
        ui_inputs["sleep_quality"], ui_inputs["coping"], ui_inputs["notes"],
    ]

    save_input_btn.click(fn=save_user_input, inputs=inputs_list, outputs=[status])
    gen_btn.click(fn=generate_ai, inputs=inputs_list, outputs=[output, status])
    save_ai_btn.click(fn=save_ai_output, inputs=[output], outputs=[status])

    # -------------------------
    # Demo Hook Registration (Mental)
    # -------------------------
    try:
        def _mental_demo_generate():
            print("[mental-demo] 🚀 Demo generate using saved mental data")

            raw = store.load_json(local_user_file, [])
            if not isinstance(raw, list) or not raw:
                return "⚠️ No saved mental demo input found.", "No data"

            last = raw[-1].get("user_input", {})
            if not isinstance(last, dict) or not last:
                return "⚠️ Saved mental record is empty.", "No data"

            prompt = (
                "You are a preventive workplace mental wellbeing assistant.\n"
                "Analyze the following stress/mood/workload data.\n"
                "Provide supportive, non-diagnostic recommendations.\n"
                "Focus on stress management, burnout prevention, work boundaries, and recovery habits.\n"
                "Include red-flag warning signs requiring professional help.\n\n"
                f"MENTAL WELLBEING DATA:\n{last}"
            )

            current_lang = lang_state.value if hasattr(lang_state, "value") else lang_state

            resp = gemini.generate(prompt, response_language=current_lang)
            text = resp.text.strip() if hasattr(resp, "text") else str(resp).strip()

            return text, "✅ Mental demo output generated."

        register_demo_hook(
            domain=local_domain,
            label="Mental Wellbeing Module",
            generate_fn=_mental_demo_generate
        )

    except Exception as e:
        print(f"[mental] ⚠️ Demo hook registration failed: {e}")


## Final Assembly — Mental Tab Builder

This wrapper function is the entry point for the Mental Wellbeing module.

It:
- loads the latest stored mental wellbeing entry
- builds the full input UI
- builds output UI (AI guidance + status)
- connects Gemini generation + saving logic

This design makes the module easy to maintain and easy to read for hackathon judges.


In [49]:
# ==========================================
# Mental Wellbeing Module — Final Tab Builder (FIXED)
# ==========================================

def build_mental_tab(
    store,
    gemini,
    lang_state: gr.State,
    locales_dir: Optional[Path] = None,
):

    local_domain = "mental"
    local_user_file = f"{local_domain}_user_input.json"
    local_ai_file = f"{local_domain}_ai_output.json"

    print(f"[mental] ✅ build_mental_tab executed. user_file={local_user_file}, ai_file={local_ai_file}")

    lang = lang_state.value if hasattr(lang_state, "value") else lang_state

    saved_user_input, saved_ai_text = load_domain_history(store, local_domain)

    # --- Title ---
    gr.Markdown(
        _safe_t(
            lang,
            "mental.title",
            locales_dir,
            default="🧠 Mental Well-Being & Cognitive Load",
        )
    )

    # --- Info Panel Button (Mental Tab) ---
    add_info_panel(
        title="📘 Mental Wellbeing Tab Guide",
        content_md=MENTAL_WELLBEING_INFO_MD,
        button_label="ℹ️ Mental Wellbeing Info",
        open_label="❌ Hide Mental Info",
    )

    # --- Inputs ---
    ui_inputs = build_mental_inputs(
        saved_user_input,
        lang,
        locales_dir,
    )

    # --- Outputs ---
    output, status = build_mental_outputs(
        saved_ai_text,
        lang,
        locales_dir,
    )

    # --- Logic wiring ---
    connect_mental_logic(
        store=store,
        gemini=gemini,
        lang_state=lang_state,
        locales_dir=locales_dir,
        ui_inputs=ui_inputs,
        output=output,
        status=status,
    )


Hydration Tab
The Hydration tab is for tracking water intake or hydration habits. The user might input how much water they've drunk or rate their hydration. The tab then provides feedback or tips (like drinking more water at certain times). The hydration status is summarized and saved to context, contributing to the overall recommendations.

In [50]:
from IPython.display import Markdown, display

display(Markdown(HYDRATION_INFO_MD))



# 💧 Hydration & Kidney Health Module

Hydration plays a critical role in concentration, energy levels, kidney function,
and overall physical wellbeing—especially for people who spend long hours at a desk.
Mild dehydration is common in office settings and often goes unnoticed until symptoms
such as headache, fatigue, or brain fog appear.

This module is designed to move beyond simple “water tracking” by combining
**daily hydration behaviors** with **physiological bio-feedback signals**. Together,
these provide a more realistic picture of hydration status than intake alone.

### What this module captures
- Daily water intake (number of glasses)
- Caffeine consumption (coffee, tea, energy drinks)
- Sugary drink intake
- Hydration habits (e.g., keeping a water bottle visible on the desk)
- Urine color indicator
- Thirst sensation
- Common dehydration symptoms (headache, brain fog, dizziness)

### Why this matters
Relying on thirst alone can be misleading, as thirst is often a **late signal**
of dehydration. Caffeine use, long screen time, and busy work routines can further
mask early warning signs.

By combining habits and body signals, this module helps identify patterns that may
affect productivity, comfort, and kidney health—without requiring perfect tracking
or medical interpretation.

### How AI is used in this section
The AI generates **non-diagnostic, preventive guidance** focused on:
- Practical hydration targets for desk-based work
- Habit nudges that fit real workdays
- Early warning signals of possible dehydration
- Short, actionable checklists for same-day improvement

The goal is awareness and prevention—not diagnosis.


In [51]:
# ==========================================
# Hydration Module — Config + History Loader
# ==========================================




def load_hydration_history(store):
    """
    Loads the most recent saved user input + AI output for the Hydration module.
    Uses the unified load_domain_history() helper.
    """
    local_domain = "hydration"
    return load_domain_history(store, local_domain)


## Step 1 — Hydration Inputs

This UI is structured into 2 blocks:

### 1) Daily Intake
Captures water, caffeine, sugary drinks, and desk habit signals.

### 2) Hydration Signals (Bio-feedback)
Captures urine color, thirst sensation, and dehydration symptoms.

These signals are more reliable than "guessing hydration" because they reflect real physiology.


In [52]:
# ==========================================
# Hydration Module — Inputs UI Builder
# ==========================================

import gradio as gr


def build_hydration_inputs(saved_user_input: dict, lang: str, locales_dir: Optional[Path]):

    gr.Markdown("### Daily Intake")

    with gr.Row():
        water_intake = gr.Slider(
            minimum=0, maximum=15, step=1,
            value=_to_int(saved_user_input.get("water_intake", 0), 0),
            label=_safe_t(lang, "hydration.water", locales_dir, default="Water Glasses (250ml)")
        )

        caffeine_intake = gr.Number(
            value=_to_float(saved_user_input.get("caffeine_intake", 0), 0.0),
            label=_safe_t(lang, "hydration.caffeine", locales_dir, default="Cups of Coffee/Tea")
        )

    with gr.Row():
        bottle_on_desk = gr.Checkbox(
            value=_to_bool(saved_user_input.get("bottle_on_desk", False)),
            label="Water bottle is visible on desk?"
        )

        sugary_drinks = gr.Number(
            value=_to_float(saved_user_input.get("sugary_drinks", 0), 0.0),
            label="Sugary Drinks (Soda/Juice)"
        )

    with gr.Accordion("Hydration Signals (Bio-Feedback)", open=True):
        gr.Markdown("*Your body is the best sensor. Check these signals:*")

        urine_choices = ["Clear / Pale Yellow (Good)", "Yellow (Okay)", "Dark Yellow / Amber (Dehydrated)"]
        saved_urine = saved_user_input.get("urine_color", "Yellow (Okay)")
        if saved_urine not in urine_choices:
            saved_urine = "Yellow (Okay)"

        with gr.Row():
            urine_color = gr.Dropdown(
                choices=urine_choices,
                value=saved_urine,
                label=_safe_t(lang, "hydration.urine", locales_dir, default="Urine Color Indicator")
            )

            thirst_level = gr.Radio(
                choices=["Not Thirsty", "Mildly Thirsty", "Very Thirsty / Parched"],
                value=str(saved_user_input.get("thirst_level", "Not Thirsty") or "Not Thirsty"),
                label="Thirst Sensation"
            )

        symptoms = gr.CheckboxGroup(
            choices=["Dry Mouth/Lips", "Headache", "Dizziness", "Brain Fog"],
            value=_to_list(saved_user_input.get("symptoms", [])),
            label="Dehydration Symptoms"
        )

    notes_input = gr.Textbox(
        value=str(saved_user_input.get("notes", "") or ""),
        label=_safe_t(lang, "hydration.notes", locales_dir, default="Notes"),
        lines=1
    )

    return {
        "water_intake": water_intake,
        "caffeine_intake": caffeine_intake,
        "bottle_on_desk": bottle_on_desk,
        "sugary_drinks": sugary_drinks,
        "urine_color": urine_color,
        "thirst_level": thirst_level,
        "symptoms": symptoms,
        "notes_input": notes_input,
    }


## Step 2 — Output Components

The output section contains:

- **Status panel**: used to display save confirmations and error messages.
- **AI output box**: a read-only field that displays Gemini-generated hydration analysis.

The latest saved AI output is loaded automatically at startup.


In [53]:
# ==========================================
# Hydration Module — Output UI Builder
# ==========================================

def build_hydration_outputs(saved_ai_text: str, lang: str, locales_dir: Optional[Path]):

    status = gr.Markdown("")

    output = gr.Textbox(
        label=_safe_t(lang, "hydration.output", locales_dir, default="Hydration Analysis"),
        interactive=False,
        value=saved_ai_text,
        lines=10
    )

    return output, status


## Step 3 — AI Analysis + Saving Logic

This section implements the logic behind the three buttons:

### Button 1: Save User Input
Appends user input to a timestamped history file.

### Button 2: Generate AI Analysis
Uses Gemini to interpret hydration risks and suggest practical actions.

### Button 3: Save AI Output
Saves the generated analysis into an AI history file.

This design supports longitudinal tracking and allows future visualization modules.


In [54]:
# ==========================================
# Hydration Module — AI + Saving + Wiring
# ==========================================

def connect_hydration_logic(
    store,
    gemini,
    lang_state: gr.State,
    locales_dir: Optional[Path],
    ui_inputs: dict,
    output: gr.Textbox,
    status: gr.Markdown
):

    # -------------------------
    # LOCAL DOMAIN CONSTANTS
    # (prevents naming conflicts)
    # -------------------------
    local_domain = "hydration"
    local_user_file = "hydration_user_input.json"
    local_ai_file = "hydration_ai_output.json"

    # -------------------------
    # Collect JSON-safe input
    # -------------------------
    def collect_user_input(
        water_val, caffeine_val, bottle_val, sugar_val,
        urine_val, thirst_val, symp_val, notes_val
    ):
        return {
            "water_intake": _to_int(water_val, 0),
            "caffeine_intake": _to_float(caffeine_val, 0.0),
            "bottle_on_desk": _to_bool(bottle_val),
            "sugary_drinks": _to_float(sugar_val, 0.0),
            "urine_color": str(urine_val or ""),
            "thirst_level": str(thirst_val or ""),
            "symptoms": _to_list(symp_val),
            "notes": str(notes_val or ""),
        }

    # -------------------------
    # Save user input (APPEND)
    # -------------------------
    def save_user_input(
        water_val, caffeine_val, bottle_val, sugar_val,
        urine_val, thirst_val, symp_val, notes_val
    ):
        print("[hydration] 🔘 Save User Input clicked")

        try:
            user_input = collect_user_input(
                water_val, caffeine_val, bottle_val, sugar_val,
                urine_val, thirst_val, symp_val, notes_val
            )

            record = {"timestamp": _now(), "user_input": user_input}

            existing = store.load_json(local_user_file, [])
            if not isinstance(existing, list):
                existing = []

            existing.append(record)
            _save_json_best_effort(store, local_user_file, existing)

            return f"✅ Saved hydration input. File: `{_debug_store_path(store, local_user_file)}`"

        except Exception as e:
            print(f"[hydration] ❌ Save failed: {e}")
            return f"❌ Save failed: {e}"

    # -------------------------
    # Generate AI output
    # -------------------------
    def generate_ai(
        water_val, caffeine_val, bottle_val, sugar_val,
        urine_val, thirst_val, symp_val, notes_val
    ):
        print("[hydration] 🔘 Generate AI clicked")

        user_input = collect_user_input(
            water_val, caffeine_val, bottle_val, sugar_val,
            urine_val, thirst_val, symp_val, notes_val
        )

        prompt = (
            "You are a preventive hydration and kidney health assistant.\n"
            "Analyze the hydration data and provide non-diagnostic guidance.\n"
            "1) Consider caffeine as a mild diuretic.\n"
            "2) If urine is dark or headache exists, issue a clear hydration alert.\n"
            "3) If bottle_on_desk is false, explain nudge theory benefit.\n"
            "4) Give practical hydration targets and habits for desk workers.\n"
            "5) Provide a short checklist of next actions.\n\n"
            f"HYDRATION DATA:\n{user_input}"
        )

        current_lang = lang_state.value if hasattr(lang_state, "value") else lang_state

        try:
            resp = gemini.generate(prompt, response_language=current_lang)
            text = resp.text.strip() if hasattr(resp, "text") and resp.text else str(resp).strip()
        except Exception as e:
            print(f"[hydration] ❌ AI generation failed: {e}")
            return "", f"⚠️ AI generation failed: {e}"

        if not text:
            return "", "⚠️ AI returned empty output."

        return text, "✅ AI output generated (not saved yet)."

    # -------------------------
    # Save AI output (APPEND)
    # -------------------------
    def save_ai_output(text: str):
        print("[hydration] 🔘 Save AI Output clicked")

        if not (text or "").strip():
            return "⚠️ Nothing to save."

        try:
            record = {"timestamp": _now(), "ai_output": text.strip()}

            existing = store.load_json(local_ai_file, [])
            if not isinstance(existing, list):
                existing = []

            existing.append(record)
            _save_json_best_effort(store, local_ai_file, existing)

            try:
                updater = globals().get("update_domain_summary")
                if updater:
                    updater(store, local_domain, text[:200])
            except Exception as e:
                print(f"[hydration] ⚠️ update_domain_summary failed: {e}")

            return f"💾 Saved AI output. File: `{_debug_store_path(store, local_ai_file)}`"

        except Exception as e:
            print(f"[hydration] ❌ Save failed: {e}")
            return f"❌ Save failed: {e}"

    # -------------------------
    # Buttons UI
    # -------------------------
    with gr.Row():
        save_input_btn = gr.Button("💾 Save User Input", variant="secondary")
        gen_btn = gr.Button("🤖 Generate AI Hydration Analysis", variant="primary")
        save_ai_btn = gr.Button("💾 Save AI Output", variant="secondary")

    inputs_list = [
        ui_inputs["water_intake"],
        ui_inputs["caffeine_intake"],
        ui_inputs["bottle_on_desk"],
        ui_inputs["sugary_drinks"],
        ui_inputs["urine_color"],
        ui_inputs["thirst_level"],
        ui_inputs["symptoms"],
        ui_inputs["notes_input"],
    ]

    save_input_btn.click(fn=save_user_input, inputs=inputs_list, outputs=[status])
    gen_btn.click(fn=generate_ai, inputs=inputs_list, outputs=[output, status])
    save_ai_btn.click(fn=save_ai_output, inputs=[output], outputs=[status])

    # -------------------------
    # Demo Hook Registration (Hydration)
    # -------------------------
    try:
        def _hydration_demo_generate():
            print("[hydration-demo] 🚀 Demo generate using saved hydration data")

            raw = store.load_json(local_user_file, [])
            if not isinstance(raw, list) or not raw:
                return "⚠️ No saved hydration demo input found.", "No data"

            last = raw[-1].get("user_input", {})
            if not isinstance(last, dict) or not last:
                return "⚠️ Saved hydration record is empty.", "No data"

            prompt = (
                "You are a preventive workplace hydration assistant.\n"
                "Analyze the following hydration and caffeine data.\n"
                "Provide actionable, non-diagnostic hydration recommendations.\n"
                "Discuss headaches, fatigue, urine color, caffeine balance, and daily water habits.\n\n"
                f"HYDRATION DATA:\n{last}"
            )

            current_lang = lang_state.value if hasattr(lang_state, "value") else lang_state

            resp = gemini.generate(prompt, response_language=current_lang)
            text = resp.text.strip() if hasattr(resp, "text") else str(resp).strip()

            return text, "✅ Hydration demo output generated."

        register_demo_hook(
            domain="hydration",
            label="Hydration Module",
            generate_fn=_hydration_demo_generate
        )

    except Exception as e:
        print(f"[hydration] ⚠️ Demo hook registration failed: {e}")


## Final Assembly — Hydration Tab Builder

This wrapper function assembles the full Hydration module:

1. Loads the latest stored history
2. Builds the UI input fields
3. Builds the output fields
4. Connects Gemini + saving logic

This design keeps the notebook clean and makes the project easy to judge and maintain.


In [55]:
# ==========================================
# Hydration Module — Final Tab Builder (FIXED)
# ==========================================

def build_hydration_tab(
    store,
    gemini,
    lang_state: gr.State,
    locales_dir: Optional[Path] = None,
):

    local_domain = "hydration"
    local_user_file = f"{local_domain}_user_input.json"
    local_ai_file = f"{local_domain}_ai_output.json"

    print(f"[hydration] ✅ build_hydration_tab executed. user_file={local_user_file}, ai_file={local_ai_file}")

    lang = lang_state.value if hasattr(lang_state, "value") else lang_state

    saved_user_input, saved_ai_text = load_domain_history(store, local_domain)

    # --- Title ---
    gr.Markdown(
        _safe_t(
            lang,
            "hydration.title",
            locales_dir,
            default="💧 Hydration & Kidney Health",
        )
    )

    # --- Info Panel Button (Hydration Tab) ---
    add_info_panel(
        title="📘 Hydration Tab Guide",
        content_md=HYDRATION_INFO_MD,
        button_label="ℹ️ Hydration Info",
        open_label="❌ Hide Hydration Info",
    )

    # --- Inputs ---
    ui_inputs = build_hydration_inputs(
        saved_user_input,
        lang,
        locales_dir,
    )

    # --- Outputs ---
    output, status = build_hydration_outputs(
        saved_ai_text,
        lang,
        locales_dir,
    )

    # --- Logic wiring ---
    connect_hydration_logic(
        store=store,
        gemini=gemini,
        lang_state=lang_state,
        locales_dir=locales_dir,
        ui_inputs=ui_inputs,
        output=output,
        status=status,
    )


Productivity Tab
The Productivity tab monitors work efficiency and break habits. The user could input their perceived productivity level or number of breaks taken. Using this, the AI offers suggestions to balance productivity with health (for instance, regular short breaks to maintain focus). This information is summarized and stored for later use in global advice.

In [56]:
from IPython.display import Markdown, display

display(Markdown(PRODUCTIVITY_INFO_MD))


# 📈 Productivity & Focus Module  
*Attention, Workflow & Efficiency Awareness*

Productivity during desk work is influenced not only by workload, but also by
attention management, task structure, and environmental distractions. Cognitive
overload, frequent interruptions, and unclear task boundaries can gradually
reduce focus and perceived efficiency.

This module is designed to support **productivity and focus awareness**, not
performance evaluation. It helps identify patterns related to attention,
distraction, and workflow that may affect daily output and mental effort.

### What this module captures
- Perceived focus quality
- Workload intensity and pacing
- Task switching frequency
- Internal and external distractions
- Energy consistency throughout the day
- Break patterns and recovery pauses
- Perceived productivity barriers

### Why this matters
Reduced productivity is often a signal of system strain rather than individual
failure. Persistent distraction and overload can increase mental fatigue and
reduce work satisfaction over time.

By identifying attention patterns and workflow challenges, this module supports
small, realistic adjustments that may improve focus and sustainability at work.

### How AI is used in this section
The AI provides **non-diagnostic, supportive guidance** focused on:
- Attention management strategies
- Structuring work to reduce cognitive overload
- Improving task flow and break timing
- Supporting sustainable productivity habits

The aim is to enhance clarity and efficiency—not to measure or judge performance.


In [57]:
# ==========================================
# Productivity Module — Config + History Load
# ==========================================




def load_productivity_history(store):
    """
    Loads the most recent saved user input + AI output for the Productivity module.
    Uses the unified load_domain_history() helper.
    """
    local_domain = "productivity"
    return load_domain_history(store, local_domain)


## Step 1 — User Inputs (Daily Productivity Signals)

This section builds the UI inputs for the Productivity tab.

The user provides:
- Focus quality and deep work hours
- Whether they achieved flow state
- Whether the most important task was completed
- Health interference factors (pain, fatigue, brain fog)
- Energy rhythm (peak energy + slump severity)
- Distraction sources and focus techniques used

These inputs will later be passed to Gemini for analysis.


In [58]:
# ==========================================
# Productivity Module — Inputs UI Builder
# ==========================================

import gradio as gr


def build_productivity_inputs(saved_user_input: dict, lang: str, locales_dir: Optional[Path]):

    gr.Markdown("### Performance Metrics")

    with gr.Row():
        focus_quality = gr.Slider(
            minimum=0, maximum=10, step=1,
            value=_to_int(saved_user_input.get("focus_quality", 5), 5),
            label=_safe_t(lang, "productivity.focus", locales_dir, default="Focus Quality (1=Scattered, 10=Laser)")
        )
        deep_work = gr.Number(
            value=_to_float(saved_user_input.get("deep_work", 0), 0.0),
            label="Deep Work Hours (Distraction-Free)"
        )

    with gr.Row():
        flow_state = gr.Checkbox(
            value=_to_bool(saved_user_input.get("flow_state", False)),
            label="Achieved 'Flow State' today?"
        )
        satisfaction = gr.Checkbox(
            value=_to_bool(saved_user_input.get("satisfaction", False)),
            label="Completed MOST important task?"
        )

    with gr.Accordion("Health Interference & Energy", open=True):
        gr.Markdown("*How much is your body holding you back?*")

        blocker_choices = ["None", "Neck/Back Pain", "Eye Fatigue", "Brain Fog", "Hunger", "Headache"]
        saved_blocker = _to_str(saved_user_input.get("blocker", "None"), "None")
        if saved_blocker not in blocker_choices:
            saved_blocker = "None"

        peak_choices = ["Early Morning", "Late Morning", "After Lunch", "Evening", "Late Night"]
        saved_peak = _to_str(saved_user_input.get("peak_energy", "Late Morning"), "Late Morning")
        if saved_peak not in peak_choices:
            saved_peak = "Late Morning"

        slump_choices = ["No slump", "Mild dip", "Total crash"]
        saved_slump = _to_str(saved_user_input.get("slump_severity", "Mild dip"), "Mild dip")
        if saved_slump not in slump_choices:
            saved_slump = "Mild dip"

        with gr.Row():
            health_tax = gr.Slider(
                minimum=0, maximum=10, step=1,
                value=_to_int(saved_user_input.get("health_tax", 0), 0),
                label="Physical Interference (Pain/Fatigue Impact)"
            )
            blocker = gr.Dropdown(
                choices=blocker_choices,
                value=saved_blocker,
                label="Primary Blocker"
            )

        with gr.Row():
            peak_energy = gr.Dropdown(
                choices=peak_choices,
                value=saved_peak,
                label="Peak Energy Window"
            )
            slump_severity = gr.Radio(
                choices=slump_choices,
                value=saved_slump,
                label="Afternoon Slump Severity"
            )

    with gr.Accordion("Habits & Distractions", open=False):

        distraction_choices = ["Notifications/Email", "Colleagues/Meetings", "Noise", "Procrastination", "Phone/Social Media"]
        saved_dist = _to_str(saved_user_input.get("top_distraction", "Notifications/Email"), "Notifications/Email")
        if saved_dist not in distraction_choices:
            saved_dist = "Notifications/Email"

        switch_choices = ["Low (Single Tasking)", "High (Multitasking)"]
        saved_switch = _to_str(saved_user_input.get("context_switching", "Low (Single Tasking)"), "Low (Single Tasking)")
        if saved_switch not in switch_choices:
            saved_switch = "Low (Single Tasking)"

        with gr.Row():
            top_distraction = gr.Dropdown(
                choices=distraction_choices,
                value=saved_dist,
                label="Top Distraction"
            )
            context_switching = gr.Radio(
                choices=switch_choices,
                value=saved_switch,
                label="Context Switching Level"
            )

        methods = gr.CheckboxGroup(
            choices=["Pomodoro Timer", "Time Blocking", "Music/White Noise", "Standing Desk", "Do Not Disturb Mode"],
            value=_to_list(saved_user_input.get("methods", [])),
            label="Techniques Used Today"
        )

    notes_input = gr.Textbox(
        value=_to_str(saved_user_input.get("notes", ""), ""),
        label=_safe_t(lang, "productivity.notes", locales_dir, default="Tasks & Observations"),
        lines=2
    )

    return {
        "focus_quality": focus_quality,
        "deep_work": deep_work,
        "flow_state": flow_state,
        "satisfaction": satisfaction,
        "health_tax": health_tax,
        "blocker": blocker,
        "peak_energy": peak_energy,
        "slump_severity": slump_severity,
        "top_distraction": top_distraction,
        "context_switching": context_switching,
        "methods": methods,
        "notes_input": notes_input,
    }


## Step 2 — Outputs

This section defines the output UI components:

- A **status box** for save/generate messages
- An **AI output text box** for Gemini recommendations

The AI output is loaded from the latest saved history so the user always sees their last generated analysis.


In [59]:
# ==========================================
# Productivity Module — Output UI Builder
# ==========================================

def build_productivity_outputs(saved_ai_text: str, lang: str, locales_dir: Optional[Path]):

    status = gr.Markdown("")

    output = gr.Textbox(
        label=_safe_t(lang, "productivity.output", locales_dir, default="Productivity Analysis"),
        interactive=False,
        value=saved_ai_text,
        lines=10
    )

    return output, status


## Step 3 — AI Interaction + Saving Logic

This section handles:

1. Collecting user inputs into a clean JSON dictionary
2. Saving user input records into `productivity_user_input.json`
3. Sending the structured data to Gemini as a prompt
4. Saving Gemini output into `productivity_ai_output.json`

All saved data is stored as a **list of timestamped records**, allowing longitudinal tracking.


In [60]:
# ==========================================
# Productivity Module — AI + Saving + Wiring (FIXED)
# ==========================================

def connect_productivity_logic(
    store,
    gemini,
    lang_state,
    locales_dir: Optional[Path],
    ui_inputs: dict,
    output: gr.Textbox,
    status: gr.Markdown,
):

    # -------------------------
    # LOCAL DOMAIN CONSTANTS
    # (prevents naming conflicts)
    # -------------------------
    local_domain = "productivity"
    local_user_file = f"{local_domain}_user_input.json"
    local_ai_file = f"{local_domain}_ai_output.json"

    # -------------------------
    # Collect user input helper
    # -------------------------
    def collect_user_input(
        focus_val, deep_val, flow_val, sat_val,
        tax_val, blocker_val, peak_val, slump_val,
        dist_val, switch_val, meth_val, notes_val
    ):
        return {
            "focus_quality": _to_int(focus_val, 5),
            "deep_work": _to_float(deep_val, 0.0),
            "flow_state": _to_bool(flow_val),
            "satisfaction": _to_bool(sat_val),
            "health_tax": _to_int(tax_val, 0),
            "blocker": _to_str(blocker_val, "None"),
            "peak_energy": _to_str(peak_val, "Late Morning"),
            "slump_severity": _to_str(slump_val, "Mild dip"),
            "top_distraction": _to_str(dist_val, "Notifications/Email"),
            "context_switching": _to_str(switch_val, "Low (Single Tasking)"),
            "methods": _to_list(meth_val),
            "notes": _to_str(notes_val, ""),
        }

    # -------------------------
    # Save user input (APPEND)
    # -------------------------
    def save_user_input(
        focus_val, deep_val, flow_val, sat_val,
        tax_val, blocker_val, peak_val, slump_val,
        dist_val, switch_val, meth_val, notes_val
    ):
        print("[productivity] 🔘 Save User Input clicked")

        try:
            user_input = collect_user_input(
                focus_val, deep_val, flow_val, sat_val,
                tax_val, blocker_val, peak_val, slump_val,
                dist_val, switch_val, meth_val, notes_val
            )

            record = {"timestamp": _now(), "user_input": user_input}

            existing = store.load_json(local_user_file, [])
            if not isinstance(existing, list):
                existing = []

            existing.append(record)
            _save_json_best_effort(store, local_user_file, existing)

            return f"✅ Productivity input saved. File: `{_debug_store_path(store, local_user_file)}`"

        except Exception as e:
            print(f"[productivity] ❌ Save failed: {e}")
            return f"❌ Save failed: {e}"

    # -------------------------
    # Generate AI output
    # -------------------------
    def generate_ai(
        focus_val, deep_val, flow_val, sat_val,
        tax_val, blocker_val, peak_val, slump_val,
        dist_val, switch_val, meth_val, notes_val
    ):
        print("[productivity] 🔘 Generate AI clicked")

        user_input = collect_user_input(
            focus_val, deep_val, flow_val, sat_val,
            tax_val, blocker_val, peak_val, slump_val,
            dist_val, switch_val, meth_val, notes_val
        )

        prompt = (
            "You are a sustainable productivity and desk-work performance assistant.\n"
            "Analyze the productivity data and provide actionable, non-diagnostic recommendations.\n"
            "1. Analyze the Health Tax and how physical discomfort affects focus.\n"
            "2. If slump severity is high, suggest a circadian-based schedule adjustment.\n"
            "3. If context switching is high but satisfaction is low, recommend a specific focus technique.\n"
            "4. Suggest one practical plan for tomorrow.\n\n"
            f"PRODUCTIVITY DATA:\n{json.dumps(user_input, indent=2, ensure_ascii=False)}"
        )

        current_lang = lang_state.value if hasattr(lang_state, "value") else lang_state

        try:
            resp = gemini.generate(prompt, response_language=current_lang)
            text = resp.text.strip() if hasattr(resp, "text") and resp.text else str(resp).strip()
        except Exception as e:
            print(f"[productivity] ❌ AI generation failed: {e}")
            return "", f"⚠️ AI generation failed: {e}"

        if not text:
            return "", "⚠️ AI returned empty output."

        return text, "✅ AI output generated (not saved yet)."

    # -------------------------
    # Save AI output (APPEND)
    # -------------------------
    def save_ai_output(text: str):
        print("[productivity] 🔘 Save AI Output clicked")

        if not (text or "").strip():
            return "⚠️ Nothing to save."

        try:
            record = {"timestamp": _now(), "ai_output": text.strip()}

            existing = store.load_json(local_ai_file, [])
            if not isinstance(existing, list):
                existing = []

            existing.append(record)
            _save_json_best_effort(store, local_ai_file, existing)

            try:
                updater = globals().get("update_domain_summary")
                if updater:
                    updater(store, local_domain, text[:200])
            except Exception as e:
                print(f"[productivity] ⚠️ update_domain_summary failed: {e}")

            return f"💾 Productivity AI output saved. File: `{_debug_store_path(store, local_ai_file)}`"

        except Exception as e:
            print(f"[productivity] ❌ Save failed: {e}")
            return f"❌ Save failed: {e}"

    # -------------------------
    # Buttons UI
    # -------------------------
    with gr.Row():
        save_input_btn = gr.Button("💾 Save User Input", variant="secondary")
        gen_btn = gr.Button("🤖 Generate AI Productivity Analysis", variant="primary")
        save_ai_btn = gr.Button("💾 Save AI Output", variant="secondary")

    inputs_list = [
        ui_inputs["focus_quality"],
        ui_inputs["deep_work"],
        ui_inputs["flow_state"],
        ui_inputs["satisfaction"],
        ui_inputs["health_tax"],
        ui_inputs["blocker"],
        ui_inputs["peak_energy"],
        ui_inputs["slump_severity"],
        ui_inputs["top_distraction"],
        ui_inputs["context_switching"],
        ui_inputs["methods"],
        ui_inputs["notes_input"],
    ]

    save_input_btn.click(fn=save_user_input, inputs=inputs_list, outputs=[status])
    gen_btn.click(fn=generate_ai, inputs=inputs_list, outputs=[output, status])
    save_ai_btn.click(fn=save_ai_output, inputs=[output], outputs=[status])

    # -------------------------
    # Demo Hook Registration (Productivity) — FIXED
    # -------------------------
    try:
        def _productivity_demo_generate():
            print("[productivity-demo] 🚀 Demo generate using saved productivity data")

            raw = store.load_json(local_user_file, [])
            if not isinstance(raw, list) or not raw:
                return "⚠️ No saved productivity demo input found.", "No data"

            last = raw[-1].get("user_input", {})
            if not isinstance(last, dict) or not last:
                return "⚠️ Saved productivity record is empty.", "No data"

            prompt = (
                "You are a preventive workplace productivity assistant.\n"
                "Analyze the following productivity, focus, and distraction data.\n"
                "Provide actionable, non-diagnostic productivity improvement recommendations.\n"
                "Focus on deep work, breaks, attention management, routines, and mental clarity.\n\n"
                f"PRODUCTIVITY DATA:\n{json.dumps(last, indent=2, ensure_ascii=False)}"
            )

            current_lang = lang_state.value if hasattr(lang_state, "value") else lang_state

            resp = gemini.generate(prompt, response_language=current_lang)
            text = resp.text.strip() if hasattr(resp, "text") else str(resp).strip()

            return text, "✅ Productivity demo output generated."

        register_demo_hook(
            domain=local_domain,
            label="Productivity Module",
            generate_fn=_productivity_demo_generate
        )

    except Exception as e:
        print(f"[productivity] ⚠️ Demo hook registration failed: {e}")


## Final Assembly — Productivity Tab Builder

This is the final wrapper function that builds the Productivity tab.

It:
- Loads the latest history
- Creates the UI inputs
- Creates the output components
- Connects buttons to AI generation and saving logic

This function is what the main app uses when registering the Productivity tab.


In [61]:
# ==========================================
# Productivity Module — Final Tab Builder (FIXED)
# ==========================================

def build_productivity_tab(
    store,
    gemini,
    lang_state: gr.State,
    locales_dir: Optional[Path] = None,
):

    local_domain = "productivity"
    local_user_file = f"{local_domain}_user_input.json"
    local_ai_file = f"{local_domain}_ai_output.json"

    print(f"[productivity] ✅ build_productivity_tab executed. user_file={local_user_file}, ai_file={local_ai_file}")

    lang = lang_state.value if hasattr(lang_state, "value") else lang_state

    saved_user_input, saved_ai_text = load_domain_history(store, local_domain)

    # --- Title ---
    gr.Markdown(
        _safe_t(
            lang,
            "productivity.title",
            locales_dir,
            default="📈 Sustainable Productivity & Focus",
        )
    )

    # --- Info Panel Button (Productivity Tab) ---
    add_info_panel(
        title="📘 Productivity Tab Guide",
        content_md=PRODUCTIVITY_INFO_MD,
        button_label="ℹ️ Productivity Info",
        open_label="❌ Hide Productivity Info",
    )

    # --- Inputs ---
    ui_inputs = build_productivity_inputs(saved_user_input, lang, locales_dir)

    # --- Outputs ---
    output, status = build_productivity_outputs(saved_ai_text, lang, locales_dir)

    # --- Logic wiring ---
    connect_productivity_logic(
        store=store,
        gemini=gemini,
        lang_state=lang_state,
        locales_dir=locales_dir,
        ui_inputs=ui_inputs,
        output=output,
        status=status,
    )


Recovery/Sleep Tab
The Recovery/Sleep tab deals with rest quality and recovery. The user can log sleep patterns or how rested they feel. The AI might provide tips for better sleep hygiene or recovery techniques if it detects poor rest. The outcome is a summary of the user's recovery status, which is saved for context.

In [62]:
from IPython.display import Markdown, display

display(Markdown(RECOVERY_SLEEP_INFO_MD))



# 😴 Recovery & Sleep Module  
*Rest, Recovery & Sleep Awareness*

Adequate recovery is essential for both physical and mental performance. Desk
work, irregular schedules, stress, and extended screen exposure—especially late
in the day—can interfere with sleep quality and overnight recovery.

This module is designed to support **recovery and sleep awareness**, not diagnosis.
It focuses on identifying rest patterns, sleep-related behaviors, and recovery
signals that influence daily energy and resilience.

### What this module captures
- Sleep duration and perceived quality
- Sleep consistency and timing
- Daytime fatigue or sleepiness
- Evening screen exposure
- Recovery practices and wind-down routines
- Use of caffeine or stimulants
- Perceived restfulness upon waking

### Why this matters
Poor sleep and inadequate recovery often accumulate gradually, affecting mood,
focus, physical comfort, and stress tolerance. These effects may be subtle at
first but can compound over time.

Early awareness of sleep and recovery patterns allows for simple behavioral
adjustments that may support long-term wellbeing and daily performance.

### How AI is used in this section
The AI provides **non-diagnostic, preventive guidance** focused on:
- Sleep hygiene and consistency
- Evening routines that support recovery
- Managing stimulants and screen exposure
- Practical steps to improve daily restoration

The goal is to promote sustainable recovery—not to replace sleep or medical care.


In [63]:
# ==========================================
# Recovery/Sleep Module — Config + History Loader (SAFE)
# ==========================================

def load_recovery_sleep_history(store):
    """
    Load the latest saved user input and AI output for the Recovery/Sleep module.
    Uses the unified load_domain_history() helper.
    """

    local_domain = "recovery_sleep"
    return load_domain_history(store, local_domain)


## Step 1 — Recovery & Sleep Inputs

This tab collects both:
### Sleep outcome metrics
- Sleep duration
- Feeling well-rested

### Evening habits that influence sleep
- Screen cutoff (1 hour before bed)
- Blue light filter usage
- Light stretching/movement

### Stress & sleep environment
- Stress level (burnout risk signal)
- Room comfort (temperature/light/noise)

These are highly relevant for desk workers because screen use, posture stiffness,
and stress dysregulation strongly impact sleep quality.


In [64]:
# ==========================================
# Recovery/Sleep Module — Inputs UI Builder
# ==========================================

import gradio as gr


def build_recovery_sleep_inputs(saved_user_input: dict, lang: str, locales_dir: Optional[Path]):

    gr.Markdown(f"### {_safe_t(lang, 'sleep.title', locales_dir, 'Recovery and Sleep Quality')}")
    gr.Markdown("Tracking habits that bridge the gap between your desk work and restorative rest.")

    with gr.Row():
        with gr.Column():
            gr.Markdown("#### Nightly Metrics")

            well_rested = gr.Checkbox(
                value=bool(saved_user_input.get("well_rested", False)),
                label=_safe_t(lang, "sleep.well_rested", locales_dir, "Feel well-rested today")
            )

            sleep_hours = gr.Radio(
                choices=["<5 hours", "5-7 hours", "7-9 hours", ">9 hours"],
                value=saved_user_input.get("sleep_hours", "7-9 hours"),
                label=_safe_t(lang, "sleep.duration", locales_dir, "Last Night's Sleep Duration")
            )

        with gr.Column():
            gr.Markdown("#### Evening Habits (from 'Healthier at the Desk')")

            screen_cutoff = gr.Checkbox(
                value=bool(saved_user_input.get("screen_cutoff", False)),
                label="Stopped screen use 1 hour before bed"
            )

            blue_light_filter = gr.Checkbox(
                value=bool(saved_user_input.get("blue_light_filter", False)),
                label="Used blue light filters/night mode on devices"
            )

            evening_movement = gr.Checkbox(
                value=bool(saved_user_input.get("evening_movement", False)),
                label="Light stretching/movement to ease desk stiffness"
            )

    with gr.Row():
        stress_level = gr.Slider(
            minimum=1, maximum=10, step=1,
            value=int(saved_user_input.get("stress_level", 5)),
            label="Workday Stress Level (1=Low, 10=Burnout Risk)"
        )

        room_comfort = gr.Dropdown(
            choices=["Cool & Dark", "Too Warm", "Too Bright", "Noisy"],
            value=saved_user_input.get("room_comfort", "Cool & Dark"),
            label="Sleep Environment"
        )

    return {
        "well_rested": well_rested,
        "sleep_hours": sleep_hours,
        "screen_cutoff": screen_cutoff,
        "blue_light_filter": blue_light_filter,
        "evening_movement": evening_movement,
        "stress_level": stress_level,
        "room_comfort": room_comfort,
    }


## Step 2 — Output Components

The output section includes:
- a status message field (save success, generation errors)
- an AI output field that displays recovery advice

This advice is meant to be actionable and non-diagnostic.


In [65]:
# ==========================================
# Recovery/Sleep Module — Output UI Builder
# ==========================================

def build_recovery_sleep_outputs(saved_ai_text: str, lang: str, locales_dir: Optional[Path]):

    status = gr.Markdown("")

    output_display = gr.Textbox(
        label=_safe_t(lang, "sleep.output", locales_dir, "AI Recovery Advice"),
        interactive=False,
        value=saved_ai_text,
        lines=10
    )

    return output_display, status


## Step 3 — AI Recovery Advice + Saving

This section defines the core workflow:

### Save User Input
Appends the daily sleep entry into a timestamped history list.

### Generate AI Advice
Uses Gemini to produce recovery guidance focused on:
- stress reduction habits
- sleep environment optimization
- screen time reduction strategies
- desk-work recovery routines for tomorrow

### Save AI Output
Stores the generated advice for future review and longitudinal tracking.


In [66]:
# ==========================================
# Recovery/Sleep Module — AI + Saving + Wiring
# ==========================================

def connect_recovery_sleep_logic(
    store,
    gemini,
    lang_state: gr.State,
    locales_dir: Optional[Path],
    ui_inputs: dict,
    output_display: gr.Textbox,
    status: gr.Markdown
):

    local_domain = "recovery_sleep"
    local_user_file = "recovery_sleep_user_input.json"
    local_ai_file = "recovery_sleep_ai_output.json"

    def collect_user_input(rested, hours, screen, blue, move, stress, room):
        return {
            "well_rested": _to_bool(rested),
            "sleep_hours": str(hours or ""),
            "screen_cutoff": _to_bool(screen),
            "blue_light_filter": _to_bool(blue),
            "evening_movement": _to_bool(move),
            "stress_level": _to_int(stress, 5),
            "room_comfort": str(room or "")
        }

    def save_user_input(rested, hours, screen, blue, move, stress, room):
        print("[recovery_sleep] 🔘 Save User Input clicked")

        try:
            user_input = collect_user_input(rested, hours, screen, blue, move, stress, room)
            record = {"timestamp": _now(), "user_input": user_input}

            existing = store.load_json(local_user_file, [])
            if not isinstance(existing, list):
                existing = []

            existing.append(record)

            # ✅ direct save (strong)
            store.save_json(local_user_file, existing)

            return f"✅ Recovery/Sleep data saved. File: `{_debug_store_path(store, local_user_file)}`"

        except Exception as e:
            print(f"[recovery_sleep] ❌ Save failed: {e}")
            return f"❌ Save failed: {e}"

    def generate_ai(rested, hours, screen, blue, move, stress, room):
        print("[recovery_sleep] 🔘 Generate AI clicked")

        user_input = collect_user_input(rested, hours, screen, blue, move, stress, room)

        prompt = (
            "You are a preventive health assistant specializing in workplace wellness.\n"
            "Analyze the following sleep and recovery data to provide actionable, "
            "non-diagnostic advice for a desk-based professional.\n\n"
            f"USER DATA:\n{user_input}\n\n"
            "INSTRUCTIONS:\n"
            "1. Focus on the connection between screen use and sleep quality.\n"
            "2. If stress is high, suggest micro-breaks or posture resets for the next day.\n"
            "3. Address the environment and pre-sleep habits mentioned in the data.\n"
            "4. Provide a simple 3-step recovery plan for tonight.\n"
        )

        current_lang = lang_state.value if hasattr(lang_state, "value") else lang_state

        try:
            resp = gemini.generate(prompt, response_language=current_lang)
            text = resp.text.strip() if hasattr(resp, "text") and resp.text else str(resp).strip()
        except Exception as e:
            print(f"[recovery_sleep] ❌ AI Error: {e}")
            return "", f"⚠️ AI Error: {e}"

        if not text:
            return "", "⚠️ AI returned empty output."

        return text, "✅ Advice generated (not saved yet)."

    def save_ai_output(text: str):
        print("[recovery_sleep] 🔘 Save AI Output clicked")

        if not (text or "").strip():
            return "⚠️ Nothing to save."

        try:
            record = {"timestamp": _now(), "ai_output": text.strip()}

            existing = store.load_json(local_ai_file, [])
            if not isinstance(existing, list):
                existing = []

            existing.append(record)

            # ✅ direct save (strong)
            store.save_json(local_ai_file, existing)

            try:
                updater = globals().get("update_domain_summary")
                if updater:
                    updater(store, local_domain, text[:200])
            except Exception as e:
                print(f"[recovery_sleep] ⚠️ update_domain_summary failed: {e}")

            return f"💾 Recovery/Sleep AI output saved. File: `{_debug_store_path(store, local_ai_file)}`"

        except Exception as e:
            print(f"[recovery_sleep] ❌ Save failed: {e}")
            return f"❌ Save failed: {e}"

    with gr.Row():
        save_input_btn = gr.Button("💾 Save User Input", variant="secondary")
        gen_btn = gr.Button("🤖 Generate AI Recovery Advice", variant="primary")
        save_ai_btn = gr.Button("💾 Save AI Output", variant="secondary")

    inputs_list = [
        ui_inputs["well_rested"],
        ui_inputs["sleep_hours"],
        ui_inputs["screen_cutoff"],
        ui_inputs["blue_light_filter"],
        ui_inputs["evening_movement"],
        ui_inputs["stress_level"],
        ui_inputs["room_comfort"],
    ]

    save_input_btn.click(fn=save_user_input, inputs=inputs_list, outputs=[status])
    gen_btn.click(fn=generate_ai, inputs=inputs_list, outputs=[output_display, status])
    save_ai_btn.click(fn=save_ai_output, inputs=[output_display], outputs=[status])

    try:
        def _recovery_sleep_demo_generate():
            print("[recovery_sleep-demo] 🚀 Demo generate using saved recovery_sleep data")

            raw = store.load_json(local_user_file, [])
            if not isinstance(raw, list) or not raw:
                return "⚠️ No saved recovery_sleep demo input found.", "No data"

            last = raw[-1].get("user_input", {})
            if not isinstance(last, dict) or not last:
                return "⚠️ Saved recovery_sleep record is empty.", "No data"

            prompt = (
                "You are a preventive workplace recovery and sleep assistant.\n"
                "Analyze the following sleep and recovery data.\n"
                "Provide actionable, non-diagnostic recommendations to improve sleep quality and recovery.\n"
                "Include screen cutoff, stress, evening routines, and room comfort.\n"
                "Mention red flags (severe insomnia, breathing issues, extreme fatigue).\n\n"
                f"SLEEP/RECOVERY DATA:\n{last}"
            )

            current_lang = lang_state.value if hasattr(lang_state, "value") else lang_state

            resp = gemini.generate(prompt, response_language=current_lang)
            text = resp.text.strip() if hasattr(resp, "text") else str(resp).strip()

            return text, "✅ Recovery/Sleep demo output generated."

        register_demo_hook(
            domain=local_domain,
            label="Recovery / Sleep Module",
            generate_fn=_recovery_sleep_demo_generate
        )

    except Exception as e:
        print(f"[recovery_sleep] ⚠️ Demo hook registration failed: {e}")


## Final Assembly — Recovery & Sleep Tab Builder

This wrapper function builds the Recovery/Sleep tab as one clean unit.

It:
- loads the latest saved data
- builds the UI inputs
- builds the AI output area
- connects button logic for save/generate/save

This modular structure is hackathon-friendly and easy to maintain.


In [67]:
# ==========================================
# Recovery/Sleep Module — Final Tab Builder
# ==========================================

def build_recovery_sleep_tab(
    store,
    gemini,
    lang_state: gr.State,
    locales_dir: Optional[Path] = None,
):

    lang = lang_state.value if hasattr(lang_state, "value") else lang_state

    saved_user_input, saved_ai_text = load_recovery_sleep_history(store)

    # --- Title ---
    gr.Markdown(
        _safe_t(
            lang,
            "sleep.title",
            locales_dir,
            default="😴 Recovery & Sleep Quality",
        )
    )

    # --- Info Panel Button (Recovery/Sleep Tab) ---
    add_info_panel(
        title="📘 Recovery & Sleep Tab Guide",
        content_md=RECOVERY_SLEEP_INFO_MD,
        button_label="ℹ️ Recovery/Sleep Info",
        open_label="❌ Hide Recovery/Sleep Info",
    )

    # --- Inputs ---
    ui_inputs = build_recovery_sleep_inputs(saved_user_input, lang, locales_dir)

    # --- Outputs ---
    output_display, status = build_recovery_sleep_outputs(saved_ai_text, lang, locales_dir)

    # --- Logic wiring ---
    connect_recovery_sleep_logic(
        store=store,
        gemini=gemini,
        lang_state=lang_state,
        locales_dir=locales_dir,
        ui_inputs=ui_inputs,
        output_display=output_display,
        status=status
    )


In [68]:
import json
from pathlib import Path


def trim_any_json_file(path: Path, keep_last: int = 5) -> dict:
    """
    If JSON file contains a LIST → trim it to last N entries.
    If JSON contains a DICT → leave it unchanged.
    If invalid → report error.
    """

    if not path.exists():
        return {"file": path.name, "status": "missing"}

    try:
        data = json.loads(path.read_text(encoding="utf-8"))
    except Exception as e:
        return {"file": path.name, "status": f"invalid_json: {e}"}

    # If dict, don't touch
    if isinstance(data, dict):
        return {"file": path.name, "status": "dict (skipped)"}

    # If list, trim
    if isinstance(data, list):
        original_len = len(data)

        if original_len <= keep_last:
            return {"file": path.name, "status": f"ok ({original_len} entries)"}

        trimmed = data[-keep_last:]

        try:
            path.write_text(
                json.dumps(trimmed, ensure_ascii=False, indent=2),
                encoding="utf-8"
            )
        except Exception as e:
            return {"file": path.name, "status": f"write_failed: {e}"}

        return {"file": path.name, "status": f"trimmed {original_len} → {len(trimmed)}"}

    # Unknown type
    return {"file": path.name, "status": f"unknown_type ({type(data).__name__})"}


def refresh_all_json_files(store, keep_last: int = 5) -> str:
    """
    Scans ALL .json files inside store.data_dir and trims list-based history files.
    """

    data_dir = Path(getattr(store, "data_dir", "data"))
    data_dir.mkdir(parents=True, exist_ok=True)

    files = sorted(list(data_dir.glob("*.json")))

    if not files:
        return "⚠️ No JSON files found."

    report = [f"### 🧹 Full Data Refresh Report (keep_last={keep_last})"]

    for f in files:
        result = trim_any_json_file(f, keep_last=keep_last)
        report.append(f"- `{result['file']}` → {result['status']}")

    return "\n".join(report)


Now the general recommendations tab or the action center it uses what ever availlable of the data from all the prevoius cells


## General Workspace Health Chat (Ask AI)

This cell defines a general-purpose, scoped AI chat for the application.

The chat allows users to ask open-ended questions related to **workspace health,
ergonomics, daily habits, and preventive wellbeing**. Unlike the structured
assessment tabs, this section supports exploratory questions while remaining
within a clearly defined, non-diagnostic scope.

Key design principles:
- The AI provides **preventive, non-diagnostic guidance only**
- Medical diagnosis or treatment advice is intentionally excluded
- Model calls are triggered **only on explicit user action**

Optional personalization:
- Users may choose to include their previously saved inputs to personalize
  responses
- Personal data is **opt-in** and loaded only when the user explicitly enables it
- No data is sent to the model automatically or without user consent

This design balances flexibility, safety, and transparency, while clearly
demonstrating responsible use of a generative model within the application.


In [69]:
from __future__ import annotations

import json
import gradio as gr
from pathlib import Path
from typing import Dict, Any


# =========================================================
# Helpers
# =========================================================
def _load_user_data_for_context(store) -> Dict[str, Any]:
    """
    Load user-input-only data from the data folder.
    Excludes AI outputs and summaries.
    """
    data_dir = Path(getattr(store, "data_dir", Path("data")))
    if not data_dir.exists():
        return {}

    context = {}
    for p in data_dir.glob("*_user_input.json"):
        try:
            context[p.stem.replace("_user_input", "")] = json.loads(
                p.read_text(encoding="utf-8")
            )
        except Exception:
            continue

    return context


# =========================================================
# Tab Builder
# =========================================================
def build_general_chat_tab(store, gemini, lang_state, locales_dir):

    lang = lang_state.value if hasattr(lang_state, "value") else lang_state

    # -------------------------
    # Header
    # -------------------------
    gr.Markdown("### 🧠 Ask AI — Workspace Health")
    gr.Markdown(
        "Ask general questions about workspace health, ergonomics, habits, and prevention. "
        "This chat provides **non-diagnostic, preventive guidance**."
    )

    # -------------------------
    # User input
    # -------------------------
    user_question = gr.Textbox(
        label="Your question",
        placeholder="e.g. How can I reduce neck pain during long desk work?",
        lines=3,
    )

    use_context = gr.Checkbox(
        label="Use my saved data to personalize the answer (optional)",
        value=False,
    )

    ask_btn = gr.Button("🧠 Ask AI")
    output = gr.Markdown("")
    status = gr.Markdown("")

    # -------------------------
    # Logic
    # -------------------------
    def ask_ai(question: str, use_ctx: bool):
        if not question or not question.strip():
            return "⚠️ Please enter a question.", ""

        # Base system context (always included)
        system_prompt = """
You are a preventive workplace health assistant.

SCOPE:
- Workspace ergonomics
- Desk-work habits
- Posture, movement, hydration, focus, recovery
- General wellbeing in office or desk-based work

RULES:
- Do NOT provide diagnosis or treatment
- Do NOT replace medical professionals
- Provide practical, preventive, non-alarming guidance
- If a question requires medical care, advise consulting a professional
"""

        # Optional personalization
        user_context_text = ""
        if use_ctx:
            user_data = _load_user_data_for_context(store)
            if user_data:
                user_context_text = f"""
OPTIONAL USER CONTEXT (for personalization only):
{json.dumps(user_data, indent=2, ensure_ascii=False)}
"""
            else:
                user_context_text = "\n(No saved user data available.)\n"

        prompt = f"""
{system_prompt}

USER QUESTION:
{question}

{user_context_text}
"""

        try:
            resp = gemini.generate(prompt=prompt, response_language=lang)
            text = resp.text.strip() if hasattr(resp, "text") else str(resp).strip()
        except Exception as e:
            return "", f"⚠️ AI request failed: {e}"

        if not text:
            return "", "⚠️ AI returned empty output."

        return text, "✅ Answer generated."

    # -------------------------
    # Wiring
    # -------------------------
    ask_btn.click(
        fn=ask_ai,
        inputs=[user_question, use_context],
        outputs=[output, status],
    )

    # -------------------------
    # Demo Hook Registration (Ask AI / General Chat Tab)
    # -------------------------
    try:
        def _ask_ai_demo_generate():
            print("[ask-ai-demo] 🚀 Demo generate using saved user input data")

            if "build_context_from_data_folder" in globals() and callable(globals()["build_context_from_data_folder"]):
                context = globals()["build_context_from_data_folder"](store)
            else:
                return "⚠️ build_context_from_data_folder() not found.", "No context builder"

            if not context or not str(context).strip():
                return "⚠️ No saved user input data found in data folder.", "No data"

            demo_question = (
                "I work long hours at a desk. Based on my saved health/workplace data, "
                "what are the top 5 things I should improve this week?"
            )

            prompt = (
                "You are a preventive workplace health assistant.\n"
                "Answer the user's question using the provided saved data.\n"
                "Be concise, practical, and non-diagnostic.\n"
                "If some data is missing, mention it gently.\n\n"
                f"USER DATA:\n{context}\n\n"
                f"USER QUESTION:\n{demo_question}\n\n"
                "Answer with:\n"
                "- 5 bullet recommendations\n"
                "- 1 short daily routine example\n"
            )

            current_lang = lang_state.value if hasattr(lang_state, "value") else lang_state

            resp = gemini.generate(prompt, response_language=current_lang)
            text = resp.text.strip() if hasattr(resp, "text") else str(resp).strip()

            return text, "✅ Ask AI demo output generated."

        register_demo_hook(
            domain="ask_ai",
            label="💬 Ask AI (General Chat)",
            generate_fn=_ask_ai_demo_generate
        )

    except Exception as e:
        print(f"[ask_ai] ⚠️ Demo hook registration failed: {e}")


# 🏢 Global Recommendations (Action Center)

This tab is the final synthesis layer of the whole platform.

It does two things:

## 1) General Recommendations
Uses all saved domain data (baseline, MSK, eye, hydration, mental, etc.)
to generate a single coherent preventive health report.

## 2) Tasks & Reminders Generator
Transforms the AI analysis into:
- daily actionable tasks
- scheduled reminders

This makes the output operational (not just advice).


In [70]:
# =================================================
# Global Recommendations Tab — Session Loader (FIXED)
# =================================================

def load_global_recommendations_state(store):
    # SECTION 1 (AI recommendations output)
    gen_domain = "general_recommendations"
    gen_ai_file = f"{gen_domain}_ai_output.json"

    raw_ai = store.load_json(gen_ai_file, [])
    ai_list = raw_ai if isinstance(raw_ai, list) else []
    latest_ai = ai_list[-1] if ai_list else {}
    last_recs_text = latest_ai.get("ai_output", "") if isinstance(latest_ai.get("ai_output", ""), str) else ""

    # SECTION 2 (Tasks & reminders stored as "user_input")
    tasks_domain = "tasks_reminders"
    tasks_user_file = f"{tasks_domain}_user_input.json"

    raw_tasks = store.load_json(tasks_user_file, [])
    tasks_list = raw_tasks if isinstance(raw_tasks, list) else []
    latest_tasks = tasks_list[-1] if tasks_list else {}
    saved_tasks_input = latest_tasks.get("user_input", {}) if isinstance(latest_tasks.get("user_input", {}), dict) else {}

    last_tasks_text = saved_tasks_input.get("tasks_list", "")
    last_reminders_text = saved_tasks_input.get("reminders_list", "")

    return last_recs_text, last_tasks_text, last_reminders_text


## Step 1 — UI Layout

This tab has two sections:

### Section 1: General Recommendations
- Displays full AI health analysis report

### Section 2: Tasks & Reminders
- Extracts daily tasks
- Extracts scheduled reminders

Both sections support saving into session history.


In [71]:
# =================================================
# Global Recommendations Tab — UI Builder
# =================================================

import gradio as gr


def build_global_recommendations_ui(last_recs_text, last_tasks_text, last_reminders_text):

    gr.Markdown("## 🏢 General Recommendations (Action Center)")
    gr.Markdown(
        "This tab synthesizes **preventive workplace health guidance** using "
        "**all data entered across the platform**."
    )

    # --- SECTION 1 ---
    with gr.Accordion("1) General Recommendations", open=True):

        rec_output = gr.Textbox(
            label="AI Health Analysis",
            lines=14,
            interactive=False,
            value=last_recs_text,
        )

        rec_status = gr.Markdown("")

        with gr.Row():
            gen_btn = gr.Button("Generate Recommendations", variant="primary")
            save_report_btn = gr.Button("Save to Text File")

    # --- SECTION 2 ---
    with gr.Accordion("2) Daily Tasks & Reminders", open=True):

        gr.Markdown("Convert your health analysis into a concrete action plan.")

        with gr.Row():
            task_output = gr.Textbox(
                label="Daily Action Tasks",
                lines=8,
                interactive=False,
                value=last_tasks_text
            )

            reminder_output = gr.Textbox(
                label="Scheduled Reminders",
                lines=8,
                interactive=False,
                value=last_reminders_text
            )

        task_status = gr.Markdown("")

        with gr.Row():
            gen_tasks_btn = gr.Button("Generate Tasks from Analysis", variant="primary")
            save_tasks_btn = gr.Button("Save Tasks & Reminders")

    return {
        "rec_output": rec_output,
        "rec_status": rec_status,
        "gen_btn": gen_btn,
        "save_report_btn": save_report_btn,
        "task_output": task_output,
        "reminder_output": reminder_output,
        "task_status": task_status,
        "gen_tasks_btn": gen_tasks_btn,
        "save_tasks_btn": save_tasks_btn,
    }


## Step 2 — AI Logic

This tab uses two AI workflows:

### A) General Recommendations Generator
Builds context from all stored user data and generates a unified report.

### B) Tasks & Reminders Generator
Uses the report text to produce:
- daily action tasks
- scheduled reminders

Then saves the outputs into session history.


In [72]:
# =================================================
# Global Recommendations Tab — AI + Saving Logic (RAW USER INPUT FIX)
# =================================================

import json
from pathlib import Path


def build_context_from_data_folder(store) -> str:
    """
    Builds context ONLY from *_user_input.json files.
    Uses the latest record in each file (list[-1]["user_input"]).
    Excludes AI outputs completely.
    """

    data_dir = Path(getattr(store, "data_dir", Path("data")))
    if not data_dir.exists():
        return ""

    # Only raw user input files
    files = sorted(data_dir.glob("*_user_input.json"))

    context_chunks = []

    for f in files:
        domain = f.stem.replace("_user_input", "")

        try:
            raw = json.loads(f.read_text(encoding="utf-8"))
        except Exception:
            continue

        # expected format: list of {"timestamp":..., "user_input": {...}}
        if isinstance(raw, list) and raw:
            last = raw[-1]
            if isinstance(last, dict) and isinstance(last.get("user_input"), dict):
                user_data = last["user_input"]
            else:
                continue

        # fallback if stored as dict
        elif isinstance(raw, dict):
            user_data = raw.get("user_input", raw)
            if not isinstance(user_data, dict):
                continue

        else:
            continue

        # Build readable context section
        context_chunks.append(
            f"## {domain.upper()} USER INPUT\n"
            + json.dumps(user_data, indent=2, ensure_ascii=False)
        )

    return "\n\n".join(context_chunks).strip()


def generate_global_recommendations(store, gemini, lang_state):
    context = build_context_from_data_folder(store)

    if not context.strip():
        return "", "⚠️ No usable user input data found yet."

    prompt = (
        "You are a preventive workplace health reasoning assistant.\n\n"
        "TASK:\n"
        "Generate a cohesive summary of risks and actionable recommendations.\n"
        "Use ONLY the provided user input data.\n\n"
        "RULES:\n"
        "- Non-diagnostic guidance only\n"
        "- Ergonomics + routines + hydration + sleep + stress + eye strain\n"
        "- Mention missing areas gently\n"
        "- Provide practical step-by-step recommendations\n\n"
        f"--- RAW USER DATA ---\n{context}"
    )

    current_lang = lang_state.value if hasattr(lang_state, "value") else lang_state

    try:
        response = gemini.generate(prompt, response_language=current_lang)
        text = response.text.strip() if response and hasattr(response, "text") else ""
    except Exception as e:
        return "", f"⚠️ AI generation failed: {e}"

    if not text.strip():
        return "", "⚠️ AI returned empty recommendations."

    # -------------------------
    # Save AI output (APPEND)
    # -------------------------
    GENERAL_REC_FILE = "general_recommendations_ai_output.json"

    try:
        record = {
            "timestamp": _now(),
            "ai_output": text.strip()
        }

        existing = store.load_json(GENERAL_REC_FILE, [])
        if not isinstance(existing, list):
            existing = []

        existing.append(record)

        _save_json_best_effort(store, GENERAL_REC_FILE, existing)

        # optional summary update
        try:
            updater = globals().get("update_domain_summary")
            if updater:
                updater(store, "general_recommendations", text[:200])
        except Exception as e:
            print(f"[general_recommendations] ⚠️ update_domain_summary failed: {e}")

        return text, f"✅ Recommendations generated and saved -> `{_debug_store_path(store, GENERAL_REC_FILE)}`"

    except Exception as e:
        return text, f"⚠️ Generated but failed to save: {e}"


def generate_tasks_and_reminders_from_analysis(gemini, lang_state, analysis_text: str):
    if not analysis_text.strip():
        return "", "", "⚠️ Please generate the Health Analysis in Section 1 first."

    prompt = (
        "Based on the following health analysis, create two distinct lists:\n"
        "1. DAILY ACTION TASKS: Specific physical actions the user should do today (e.g., 'Adjust chair height').\n"
        "2. SCHEDULED REMINDERS: Time-based triggers (e.g., 'Every 20 minutes: Look 20 feet away').\n"
        "Format the response clearly with two headers: [TASKS] and [REMINDERS].\n\n"
        f"ANALYSIS:\n{analysis_text}"
    )

    current_lang = lang_state.value if hasattr(lang_state, "value") else lang_state

    try:
        response = gemini.generate(prompt, response_language=current_lang)
        raw_text = response.text if hasattr(response, "text") else ""

        tasks = ""
        reminders = ""

        if "[TASKS]" in raw_text and "[REMINDERS]" in raw_text:
            parts = raw_text.split("[REMINDERS]")
            tasks = parts[0].replace("[TASKS]", "").strip()
            reminders = parts[1].strip()
        else:
            tasks = raw_text.strip()

        return tasks, reminders, "✅ Tasks and Reminders generated."

    except Exception as e:
        return "", "", f"⚠️ Failed to generate tasks: {e}"


def save_tasks_and_reminders(store, t_text: str, r_text: str):
    if not t_text.strip() and not r_text.strip():
        return "⚠️ Nothing to save."

    TASKS_REMINDERS_FILE = "tasks_reminders_list.json"

    payload = {
        "timestamp": _now(),
        "tasks": t_text.strip(),
        "reminders": r_text.strip(),
    }

    try:
        existing = store.load_json(TASKS_REMINDERS_FILE, [])
        if not isinstance(existing, list):
            existing = []

        existing.append(payload)

        _save_json_best_effort(store, TASKS_REMINDERS_FILE, existing)

        return f"💾 Tasks & Reminders saved -> `{_debug_store_path(store, TASKS_REMINDERS_FILE)}`"

    except Exception as e:
        return f"❌ Save failed: {e}"


In [73]:
from pathlib import Path

DATA_DIR = Path("data")
DATA_DIR.mkdir(exist_ok=True)

# create store globally
store = LocalStore(DATA_DIR)
print(build_context_from_data_folder(store))


## BASELINE USER INPUT
{
  "height": 171.0,
  "weight": 91.0,
  "bp_systolic": 130,
  "bp_diastolic": 90,
  "rhr": 75,
  "body_fat": 30.0,
  "waist_cm": 0.0,
  "activity_level": "Moderately active",
  "notes": ""
}

## EYE USER INPUT
{
  "strain_level": 6,
  "session_length": "4+ hours",
  "symptoms": [
    "Eye Twitching",
    "Headache (behind eyes)",
    "Watery Eyes"
  ],
  "lighting": "Natural Light",
  "screen_brightness": "Balanced",
  "glare": false,
  "distance_check": true,
  "correction": "None (Naked Eye)",
  "rule_20_20_20": "Occasionally",
  "used_drops": false,
  "notes": ""
}

## GENERAL_RECOMMENDATIONS USER INPUT
{
  "context_used": "data_folder"
}

## HYDRATION USER INPUT
{
  "water_intake": 6,
  "caffeine_intake": 3.0,
  "bottle_on_desk": false,
  "sugary_drinks": 3.0,
  "urine_color": "Yellow (Okay)",
  "thirst_level": "Not Thirsty",
  "symptoms": [],
  "notes": ""
}

## LONGITUDINAL USER INPUT
{
  "notes": "",
  "hb": 15,
  "wbc": 10000,
  "platelets": null,
  "glu

## Step 3 — Wiring the Buttons

This connects:
- Generate Recommendations → fills analysis output
- Generate Tasks → uses analysis output as input
- Save Tasks → saves tasks + reminders into session history


In [74]:
# =================================================
# Global Recommendations Tab — Event Wiring
# =================================================

def connect_global_recommendations_events(store, gemini, lang_state, ui):

    ui["gen_btn"].click(
        fn=lambda: generate_global_recommendations(store, gemini, lang_state),
        outputs=[ui["rec_output"], ui["rec_status"]],
    )

    ui["gen_tasks_btn"].click(
        fn=lambda analysis_text: generate_tasks_and_reminders_from_analysis(gemini, lang_state, analysis_text),
        inputs=[ui["rec_output"]],
        outputs=[ui["task_output"], ui["reminder_output"], ui["task_status"]],
    )

    ui["save_tasks_btn"].click(
        fn=lambda t_text, r_text: save_tasks_and_reminders(store, t_text, r_text),
        inputs=[ui["task_output"], ui["reminder_output"]],
        outputs=[ui["task_status"]],
    )


## Final Assembly — Global Recommendations Tab

This final function:

1. Loads latest saved outputs  
2. Builds the UI  
3. Connects AI logic + button wiring  

So the tab becomes fully modular and hackathon-clean.


In [75]:
# =================================================
# Global Recommendations Tab — Final Builder
# =================================================

from typing import Optional
from pathlib import Path
import gradio as gr


def build_global_recommendations_tab(
    store,
    gemini,
    lang_state,
    locales_dir: Optional[Path] = None,
):

    last_recs_text, last_tasks_text, last_reminders_text = load_global_recommendations_state(store)

    ui = build_global_recommendations_ui(
        last_recs_text=last_recs_text,
        last_tasks_text=last_tasks_text,
        last_reminders_text=last_reminders_text
    )

    # ============================
    # 🧹 Data Maintenance Tools
    # ============================
    with gr.Accordion("🧹 Data Maintenance Tools", open=False):
        keep_limit = gr.Number(value=5, label="Keep last N records", precision=0)
        refresh_btn = gr.Button("🔄 Refresh / Fix All JSON Files", variant="secondary")
        refresh_status = gr.Markdown("")

    refresh_btn.click(
        fn=lambda n: refresh_all_json_files(store, keep_last=int(n)),
        inputs=[keep_limit],
        outputs=[refresh_status]
    )

    # ============================
    # Main logic wiring
    # ============================
    connect_global_recommendations_events(
        store=store,
        gemini=gemini,
        lang_state=lang_state,
        ui=ui
    )


Checklist Tab
The Checklist tab provides a list of healthy tasks or habits (like "stretch every hour" or "drink water"). The user can mark items as done. When submitted, it can save the completion status and possibly update the global context summary (indicating the user has completed certain healthy actions). This helps ensure that actionable tasks are tracked and encourages consistent habits.

In [76]:
from __future__ import annotations

import json
import uuid
from pathlib import Path
from datetime import datetime
from typing import List, Dict, Any

import gradio as gr


# =========================================================
# Utilities
# =========================================================
def _now():
    return datetime.now().isoformat()


def _atomic_write(path: Path, data: Dict[str, Any]):
    path.parent.mkdir(parents=True, exist_ok=True)
    tmp = path.with_suffix(".tmp")
    with open(tmp, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)
    tmp.replace(path)


def _safe_json_extract(text: str) -> Dict[str, Any]:
    if not text:
        return {}
    text = text.strip()
    try:
        start = text.index("{")
        end = text.rindex("}") + 1
        return json.loads(text[start:end])
    except Exception:
        return {}


def _sort_items(items: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    return sorted(
        items,
        key=lambda x: (
            not x.get("priority", False),     # ⭐ first
            x.get("type") != "repetitive",    # repetitive before non-repetitive
        )
    )


def _items_to_table(items: List[Dict[str, Any]]) -> List[List[Any]]:
    items = _sort_items(items)
    return [
        [
            it.get("priority", False),
            it.get("completed_today", False),
            it.get("type", ""),
            it.get("item", ""),
        ]
        for it in items
    ]


# =========================================================
# Tab Builder (Notebook-safe)
# =========================================================
def build_checklist_tab(store, gemini, lang_state, locales_dir):

    # -------------------------
    # LOCAL CONSTANTS (SAFE)
    # -------------------------
    local_domain = "checklist"
    data_path = Path("data") / f"{local_domain}.json"
    summary_key = "global_context_summary"
    tasks_reminders_file = "tasks_reminders_list.json"

    def _load_raw_checklist() -> List[str]:
        """
        Load checklist tasks from data/tasks_reminders_list.json
        Uses the latest entry only.
        """
        data = store.load_json(tasks_reminders_file, []) or []

        if not isinstance(data, list) or not data:
            return []

        latest = data[-1]
        tasks_text = latest.get("tasks", "")

        if not tasks_text:
            return []

        return [
            line.lstrip("*").strip()
            for line in tasks_text.splitlines()
            if line.strip()
        ]

    # -------------------------
    # Load saved data
    # -------------------------
    saved = store.load_json(f"{local_domain}.json", {}) or {}
    checklist_items = saved.get("items", [])

    summary_obj = store.load_json(f"{summary_key}.json", {}) or {}
    summary_time = summary_obj.get("generated_at")
    summary_text = summary_obj.get("summary", "")

    raw_checklist = _load_raw_checklist()

    # -------------------------
    # UI Header
    # -------------------------
    gr.Markdown("### ✅ Daily Checklist")
    gr.Markdown("Build a practical, prioritized checklist from your approved tasks.")

    # =====================================================
    # 1) Source checklist
    # =====================================================
    with gr.Accordion("1) Source checklist (from tasks & reminders)", open=False):
        gr.Markdown(
            "\n".join(f"- {x}" for x in raw_checklist)
            if raw_checklist
            else "_No tasks available yet._"
        )

    # =====================================================
    # 2) Data summary
    # =====================================================
    with gr.Accordion("2) Data summary (used for prioritization)", open=True):
        summary_status = gr.Markdown(
            f"Using summary generated on **{summary_time}**"
            if summary_time
            else "_No summary generated yet._"
        )
        btn_summary = gr.Button("Generate / Refresh data summary")

    # =====================================================
    # 3) Optional user context
    # =====================================================
    user_context = gr.Textbox(
        label="Optional context for today",
        placeholder="e.g. very busy day, low energy, back pain today",
        lines=2,
    )

    # =====================================================
    # 4) Generate checklist
    # =====================================================
    btn_generate = gr.Button("Generate my checklist")

    checklist_state = gr.State(checklist_items)

    checklist_df = gr.Dataframe(
        headers=["⭐", "Done", "Type", "Item"],
        datatype=["bool", "bool", "str", "str"],
        interactive=True,
        row_count=(0, "dynamic"),
    )

    btn_delete = gr.Button("🗑️ Delete selected row")
    status = gr.Markdown("")

    # =====================================================
    # Logic
    # =====================================================
    def generate_summary():
        prompt = """
Summarize all saved workplace health data.

Rules:
- Patterns only
- No advice
- No diagnosis
- JSON only

FORMAT:
{ "summary": "text" }
"""
        current_lang = lang_state.value if hasattr(lang_state, "value") else lang_state

        resp = gemini.generate(prompt, response_language=current_lang)
        data = _safe_json_extract(resp.text if hasattr(resp, "text") else "")

        if "summary" not in data:
            return "⚠️ Failed to generate summary."

        payload = {
            "generated_at": _now(),
            "summary": data["summary"],
        }

        store.save_json(f"{summary_key}.json", payload)
        return f"✅ Summary generated on {payload['generated_at']}"

    def generate_checklist(user_ctx):
        if not raw_checklist:
            return [], "⚠️ No source checklist."

        current_lang = lang_state.value if hasattr(lang_state, "value") else lang_state

        # -------------------------
        # Classification
        # -------------------------
        class_prompt = f"""
Classify checklist items into:
- non_repetitive
- repetitive

Return JSON only.

FORMAT:
{{"non_repetitive":[], "repetitive":[]}}


ITEMS:
{json.dumps(raw_checklist, ensure_ascii=False)}
"""

        classified = _safe_json_extract(
            gemini.generate(class_prompt, response_language=current_lang).text
        )

        items: List[Dict[str, Any]] = []

        for it in classified.get("non_repetitive", []):
            items.append({
                "id": str(uuid.uuid4()),
                "item": it,
                "type": "non_repetitive",
                "priority": False,
                "completed_today": False,
            })

        for it in classified.get("repetitive", []):
            items.append({
                "id": str(uuid.uuid4()),
                "item": it,
                "type": "repetitive",
                "priority": False,
                "completed_today": False,
            })

        # -------------------------
        # ⭐ Prioritization
        # -------------------------
        rep_items = [i["item"] for i in items if i["type"] == "repetitive"]

        if rep_items:
            pr_prompt = f"""
You are assisting a workplace health platform.

TASK:
From the list of repetitive habits below, select a SMALL daily focus set.

STRICT RULES:
- Select ONLY from the provided habits.
- Do NOT rewrite or add habits.
- Prefer low-effort, high-impact actions.
- Max 5 items.
- JSON only.

FORMAT:
{{
  "prioritized_habits": [
    {{ "item": "", "reason": "" }}
  ]
}}

REPETITIVE HABITS:
{json.dumps(rep_items, indent=2, ensure_ascii=False)}

OPTIONAL CONTEXT SUMMARY:
{summary_text}

USER CONTEXT:
{user_ctx}
"""
            pr_data = _safe_json_extract(
                gemini.generate(pr_prompt, response_language=current_lang).text
            )

            starred = {
                x["item"]
                for x in pr_data.get("prioritized_habits", [])
                if isinstance(x, dict) and "item" in x
            }

            for it in items:
                if it["item"] in starred:
                    it["priority"] = True

        items = _sort_items(items)
        _atomic_write(data_path, {"updated_at": _now(), "items": items})

        return _items_to_table(items), "✅ Checklist generated with priorities."

    def update_from_table(rows):
        if rows is None:
            return checklist_state.value

        rows = rows.values.tolist()
        items = checklist_state.value

        if len(rows) != len(items):
            return items

        for row, item in zip(rows, items):
            item["priority"] = bool(row[0])
            item["completed_today"] = bool(row[1])

        items = _sort_items(items)
        _atomic_write(data_path, {"updated_at": _now(), "items": items})
        return items

    def delete_selected(rows):
        if rows is None:
            return checklist_state.value, "⚠️ No selection."

        rows = rows.values.tolist()
        if not rows:
            return checklist_state.value, "⚠️ No selection."

        items = checklist_state.value
        items.pop(-1)  # safe fallback deletion

        items = _sort_items(items)
        _atomic_write(data_path, {"updated_at": _now(), "items": items})
        return items, "🗑️ Item deleted."

    # =====================================================
    # Wiring
    # =====================================================
    btn_summary.click(generate_summary, outputs=[summary_status])

    btn_generate.click(
        generate_checklist,
        inputs=[user_context],
        outputs=[checklist_df, status],
    )

    checklist_df.change(
        update_from_table,
        inputs=[checklist_df],
        outputs=[checklist_state],
    )

    btn_delete.click(
        delete_selected,
        inputs=[checklist_df],
        outputs=[checklist_state, status],
    )

    checklist_state.change(
        lambda items: _items_to_table(items),
        inputs=[checklist_state],
        outputs=[checklist_df],
    )


Reminders Tab
The Reminders tab allows the scheduling of recurring health reminders (for example, a reminder to stand up and stretch every 60 minutes). The user can set the interval or schedule (via sliders or checkboxes) and the app will save this schedule (though actual reminder notifications might require external integration). This tab mainly records the reminder preferences and confirms they are saved.

In [77]:
# =========================================================
# Tab Builder (Notebook-safe)
# =========================================================
def build_reminders_tab(store, gemini, lang_state, locales_dir):

    # -------------------------
    # LOCAL CONSTANTS (SAFE)
    # -------------------------
    local_domain = "reminders"
    tasks_reminders_file = "tasks_reminders_list.json"
    google_cal_import_url = "https://calendar.google.com/calendar/u/0/r/settings/import"

    def _safe_json_extract(text: str) -> Dict[str, Any]:
        if not text:
            return {}
        text = text.strip()
        try:
            start = text.index("{")
            end = text.rindex("}") + 1
            return json.loads(text[start:end])
        except Exception:
            return {}

    def _load_raw_reminders() -> List[str]:
        data = store.load_json(tasks_reminders_file, []) or []

        if not isinstance(data, list) or not data:
            return []

        latest = data[-1]
        reminders_text = latest.get("reminders", "")

        if not reminders_text:
            return []

        return [
            line.lstrip("*").strip()
            for line in reminders_text.splitlines()
            if line.strip()
        ]

    def _load_schedule():
        path = Path("data") / "reminder_schedule.json"
        if not path.exists():
            return None
        return json.loads(path.read_text(encoding="utf-8"))

    def _schedule_to_rows(schedule_json):
        if not schedule_json:
            return []
        rows = []
        for s in schedule_json.get("schedule", []):
            reminders = " + ".join(
                r.get("text", "") for r in s.get("combined_reminders", [])
            )
            rows.append([
                s.get("time", ""),
                ", ".join(s.get("days", [])),
                reminders
            ])
        return rows

    # =====================================================
    # UI
    # =====================================================
    gr.Markdown("### ⏰ Smart Reminders")
    gr.Markdown("Plan low-noise, merged workplace reminders based on your tasks and habits.")

    raw_reminders = _load_raw_reminders()

    reminder_df = gr.Dataframe(
        headers=["Keep", "Reminder"],
        datatype=["bool", "str"],
        value=[[True, r] for r in raw_reminders],
        interactive=True,
        row_count=(0, "dynamic"),
        label="Imported reminder candidates",
    )

    gr.Markdown("Uncheck any reminder you do not want to include. You are always in control.")

    gr.Markdown("### 🕒 Work Schedule")

    work_days = gr.CheckboxGroup(
        choices=["Mon", "Tue", "Wed", "Thu", "Fri"],
        value=["Mon", "Tue", "Wed", "Thu", "Fri"],
        label="Work days",
    )

    with gr.Row():
        start_time = gr.Textbox(label="Work start time", value="08:00")
        end_time = gr.Textbox(label="Work end time", value="16:00")

    merge_window = gr.Slider(
        minimum=5,
        maximum=30,
        step=5,
        value=10,
        label="Merge window (± minutes)",
    )

    btn_generate = gr.Button("Generate smart reminder schedule")
    status = gr.Markdown("")

    def generate_schedule(df, days, start, end, window):
        if df is None:
            return "⚠️ No reminders available.", []

        rows = df.values.tolist()
        selected = [r[1] for r in rows if r[0] is True]

        if not selected:
            return "⚠️ No reminders selected.", []

        prompt = f"""
You are assisting a workplace health platform.

TASK:
Create a merged, low-noise reminder schedule for a desk-based workday.

STRICT OUTPUT RULES:
- Return JSON ONLY
- Do NOT include explanations, markdown, or text outside JSON
- Do NOT include code fences

REQUIRED FORMAT:
{{
  "schedule": [
    {{
      "time": "HH:MM",
      "days": ["Mon","Tue","Wed","Thu","Fri"],
      "combined_reminders": [
        {{ "text": "reminder text" }}
      ]
    }}
  ]
}}

REMINDERS:
{json.dumps(selected, indent=2, ensure_ascii=False)}

WORK SCHEDULE:
Days: {days}
Start: {start}
End: {end}

MERGE WINDOW:
±{window} minutes
"""

        current_lang = lang_state.value if hasattr(lang_state, "value") else lang_state

        resp = gemini.generate(prompt, response_language=current_lang)
        data = _safe_json_extract(resp.text if hasattr(resp, "text") else "")

        if not data or "schedule" not in data:
            return "⚠️ Failed to generate reminder schedule (invalid model output).", []

        store.save_json("reminder_schedule.json", data)
        return "✅ Reminder schedule generated and saved.", _schedule_to_rows(data)

    gr.Markdown("### 📅 Reminder Schedule Preview")

    initial_data = _load_schedule()
    initial_rows = _schedule_to_rows(initial_data)

    schedule_view = gr.Dataframe(
        headers=["Time", "Days", "Reminders"],
        datatype=["str", "str", "str"],
        value=initial_rows,
        interactive=False,
        row_count=(0, "dynamic"),
    )

    status_view = gr.Markdown("")

    with gr.Row():
        btn_run = gr.Button("▶️ Run reminders in app (demo)")
        btn_export = gr.Button("📅 Export as calendar (.ics)")
        btn_google = gr.Button("📆 Send to Google Calendar")

    def run_reminders_demo():
        data = _load_schedule()
        if not data:
            return "⚠️ No schedule to run."

        schedule = data.get("schedule", [])

        def reminder_loop():
            fired = set()
            while True:
                now = datetime.now()
                now_str = now.strftime("%H:%M")
                for s in schedule:
                    if s.get("time") == now_str and now_str not in fired:
                        text = " | ".join(
                            r.get("text", "") for r in s.get("combined_reminders", [])
                        )
                        print(f"⏰ REMINDER: {text}")
                        fired.add(now_str)
                time.sleep(30)

        threading.Thread(target=reminder_loop, daemon=True).start()
        return "▶️ Demo reminders running (app must stay open)."

    def export_ics():
        data = _load_schedule()
        if not data:
            return "⚠️ No schedule to export."

        now = datetime.now()
        lines = ["BEGIN:VCALENDAR", "VERSION:2.0", "PRODID:-//Healthier at the Desk//EN"]

        for s in data.get("schedule", []):
            try:
                hour, minute = map(int, s["time"].split(":"))
                event_time = now.replace(hour=hour, minute=minute, second=0)
                summary = " + ".join(
                    r.get("text", "") for r in s.get("combined_reminders", [])
                )
                lines.extend([
                    "BEGIN:VEVENT",
                    f"DTSTART:{event_time.strftime('%Y%m%dT%H%M%S')}",
                    f"DTEND:{(event_time + timedelta(minutes=5)).strftime('%Y%m%dT%H%M%S')}",
                    f"SUMMARY:{summary}",
                    "END:VEVENT",
                ])
            except Exception:
                continue

        lines.append("END:VCALENDAR")
        out = Path("data") / "reminder_schedule.ics"
        out.parent.mkdir(parents=True, exist_ok=True)
        out.write_text("\n".join(lines), encoding="utf-8")
        return f"📅 Calendar file exported: {out.name}"

    def open_google_calendar_import():
        return (
            "✅ Calendar file ready.\n\n"
            "1️⃣ Click the link below\n"
            "2️⃣ Import the exported .ics file\n"
            "3️⃣ Choose your Google Calendar\n\n"
            f"👉 {google_cal_import_url}"
        )

    btn_generate.click(
        generate_schedule,
        inputs=[reminder_df, work_days, start_time, end_time, merge_window],
        outputs=[status, schedule_view],
    )

    btn_run.click(run_reminders_demo, outputs=[status_view])
    btn_export.click(export_ics, outputs=[status_view])
    btn_google.click(open_google_calendar_import, outputs=[status_view])


Context Tab
The Context tab gives the user insight into the summarized context data from each domain. It may list or allow selection of different health domains and display the short summaries (that were generated and saved via update_domain_summary). This helps the user understand what information the AI has "remembered" about each area. It might also allow clearing or refreshing context for a domain if needed.

In [78]:
from __future__ import annotations

import json
import gradio as gr
from pathlib import Path


def build_context_tab(store, gemini, lang_state: gr.State, locales_dir: Path):
    lang = lang_state.value if hasattr(lang_state, "value") else lang_state

    gr.Markdown(t(lang, "context.title", locales_dir, default="Context Memory Viewer"))

    # 🔹 Added explanatory line (safe, non-intrusive)
    gr.Markdown(
        t(
            lang,
            "context.description",
            locales_dir,
            default="This section shows what the system currently remembers and allows selective reset.",
        )
    )

    domain_to_clear = gr.Dropdown(
        choices=[
            "baseline",
            "longitudinal",
            "msk",
            "mental",
            "eye",
            "hydration",
            "productivity",
        ],
        label=t(lang, "context.domain_reset", locales_dir, default="Domain to Reset"),
    )

    confirm = gr.Checkbox(
        label=t(lang, "context.confirm_delete", locales_dir, default="Confirm Deletion")
    )

    status = gr.Textbox(
        label=t(lang, "context.status", locales_dir, default="Status / Current Context"),
        interactive=False,
    )

    def on_refresh():
        ctx = load_context(store)
        return json.dumps(ctx, ensure_ascii=False, indent=2)

    def on_clear(domain, confirm):
        if not domain or not confirm:
            return t(
                lang,
                "context.select_confirm",
                locales_dir,
                default="Select a domain and check confirm to clear.",
            )
        delete_domain(store, domain)
        return t(
            lang,
            "context.cleared",
            locales_dir,
            default=f"Cleared context for {domain}.",
            domain=domain,
        )

    refresh_btn = gr.Button(t(lang, "common.refresh", locales_dir, default="Refresh"))
    clear_btn = gr.Button(t(lang, "common.clear", locales_dir, default="Clear"))

    refresh_btn.click(
        fn=on_refresh,
        inputs=[],
        outputs=[status],
    )

    clear_btn.click(
        fn=on_clear,
        inputs=[domain_to_clear, confirm],
        outputs=[status],
    )


Reports Tab
The Reports tab can generate comprehensive reports based on the user’s data. For example, the user might request a "weekly report" or "summary report" from all logged information. The tab will gather data from all relevant JSON files (or use context summaries) and then prompt the AI to produce a structured report (e.g., highlighting progress, issues, and suggestions). This report is displayed to the user as text.

In [79]:
from __future__ import annotations

import gradio as gr


def build_reports_tab(store, gemini, lang_state: gr.State, locales_dir):
    lang = lang_state.value if hasattr(lang_state, "value") else lang_state

    gr.Markdown(t(lang, "reports.title", locales_dir, default="Generate Health Report"))

    # 🔹 Added explanatory line (safe, non-intrusive)
    gr.Markdown(
        t(
            lang,
            "reports.description",
            locales_dir,
            default="Generate high-level summaries from accumulated health context.",
        )
    )

    report_type = gr.Dropdown(
        choices=["Daily", "Weekly", "Monthly"],
        label=t(lang, "reports.type", locales_dir, default="Report Type"),
    )

    output = gr.Textbox(
        label=t(lang, "reports.output", locales_dir, default="Generated Report"),
        interactive=False,
    )

    def on_submit(report_type):
        # Load all context summaries
        ctx = store.load_json("context_summaries.json", default={}) or {}
        prompt = f"Generate a {report_type.lower()} report from the following context summaries: {ctx}"
        resp = gemini.generate(prompt=prompt, response_language=lang_state.value)
        text = resp.text if hasattr(resp, "text") else str(resp)
        return text

    gen_btn = gr.Button(t(lang, "common.generate", locales_dir, default="Generate"))
    gen_btn.click(fn=on_submit, inputs=[report_type], outputs=[output])


Settings Tab
The Settings tab provides controls for application settings. In this skeleton, it includes the ability to clear all stored data. If the user confirms, the tab deletes the entire data directory (wiping all saved user data) and recreates an empty structure. This is useful for privacy or starting over. (It could also be extended to include language selection or other settings as needed.)

In [80]:
from __future__ import annotations

import gradio as gr
from pathlib import Path
import shutil


def build_settings_tab(store, gemini, lang_state: gr.State, locales_dir: Path):
    lang = lang_state.value if hasattr(lang_state, "value") else lang_state
    gr.Markdown("### " + t(lang, "settings.title", locales_dir, default="Settings") + "\n\n" + t(lang, "settings.subtitle", locales_dir, default="Local-only settings for this app."))

    clear_confirm = gr.Checkbox(
        label=t(lang, "settings.clear_confirm", locales_dir, default="Clear all data (this will delete all saved data!)")
    )

    status = gr.Textbox(
        label=t(lang, "common.status", locales_dir, default="Status"),
        interactive=False,
    )

    def on_clear(confirm):
        if not confirm:
            return t(lang, "settings.prompt_confirm", locales_dir, default="Please check the box to confirm data clearance.")

        # Delete the entire data directory
        shutil.rmtree(store.data_dir)
        store.data_dir.mkdir(parents=True, exist_ok=True)

        return t(lang, "settings.cleared", locales_dir, default="All data cleared.")

    clear_btn = gr.Button(t(lang, "common.clear", locales_dir, default="Clear"))
    clear_btn.click(
        fn=on_clear,
        inputs=[clear_confirm],
        outputs=[status],
    )


Help Tab
The Help tab displays instructions and information on how to use the application. It contains static Markdown text explaining the features of the app, guidance on using each tab, and reassurance about privacy (e.g., that data is stored locally). This helps users understand the purpose and usage of the tool.

In [81]:
from __future__ import annotations
import gradio as gr
from pathlib import Path

def build_help_tab(store, gemini, lang_state: gr.State, locales_dir: Path):
    lang = lang_state.value if hasattr(lang_state, "value") else lang_state
    gr.Markdown(t(lang, "help.md", locales_dir, default="""### Help

**Important Notes:**

- **Privacy:** All your inputs and data are stored *locally* on your device. Nothing is sent to any server (unless you enable the Gemini AI with an API key, which then sends prompts to the model).
- **Using the Tabs:** Each tab focuses on a different aspect of your health or the app's functionality. Fill in the inputs and click the submit/generate buttons to interact.
- **AI Recommendations:** The suggestions provided by the AI (Gemini) are not medical advice. They are for informational purposes. Always consult a professional for serious health concerns.
- **Extensibility:** This is a starter skeleton. You can extend each tab or integrate an actual AI API for more personalized and accurate insights.

Use the Settings tab to clear your data anytime. We hope this tool helps you maintain a healthier workday!"""))

Launching the Application
Now that all components (configuration, core modules, and tabs) have been defined in the notebook, we can launch the Gradio application. Normally, running app.py directly would start the server. In this notebook, we manually construct the app and launch it.
Running the code below will instantiate the Gradio interface and launch it on the default local server (by default at 127.0.0.1:7860). In a Jupyter environment, Gradio will either display an interactive widget or provide a link to open the interface in a browser.

In [82]:
import gradio as gr
from pathlib import Path


def build_app():
    """
    Assemble the Gradio application.
    Assumes all tab builder functions and shared objects
    (TAB_BUILDERS, t(), LocalStore, GeminiClient, etc.) already exist.
    """

    # -----------------------------
    # Local data store (defined ONCE here)
    # -----------------------------
    DATA_DIR = Path("data")
    DATA_DIR.mkdir(exist_ok=True)
    store = LocalStore(DATA_DIR)

    # -----------------------------
    # Gemini / LM client (defined ONCE here)
    # -----------------------------
    client = GeminiClient()

    LOCALES_DIR = Path("locales")

    # Build-time language (UI labels are rendered at build time)
    lang = DEFAULT_LANG if "DEFAULT_LANG" in globals() else "en"

    TAB_TITLE_KEYS = {
        "Baseline": "tab.baseline",
        "Workspace": "tab.workspace",
        "Longitudinal": "tab.longitudinal",
        "MSK": "tab.msk",
        "Eye": "tab.eye",
        "Mental": "tab.mental",
        "Hydration": "tab.hydration",
        "Productivity": "tab.productivity",
        "Recovery / Sleep": "tab.recovery_sleep",

        # Optional localization support
        "Checklist": "tab.checklist",
        "Reminders": "tab.reminders",

        "Context": "tab.context",
        "Reports": "tab.reports",
        "Settings": "tab.settings",
        "Help": "tab.help",
    }

    with gr.Blocks(
        title=t(
            lang,
            "app.title",
            LOCALES_DIR,
            default=APP_NAME if "APP_NAME" in globals() else "Workday Health Reasoning Platform",
        )
    ) as demo:

        # Language state (used for model response language)
        lang_state = gr.State(lang)

        # -----------------------------
        # App header
        # -----------------------------
        gr.Markdown(
            f"# {t(lang, 'app.title', LOCALES_DIR, default='Workday Health Reasoning Platform')}"
        )
        gr.Markdown(
            t(
                lang,
                "app.subtitle",
                LOCALES_DIR,
                default="Privacy-first, local-only, context-aware health assistant for desk workers.",
            )
        )
        gr.Markdown(
            t(
                lang,
                "msg.local_only",
                LOCALES_DIR,
                default="All data is stored locally on this device. Nothing is uploaded unless you call the model.",
            )
        )

        # -----------------------------
        # App-level introduction (collapsible)
        # -----------------------------
        with gr.Accordion("ℹ️ About this app", open=False):
            gr.Markdown(ABOUT_APP_MD)

        # -----------------------------
        # Main tabs
        # -----------------------------
        # Support notebooks where TAB_BUILDERS is defined in a different cell.
        TAB_BUILDERS_LOCAL = globals().get("TAB_BUILDERS", [])
        with gr.Tabs():
            for title, builder in TAB_BUILDERS_LOCAL:
                key = TAB_TITLE_KEYS.get(title)
                tab_label = t(lang, key, LOCALES_DIR, default=title) if key else title

                with gr.TabItem(tab_label):
                    builder(
                        store=store,
                        gemini=client,
                        lang_state=lang_state,
                        locales_dir=LOCALES_DIR,
                    )

    return demo


In [83]:
# -------------------------------------------------
# Tab registry (notebook version)
# Ordered for clarity, reasoning flow, and judging
# -------------------------------------------------

TAB_BUILDERS = [
    # 1️⃣ Action Center (cross-domain synthesis)
    ("General Recommendations", build_global_recommendations_tab),
    ("Ask AI", build_general_chat_tab),


    # 2️⃣ Core user inputs
    ("Baseline", build_baseline_tab),
    ("Workspace", build_workspace_tab),
    ("Longitudinal", build_longitudinal_tab),

    # 3️⃣ Health domains
    ("MSK", build_msk_tab),
    ("Eye", build_eye_tab),
    ("Mental", build_mental_tab),
    ("Hydration", build_hydration_tab),
    ("Productivity", build_productivity_tab),
    ("Recovery / Sleep", build_recovery_sleep_tab),

    # 4️⃣ Actions & planning  ✅ NEW
    ("Checklist", build_checklist_tab),
    ("Reminders", build_reminders_tab),

    # 5️⃣ Transparency & outputs
    ("Context", build_context_tab),
    ("Reports", build_reports_tab),

    # 6️⃣ System / support
    ("Settings", build_settings_tab),
    ("Help", build_help_tab),
]


In [84]:
# Build and launch the Gradio app (this will start a local server for the UI)
app = build_app()
app.launch(server_name="127.0.0.1",  show_error=True)

[LocalStore] ✅ JSON loaded: F:\research projects\AI\HealthierAtDesk_SUBMIT\data\general_recommendations_ai_output.json
[LocalStore] ✅ JSON loaded: F:\research projects\AI\HealthierAtDesk_SUBMIT\data\tasks_reminders_user_input.json
[baseline] ✅ build_baseline_tab executed. user_file=baseline_user_input.json, ai_file=baseline_ai_output.json
[LocalStore] ✅ JSON loaded: F:\research projects\AI\HealthierAtDesk_SUBMIT\data\baseline_user_input.json
[LocalStore] ✅ JSON loaded: F:\research projects\AI\HealthierAtDesk_SUBMIT\data\baseline_ai_output.json
[baseline] load user=baseline_user_input.json ai=baseline_ai_output.json
[workspace] ✅ build_workspace_tab executed. user_file=workspace_user_input.json, ai_file=workspace_ai_output.json
[LocalStore] ✅ JSON loaded: F:\research projects\AI\HealthierAtDesk_SUBMIT\data\workspace_user_input.json
[LocalStore] ✅ JSON loaded: F:\research projects\AI\HealthierAtDesk_SUBMIT\data\workspace_ai_output.json
[workspace] load user=workspace_user_input.json ai=w

After launching, you should see the UI accessible at the above address. You can interact with all the tabs as described. All data you input will be saved to the data directory locally, and you can stop the server by interrupting the notebook kernel or closing the app.